#### Solution processing---> Choose folder and run all cells below

In [1]:
# os.user

In [30]:
import os
import ipywidgets as widgets
import pandas as pd
 
proj_dir = 'Y:/Modelling/Ukraine/2023_UKR_ST_Security/'
gen_params_path = 'I:/DOCS/RISE-GER/UKR/Power_system_modernisation/Data/2023_10_20_generator_parameters_UKR.xlsx'

soln_idx = pd.read_excel(gen_params_path,  sheet_name='SolutionIndex', engine='openpyxl')
# tech_info_idx = pd.read_excel(gen_params_path,  sheet_name='Indice', engine='openpyxl')
full_soln_idx = soln_idx.copy()

rootdir = proj_dir + '04_SolutionFiles/'
soln_dirs = [f for f in os.listdir(rootdir) if os.path.isdir(os.path.join(rootdir, f)) & (f != 'archived')]
soln_dirs.sort(reverse=True)

soln_choice = widgets.Dropdown(
                options=soln_dirs,
                value=soln_dirs[0],
                description='Model runs:',
                disabled=False,)

In [2]:
# co2_targets = pd.read_excel('C:/Users/HART_C/International Energy Agency/EMS-RISE - Ukraine/Ukraine report 2022/02_Data/01_Generation/CarbonTarget_v2_edit.xlsx').set_index('Year').iloc[:,0]

In [3]:
idn_style_idx = False
geo_cols = ['Region', 'Subregion']

In [4]:
display(soln_choice)

Dropdown(description='Model runs:', options=('test',), value='test')

`You can now run all with the above model runs being processed`

#### Functions

In [9]:
import pandas as pd
import numpy as np
import sys, os
import geopandas as gpd
import time

import matplotlib.pyplot as plt

In [6]:
import os

def get_files(root_folder, file_type, id_text, subfolder="", return_type=0):
    """Basic function to walk through folder and return all files of a certain type containing specific text in its name. 
    Can return either a list of full paths or two lkists odf directories and filenames seperately depending on the argument
    return type =0/1"""
    
    searched_files = []
    searched_file_paths = []
    searched_files_fullpath = []
    
    folder = os.path.join(root_folder, subfolder)
    
    for dirpath, subdirs, files in os.walk(folder):
        for file in files:
            if (file.endswith(file_type)) & (id_text in file) :
                searched_files_fullpath.append(os.path.join(os.path.normpath(dirpath), file))
                searched_files.append(file)
                searched_file_paths.append(os.path.normpath(dirpath))

    if return_type==0:
        return searched_files_fullpath
    else:
        return searched_file_paths, searched_files

#### Import Julia stuff....

Not sure on the difference between Main or Base and other variants of starting Julia/PyCall

In [15]:
# ## If running for the first time!!

import julia
julia.install()

JuliaNotFound: Julia executable `julia` cannot be found.

If you have installed Julia, make sure Julia executable is in the
system PATH.  Alternatively, specify file path to the Julia executable
using `julia` keyword argument.

If you have not installed Julia, download Julia from
https://julialang.org/downloads/ and install it.


In [16]:
# from julia import Main
from julia import Base
from julia.api import Julia
from julia import JuliaError

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [10]:
jl = Julia(compiled_modules=False)

In [11]:
jl.using("H5PLEXOS")

Add paths and get list of solution files and folders to process

In [23]:
soln_folder = os.path.join(rootdir, soln_choice.value)

### Just use the same name in the DaaProcessing section, to save changing 2 names
save_dir = proj_dir + '05_DataProcessing/{}/'.format(soln_choice.value) 

In [24]:
save_dir

'Y:/Modelling/Ukraine/2023_UKR_ST_Security/05_DataProcessing/test/'

In [25]:
soln_dirs, soln_files = get_files(soln_folder, file_type='.zip', id_text='Solution', return_type=1)
existing_h5_files = get_files(soln_folder, file_type='.h5', id_text='Solution', return_type=0)

In [26]:
soln_files = [s.replace('\\','/') for s in soln_files]
h5_files = [s.replace('.zip', '.h5').replace('\\','/') for s in soln_files]

In [27]:
soln_files

['Model IDN_2021_Validation Solution.zip']

#### Process using H5PLEXOS.jl
Run code to check if h5 file exists already, and if not to run H5PLEXOS.jl

In [28]:
for i, f in enumerate(soln_files):
    jl.eval("cd(\"{}\")".format(soln_dirs[i].replace('\\','/')))
    if (os.path.join(soln_dirs[i], h5_files[i])) not in existing_h5_files:
        try:
            jl.eval("process(\"{}\", \"{}\")".format(soln_files[i], h5_files[i]))
        except JuliaError:
            print("Falied to process {} with {}".format(soln_files[i]))

NameError: name 'jl' is not defined

In [18]:
"cd(\"{}\")".format(soln_dirs[i].replace('\\','/'))

'cd("Z:/Ukraine/2022-3_next_step_modelling/04_SolutionFiles/20230622_UKR_VREPlus_FINAL")'

### Process H5 solution files

In [21]:
from h5plexos.query import PLEXOSSolution

#### Some parts of the solution index were added specifically for the Indonesia project.....
There is a need to generalise some these parts//
For example we could read in the cost data as oer technology from the Indices tab[['name', 'Object_type']+ geo_cols + ['CapacityCategory', 'Category' ]]

In [34]:
### Memory allocation errors occur due to large size of arrays, mainly due to string columns of dataframes. This way, we can limit this
gen_inertia_idx = full_soln_idx[['name', 'InertiaLOW','InertiaHI']]
idn_style_idx = True

soln_idx = full_soln_idx[['name', 'Object_type']+ geo_cols + ['CapacityCategory', 'Category']]

if idn_style_idx == True:
    gen_cost_idx = full_soln_idx[['name', 'VOM', 'FOM', 'RampCost']]
    line_info_idx = full_soln_idx[['name', 'nodeFrom', 'nodeTo', 'regFrom', 'regTo', 'islFrom', 'islTo']]
    gen_addl_idx = full_soln_idx[['name', 'FlexCategory', 'CostCategory','CofiringCategory','Cofiring', 'CCUS', 'IPP']]
else:
    gen_cost_idx = full_soln_idx[['name']]
    line_info_idx = full_soln_idx[['name']]
    gen_addl_idx =  full_soln_idx[['name']]
    


In [35]:
##  Check if save_dir exists....otherwise make new directory
if os.path.exists(save_dir) is False:
    os.mkdir(save_dir)

### Get properties per object class in PLEXOS file
This currently assumes that the properties will be the same across all files. 

There may also be a need to change things for different intervals (i.e. year, month, day) and MT/PASA if used

In [36]:
h5_full_path = get_files(soln_folder, file_type='.h5', id_text='Solution', return_type=0)

In [37]:
with PLEXOSSolution(h5_full_path[-1]) as db:
    plexos_objects = list(db.h5file['data/ST/interval/'].keys())

Querying H5PLEXOS v0.6.2 file


#### Read in solution file objects and properties using the first solution file

In [38]:
### This may be housed under a 'with'
with PLEXOSSolution(h5_full_path[-1]) as db:
    plexos_objects = list(db.h5file['data/ST/interval/'].keys())
    plexos_props = {}
    plexos_props_yr = {}
    for o in plexos_objects:
        plexos_props[o] = list(db.h5file['data/ST/interval/{}/'.format(o)].keys())
        plexos_props_yr[o] = list(db.h5file['data/ST/year/{}/'.format(o)].keys())
        
    ### As Zone (or sometimes region) usually has only one object, this can be used to get time_idx
    time_idx = db.region("Load").reset_index().timestamp.drop_duplicates()     

Querying H5PLEXOS v0.6.2 file


#### Properties to filter for interval results

In [39]:
### As time-series data can be quite large, filter only certain data
filter_props = {}
filter_out_objs = ['constraints', 'contingencyreserves_regions', 'decisionvariables', 'storages', 'timeslices', 'variables']


In [40]:
filter_props['generators'] = ['Available Capacity', 'Generation', 'Installed Capacity','Min Energy Violation', 'Units',
                             'Units Generating','Units Out', 'Units Started', 'Forced Outage', 'Maintenance', 'Firm Capacity']

In [41]:
filter_props['batteries'] = ['Age', 'Energy', 'Generation', 'Hours Charging', 'Hours Discharging', 'Hours Idle', 'Installed Capacity', 'Load', 'Losses', 'Lower Reserve', 'Net Generation', 'Raise Reserve',
 'Regulation Lower Reserve', 'Regulation Raise Reserve', 'SoC', 'Generation Capacity']

In [42]:
filter_props['regions'] = ['Capacity Reserve Margin', 'Capacity Reserves', 'Customer Load', 'Dump Energy', 'Forced Outage', 
                'Generation', 'Generation Capacity', 'Load', 'Maintenance', 'Native Load', 'Pump Load', 'Battery Load',
                'Transmission Losses', 'Unserved Energy', 'Unserved Energy Hours', 'Price', 'Shadow Price', 'SRMC'] # 'Exports', 'Imports'

In [43]:
filter_props['nodes'] = ['Customer Load', 'Exports','Generation', 'Generation Capacity', 
              'Imports', 'Load', 'Min Load', 'Native Load', 'Peak Load', 'Price', 'Pump Load', 'Battery Load', 'Unserved Energy',  'Price', 'Shadow Price', 'SRMC']

In [44]:
filter_props['lines'] = ['Export Limit', 'Flow', 'Import Limit', 'Loading',  'Loss',  'Units']

In [45]:
# ## differ by emission_gens or plain emissions
# filter_props['emissions_generators'] = ['Cost', 'Intensity', 'Generator Production', ]

In [46]:
filter_props['emissions'] = ['Intensity', 'Price', 'Production', 'Shadow Price']

In [47]:
filter_props['fuels'] = ['Cost', 'Offtake', 'Price', 'Time-weighted Price']

In [48]:
filter_props['reserves'] = ['Shortage', 'Risk']
filter_props['reserves_generators'] = ['Available Response', 'Cleared Offer Cost', 'Cleared Offer Price',  'Non-spinning Reserve Provision',
                 'Provision']


### Analysis and processing

In [49]:
### Errors may arise from using input as output. Check back if weird behavior

def add_df_column(df, column_name, value, reset_index=True): 
    
    if type(df) == pd.Series:
        out_df=pd.DataFrame(df)
    else:
        out_df = df.copy()
        
    if column_name in out_df.columns:
        print('Updating /"{}/" column with new values')
        out_df.loc[:,column_name] = value
    else:
        out_df[column_name] = value
        
    if reset_index:
        out_df = out_df.reset_index().rename(columns={0:'value'})
    
    return out_df

#### Cycle through all solution files in folder and all specified properties
1. Note that this should be kept to the minimum for speed purposes.
2. Additional performance benefits may be obtained from seperate querying of interval/yearly data (currently all done in one)

### Interval

In [50]:
try: 

    del gen_df, res_gen_df, em_gen_df, em_df, res_df, reg_df, node_df, line_df, fuel_df, zone_df
    del gen_yr_df,  em_gen_yr_df, em_yr_df, res_yr_df, reg_yr_df, node_yr_df, line_yr_df, fuel_yr_df, zone_yr_df
except:
    print('DNE')

DNE


In [51]:
obj_df_dict = {}

for f in h5_full_path:
    
    ### Any spaces in the file will break it
    soln_name = f.split('\\')[-1].split('Model ')[-1].split(' Solution.h5')[0]

    print("Processing {}...".format(soln_name))
    
    with PLEXOSSolution(f) as db:
        
        plexos_objs = [ p for p in db.h5file['data/ST/interval/'].keys() if p not in filter_out_objs] #['variables']#
        plexos_props = {}
        for o in plexos_objs:
            plexos_props[o] = list(db.h5file['data/ST/interval/{}/'.format(o)].keys())
        
        for o in plexos_objs:
            
            ### New method is to populate as you run through the PLEXOS dbs
            ### If its the first loop, create the empty dataframe to concatanate
            if o not in obj_df_dict.keys():
                obj_df_dict[o] = pd.DataFrame(None)
            
            ### Filter properties for time-series data
            ### If filter props is not defined, all properties are used
            try:
                props = [ p for p in plexos_props[o] if p in filter_props[o]]
            except KeyError:
                props = plexos_props[o]
                
            
            for prop in props:          
                
                ### Relations (i.e. membership-related props) have underscores in them
                if '_' not in o:
                    ## object class is queried without the 's' at the end of its name (this may be a bug that is correccted in future versions)
                    o_key = o[:-1]
                    obj_df_dict[o] = pd.concat(
                    [obj_df_dict[o], 
                     add_df_column(
                         df = db.query_object_property(
                             object_class=o_key, prop = prop, timescale="interval", phase="ST").reset_index(),
                         column_name='model',
                         value=soln_name)], axis=0)
                else:
                    obj_df_dict[o] = pd.concat(
                    [obj_df_dict[o], 
                     add_df_column(
                         df = db.query_relation_property(
                         relation=o, prop = prop, timescale="interval", phase="ST").reset_index(),
                         column_name='model',
                         value=soln_name)], axis=0)


Processing IDN_2021_Validation...
Querying H5PLEXOS v0.6.2 file


#### Strip unnecessary columns (e.g. bands), rename '0' column to 'Value'  & add SolutionIndex 

### Annual

In [52]:
obj_df_yr_dict = {}

for f in h5_full_path:
    
    ### Any spaces in the file will break it
    soln_name = f.split('\\')[-1].split('Model ')[-1].split(' Solution.h5')[0]

    print("Processing {}...".format(soln_name))
    
    with PLEXOSSolution(f) as db:
        
        plexos_objs = [ p for p in db.h5file['data/ST/interval/'].keys() if p not in filter_out_objs] #['variables']#
        plexos_props = {}
        for o in plexos_objs:
            plexos_props[o] = list(db.h5file['data/ST/year/{}/'.format(o)].keys())
        
        for o in plexos_objs:
            
            ### New method is to populate as you run through the PLEXOS dbs
            ### If its the first loop, create the empty dataframe to concatanate
            if o not in obj_df_yr_dict.keys():
                obj_df_yr_dict[o] = pd.DataFrame(None)
            
            ### No need to filter annual
            props = plexos_props[o]
            
            for prop in props:          
                
                ### Relations (i.e. membership-related props) have underscores in them
                if '_' not in o:
                    ## object class is queried without the 's' at the end of its name (this may be a bug that is correccted in future versions)
                    o_key = o[:-1]
                    obj_df_yr_dict[o] = pd.concat(
                    [obj_df_yr_dict[o], 
                     add_df_column(
                         df = db.query_object_property(
                             object_class=o_key, prop = prop, timescale="year", phase="ST").reset_index(),
                         column_name='model',
                         value=soln_name)], axis=0)
                else:
                    obj_df_yr_dict[o] = pd.concat(
                    [obj_df_yr_dict[o], 
                     add_df_column(
                         df = db.query_relation_property(
                         relation=o, prop = prop, timescale="year", phase="ST").reset_index(),
                         column_name='model',
                         value=soln_name)], axis=0)


Processing IDN_2021_Validation...
Querying H5PLEXOS v0.6.2 file


In [53]:
def enrich_df(df,  soln_idx, common_yr = None, out_type='direct',pretty_model_names={}):
    
    ### Output can relative type (i.e. emissions from generators) or direct type (i.e. just emissions)
    if out_type=='rel':
        df = df.rename(columns={0:'value'})[[ 'parent', 'child', 'property', 'timestamp', 'model', 'value']]
        ## Add soln idx
        df = pd.merge(df,soln_idx, left_on='child', right_on='name')
    else:
        df = df.rename(columns={0:'value'})[['name', 'property', 'timestamp', 'model', 'value']]
        ## Add soln idx
        df = pd.merge(df,soln_idx, left_on='name', right_on='name')
    
    if common_yr:
    ### Add common year date .... will in future need to add line for removing leap days
        df.loc[:,'timestamp'] = pd.to_datetime({'Year':[common_yr]*len(df), 'Month':df.timestamp.dt.month.values, 'Day': df.timestamp.dt.day.values,
                                                            'Hour':df.timestamp.dt.hour.values, 'Minute':df.timestamp.dt.minute.values})
    else:
        df.loc[:,'timestamp'] = df.timestamp

    
    try:
        df.loc[:,'model'] = df.model.apply(lambda x: pretty_model_names[x] if x in pretty_model_names.keys() else x.split('Model ')[-1].split(' Solution.h5')[0])
    except:
        print('Error re-defining model names')
    
    return df

#### Strip unnecessary columns (e.g. bands), rename '0' column to 'Value'  & add SolutionIndex 

In [54]:
### No leap years....but this could become a factor soon
model_yrs = obj_df_dict['regions'].groupby(['model']).first().timestamp.dt.year.values
if len(model_yrs) > 1:
    common_yr = model_yrs[-1]
else:
    common_yr = None

In [55]:
def enrich_df(df,  soln_idx, common_yr = None, out_type='direct',pretty_model_names={}):
    
    ### Output can relative type (i.e. emissions from generators) or direct type (i.e. just emissions)
    if out_type=='rel':
        df = df.rename(columns={0:'value'})[[ 'parent', 'child', 'property', 'timestamp', 'model', 'value']]
        ## Add soln idx
        df = pd.merge(df,soln_idx, left_on='child', right_on='name')
    else:
        df = df.rename(columns={0:'value'})[['name', 'property', 'timestamp', 'model', 'value']]
        ## Add soln idx
        df = pd.merge(df,soln_idx, left_on='name', right_on='name')
    
    if common_yr:
    ### Add common year date .... will in future need to add line for removing leap days
        df.loc[:,'timestamp'] = pd.to_datetime({'Year':[common_yr]*len(df), 'Month':df.timestamp.dt.month.values, 'Day': df.timestamp.dt.day.values,
                                                            'Hour':df.timestamp.dt.hour.values, 'Minute':df.timestamp.dt.minute.values})
    else:
        df.loc[:,'timestamp'] = df.timestamp

    
    try:
        df.loc[:,'model'] = df.model.apply(lambda x: pretty_model_names[x] if x in pretty_model_names.keys() else x.split('Model ')[-1].split(' Solution.h5')[0])
    except:
        print('Error re-defining model names')
    
    return df

In [56]:
### Prettify names

pretty_model_names = { 
#         'UKR22_2021_Validation': '2021 Validation',
#         'UKR22_2025_Base': '2025 Base',
#         'UKR22_2030_Base': '2030 Base',
#         'UKR22_2037_Base': '2037 Base',
#         'UKR22_2040_Base': '2040 Base',
    
    
    
#         'UKR22_2037_EVFlex': '2037 (Smart Charging)',
#         'UKR22_2025_NoTOP': '2025 (Contractual Flex)',
#         'UKR22_2030_NoTOP': '2030 (Contractual Flex)',
#         'UKR22_2030plus_NoTOP': '2030 VRE Plus (Contractual Flex)',
#         'UKR22_2037_EVFlex_EVRampPen': '2037 (Smart Charging, limited ramping)',
#         'UKR22_2037_NoTOP': '2037 (Contractual Flex)',
#         'UKR22_2037plus_NoTOP': '2037 VRE Plus',
#         'UKR22_2037plus_NoTOP_EVFlex': '2037 VRE Plus (Smart Charging)',
#         'UKR22_2037plus_NoTOP_EVFlex_EVRampPen': '2037 (ContrFlex, Smart Charging, limited ramping)',
#         'UKR22_2030plus_NoTOP_Batt_CO2cap': '2030 (Battery & CO2 price)',
#         'UKR22_2037plus_NoTOP_EVFlex_Batt': '2037 (Smart Charging & Battery)',
#         'UKR22_2037plus_NoTOP_EVFlex_Batt_CO2cap': '2037 (Smart Charging, Battery & CO2 price)'
    
    
        'UKR22_2021_Validation' : '2021 Validation',
        'UKR22_2025_Base' : '2025 Base',
    
    
        'UKR22_2030_Base' : '2030 Base',
        'UKR22_2030plus': '2030 VRE Plus',
        'UKR22_2030plus_CO2p': '2030 VRE Plus + CO2p',

        'UKR22_2030plus_PPflex': '2030 VRE Plus + PPFlex',
        'UKR22_2030plus_Batt': '2030 VRE Plus + Sto',
        'UKR22_2030plus_Batt_PPflex': '2030 VRE Plus + PPFlex + Sto',
        'UKR22_2030plus_Batt_NoTOP_PPflex': '2030 VRE Plus + PPFlex + Sto + ContrFlex',


        'UKR22_2037_Base' : '2037 Base',
        'UKR22_2037plus': '2037 VRE Plus',
        'UKR22_2037plus_CO2p': '2037 VRE Plus + CO2p',
        'UKR22_2037plus_Batt': '2037 VRE Plus + Sto',
        'UKR22_2037plus_Batt_CO2p': '2037 VRE Plus + Sto + CO2p',
        'UKR22_2037plus_EVFlex_Batt_PPflex': '2037 VRE Plus + PPFlex + Sto + EV',
        'UKR22_2037plus_EVFlex_Batt_PPflex_CO2p': '2037 VRE Plus + PPFlex + Sto + EV + CO2p',

        'UKR22_2037plus_PPflex': '2037 VRE Plus + PPFlex',
        'UKR22_2037plus_EVFlex': '2037 VRE Plus + EV',
        'UKR22_2037plus_EVFlex_Batt_PPflex': '2037 VRE Plus + PPFlex + Sto + EV',
        'UKR22_2037plus_EVFlex_Batt_NoTOP_PPflex': '2037 VRE Plus + PPFlex + Sto + EV + ContrFlex',
      
 
        'UKR22_2030plus_Batt_NoTOP': '2030 VRE Plus + Sto + ContrFlex',
        'UKR22_2037plus_EVFlex_Batt': '2037 VRE Plus + Sto + EV',
        'UKR22_2037plus_EVFlex_Batt_NoTOP': '2037 VRE Plus + Sto + ContrFlex',
        "UKR22_2037plus_EVFlex25": '2037 VRE Plus + EV25',
        "UKR22_2037plus_EVFlex50": '2037 VRE Plus + EV50',
        "UKR22_2037plus_Batt25": '2037 VRE Plus + Sto25',
        "UKR22_2037plus_Batt50": '2037 VRE Plus + Sto50',
        "UKR22_2037plus_EVFlex25_Batt_NoTOP_PPflex": '2037 VRE Plus + Sto + EV25 + ContrFlex',
        "UKR22_2037plus_EVFlex25_Batt_PPflex": '2037 VRE Plus + Sto + EV25',
        "UKR22_2037plus_EVFlex50_Batt_NoTOP_PPflex": '2037 VRE Plus + Sto + EV50 + ContrFlex',
        "UKR22_2037plus_EVFlex50_Batt_PPflex": '2037 VRE Plus + Sto + EV50',

}


#### Filter out regions with no generation or load!
With the expansion of the model for NZE, there are now many regions which have zero load. This step serves to save memory and plot space


In [57]:
possible_geo_cols = [geo_cols[0], 'Zone',  geo_cols[1], 'Node']
geo_cols = [c for c in soln_idx.columns if c in possible_geo_cols]

In [58]:
filter_reg_by_gen = enrich_df(obj_df_yr_dict['generators'], soln_idx=soln_idx[soln_idx.Object_type.str.lower() == 'generator'].drop(columns='Object_type'), pretty_model_names=pretty_model_names)
filter_reg_by_load =  enrich_df(obj_df_yr_dict['nodes'], soln_idx=soln_idx[soln_idx.Object_type.str.lower() == 'node'].drop(columns='Object_type'), pretty_model_names=pretty_model_names)
### This is subregion in Indonesia. -1 is where the finest geo resolution would/should be found
filter_col = geo_cols[-1]


### Filter out nodes that have zero load
filter_reg_by_load = filter_reg_by_load[(filter_reg_by_load.property == 'Load')&(filter_reg_by_load.value != 0)]

filter_reg_by_gen = filter_reg_by_gen[geo_cols[-1]].unique()
filter_reg_by_load = filter_reg_by_load[geo_cols[-1]].unique()

####

filter_regs = list(set([reg for reg in filter_reg_by_gen] + [reg for reg in filter_reg_by_load] ))


In [59]:
for o in obj_df_dict.keys():

    o_key = o.replace('ies','ys')
        
    if '_' in o_key:
    ### Memebrship properties can used the generator/battery part which is the second part
    ### Also o_key is for singular form, hence drops s
        o_key = o_key.split('_')[-1][:-1]
    else:
        o_key = o_key[:-1]        
    
    ### Remove unnecessary columns, so object_type can be removed for the o_idx
    o_idx = soln_idx[soln_idx.Object_type.str.lower().str.replace(' ','')== o_key].drop(columns='Object_type')
    if len(o_idx) > 0:
        if '_' not in o:
            obj_df_dict[o] = enrich_df(obj_df_dict[o], soln_idx=o_idx, common_yr=common_yr, out_type='direct', pretty_model_names=pretty_model_names)
        else:
            obj_df_dict[o] = enrich_df(obj_df_dict[o], soln_idx=o_idx, common_yr=common_yr, out_type='rel', pretty_model_names=pretty_model_names)
        
        ### Filter out regions with no generation nor load
        if (o == 'nodes')|(o == 'regions'):
            obj_df_dict[o] = obj_df_dict[o][obj_df_dict[o][filter_col].isin(filter_regs)]

for o in obj_df_yr_dict.keys():
    
    o_key = o.replace('ies','ys')
        
    if '_' in o_key:
    ### Memebrship properties can used the generator/battery part which is the second part
    ### Also o_key is for singular form, hence drops s
        o_key = o_key.split('_')[-1][:-1]
    else:
        o_key = o_key[:-1]

    ### No need to filter out solnb_idx for the annual data as the size wont be an issue
    o_idx = full_soln_idx[full_soln_idx.Object_type.str.lower().str.replace(' ','')== o_key].drop(columns='Object_type')
    if len(o_idx) > 0:
        if '_' not in o:
            obj_df_yr_dict[o] = enrich_df(obj_df_yr_dict[o], soln_idx=o_idx, common_yr=common_yr, out_type='direct', pretty_model_names=pretty_model_names)
        else:
            obj_df_yr_dict[o] = enrich_df(obj_df_yr_dict[o], soln_idx=o_idx, common_yr=common_yr, out_type='rel', pretty_model_names=pretty_model_names)
        
        ### Filter out regions with no generation nor load
        if (o == 'nodes')|(o == 'regions'):
            obj_df_yr_dict[o] = obj_df_yr_dict[o][obj_df_yr_dict[o][filter_col].isin(filter_regs)]


### Here assign to more traditional variable names

In [60]:
# o_idx = soln_idx[soln_idx.Object_type.str.lower() == 'line']
# line_exporting_reg = geo_cols[0]
# line_importing_reg = geo_cols[1]
# line_df = line_df[['category', 'name', 'property', 'timestamp', 'model', 'value']]
# line_yr_df = line_yr_df[['category', 'name', 'property', 'timestamp', 'model', 'value']]

# line_df = enrich_df(line_df, soln_idx=o_idx, common_yr=common_yr, out_type='direct', pretty_model_names=pretty_model_names).rename(columns={line_exporting_reg:'regFrom', line_importing_reg:'regTo'})
# line_yr_df = enrich_df(line_yr_df, soln_idx=o_idx, common_yr=common_yr, out_type='direct', pretty_model_names=pretty_model_names).rename(columns={line_exporting_reg:'regFrom', line_importing_reg:'regTo'})

In [61]:
# x = obj_df_yr_dict['batteries']
# x[(x.name.str.contains('Battery'))&(x.property=='Age')]

In [62]:
obj_df_yr_dict['batteries'] = obj_df_yr_dict['batteries'].replace('Installed Capacity', 'Storage Capacity').replace('Generation Capacity', 'Installed Capacity')
obj_df_dict['batteries'] = obj_df_dict['batteries'].replace('Installed Capacity', 'Storage Capacity').replace('Generation Capacity', 'Installed Capacity')

KeyError: 'batteries'

In [ ]:
gen_yr_df = obj_df_yr_dict['generators']


try:
    
    gen_yr_df = pd.concat([gen_yr_df, obj_df_yr_dict['batteries']], axis=0)
except KeyError:
    print("No batteries for current scenarios")

try:
    purch_yr_df = obj_df_yr_dict['purchasers']
except KeyError:
    purch_yr_df = pd.DataFrame(None)
    
try:
    var_yr_df = obj_df_yr_dict['variables']
except KeyError:
    var_yr_df = pd.DataFrame(None)
    
try:
    fuelcontract_yr_df = obj_df_yr_dict['fuelcontracts']
except KeyError:
    fuelcontract_yr_df = pd.DataFrame(None)
try:
    em_fuel_yr_df = obj_df_yr_dict['emissions_fuels']
except KeyError:
    em_fuel_yr_df = pd.DataFrame(None)

try:
    line_yr_df = obj_df_yr_dict['lines']
except KeyError:
    line_yr_df = pd.DataFrame(None)

    
    
reg_yr_df = obj_df_yr_dict['regions']
node_yr_df = obj_df_yr_dict['nodes']
em_yr_df = obj_df_yr_dict['emissions']
fuel_yr_df = obj_df_yr_dict['fuels']

try:
    res_yr_df = obj_df_yr_dict['reserves']
except KeyError:
    res_yr_df = pd.DataFrame(None)
    
zone_yr_df =obj_df_yr_dict['zones']
fuelcontract_yr_df = obj_df_yr_dict['fuelcontracts']


### Emissions generators is missing....maybe good to have some fail safe so all downstream stuff can work, regardless. i.e. placeholder empty data
em_gen_yr_df =obj_df_yr_dict['emissions_generators']

In [ ]:
gen_df =obj_df_dict['generators']
reg_df = obj_df_dict['regions']
node_df = obj_df_dict['nodes']
em_df = obj_df_dict['emissions']
fuel_df = obj_df_dict['fuels']

zone_df =obj_df_dict['zones']

try:
    line_df = pd.merge(obj_df_dict['lines'], line_info_idx, on='name')
except KeyError:
    line_df = pd.DataFrame(None)
    
try:
    purch_df = obj_df_dict['purchasers']
except KeyError:
    purch_df = pd.DataFrame(None)
    
    
try:
    var_df = obj_df_dict['variables']
except KeyError:
    var_df = pd.DataFrame(None)  
    
try:
    res_df = obj_df_dict['reserves']
except KeyError:
    res_df = pd.DataFrame(None)

## Note different structure for objects derived from memberships
em_gen_df = obj_df_dict['emissions_generators']

try:
    res_gen_df = obj_df_dict['reserves_generators']
except:
    res_gen_df = pd.DataFrame(None)
    
try:
    
    gen_df = pd.concat([gen_df, obj_df_dict['batteries']], axis=0)
except KeyError:
    print("No batteries objects")
    
try:
    res_gen_df = pd.concat([res_gen_df, obj_df_dict['reserves_batteries']], axis=0)
except KeyError:
    print("No reserve batteries objects")


In [ ]:
#### For WEO_tech simpl. probably should add something to soln_idx
# gen_yr_df.loc[:,'WEO_Tech_simpl'] = gen_yr_df['WEO tech'].apply(lambda x: x.replace('NEW ','').replace(' 1','').replace(' 2','').replace(' 3','').replace(' 4','').replace(' 5','')   )

In [ ]:
### Prevent double dosing in memory

del obj_df_dict
# del obj_df_yr_dict

#### Strip unnecessary columns (e.g. bands), rename '0' column to 'Value'  & add SolutionIndex 

In [ ]:
# ### Drop addl columns
# gen_yr_df = gen_yr_df[[ 'name', 'property', 'timestamp', 'model', 'value',
#        geo_cols[0], geo_cols[1], geo_cols[0], 'CapacityCategory',
#        'Category',]]
# gen_df = gen_df[['name', 'property', 'timestamp', 'model', 'value',
#        geo_cols[0], geo_cols[1], geo_cols[0], 'CapacityCategory',
#        'Category',]]
# res_gen_df = res_gen_df[['parent', 'child', 'property', 'timestamp', 'model', 'value',
#        geo_cols[0], geo_cols[1], geo_cols[0], 'CapacityCategory',
#        'Category',]]

In [ ]:
# ### Cofiring change here! 
# ### Due to memory allocation errors, additional columns from soln_idx are used and then discarded
# cofiring_scens = [c for c in pretty_model_names.values() if ('2030' in c)| (c == '2025 Base') | (c == '2025 Enforced Cofiring')]


# # Add additional columns for interval dataframes (these are not added at first due to memory issues with the big size of the df)
# gen_df = pd.merge(gen_df, gen_addl_idx[['name', 'Cofiring', 'CofiringCategory']], on='name', how='left')
# res_gen_df = pd.merge(res_gen_df, gen_addl_idx[['name', 'Cofiring', 'CofiringCategory']], left_on='child', right_on='name',how='left')


# ### Add category
# gen_yr_df.loc[(gen_yr_df.Cofiring=='Y')&(gen_yr_df.model.isin(cofiring_scens)), 'Category'] = gen_yr_df.loc[(gen_yr_df.Cofiring=='Y')&(gen_yr_df.model.isin(cofiring_scens)), 'CofiringCategory']
# gen_df.loc[(gen_df.Cofiring=='Y')&(gen_df.model.isin(cofiring_scens)), 'Category'] = gen_df.loc[(gen_df.Cofiring=='Y')&(gen_df.model.isin(cofiring_scens)), 'CofiringCategory']
# res_gen_df.loc[(res_gen_df.Cofiring=='Y')&(res_gen_df.model.isin(cofiring_scens)), 'Category'] = res_gen_df.loc[(res_gen_df.Cofiring=='Y')&(res_gen_df.model.isin(cofiring_scens)), 'CofiringCategory']

# ### And capacity category
# gen_yr_df.loc[(gen_yr_df.Cofiring=='Y')&(gen_yr_df.model.isin(cofiring_scens)), 'CapacityCategory'] = gen_yr_df.loc[(gen_yr_df.Cofiring=='Y')&(gen_yr_df.model.isin(cofiring_scens)), 'CofiringCategory']
# gen_df.loc[(gen_df.Cofiring=='Y')&(~gen_df.model.isin(cofiring_scens)), 'CapacityCategory'] = gen_df.loc[(gen_df.Cofiring=='Y')&(~gen_df.model.isin(cofiring_scens)), 'CofiringCategory']
# res_gen_df.loc[(res_gen_df.Cofiring=='Y')&(res_gen_df.model.isin(cofiring_scens)), 'CapacityCategory'] = res_gen_df.loc[(res_gen_df.Cofiring=='Y')&(res_gen_df.model.isin(cofiring_scens)), 'CofiringCategory']

# ### Drop addl columns for interval df
# gen_df = gen_df.drop(columns=['Cofiring', 'CofiringCategory'])
# res_gen_df = res_gen_df.drop(columns=['Cofiring', 'CofiringCategory'])


#### What do we want to reproduce

1. a) Annual load by region
   b) Load time-series (net load?)
2. Unserved energy by region
3. Operating costs and variants (e.g. ToP constraints as fuel costs, incl. penalty costs, etc.) by property
4. a) Generation by technology/region
   b) Generation by plant
5. Unit starts by technology
6. a) Gen max by region/technology
   b) Sum of regional gen max by technology
7. Transmission losses
8. VRE availability (absolute & normalised) by tech/combined, VRE gen vs the rest (VRE share)
9. VRE undispatched by tech/region or summed by tech
10. a) Line flows per line/interface
    b) Line capacity per interface (import/export)
    c) Line flow time-series per interface (as % of capacity?)
11. a) CFs per technology/region
    b) CFs per tech only
12. Installed cap per region/technology

.... additional may include ramping, emissions, heat rate by generator, SRMC by generator, fuel offtake by generator

#### Generic information for sharing across calculations

Model info

In [ ]:
interval_periods = len(time_idx)
nr_days = len(time_idx.dt.date.drop_duplicates())
daily_periods = interval_periods/nr_days
hour_corr = 24/daily_periods

In [ ]:
## In past models, nodes were effectively regions while there was only a single geo_cols[0] modelled. 
## In later models, Zones are this single entity, while Regions represent control regions and 
## nodes are smaller subentities in those regions (e.g. provinces)
## node_regs is therefore a flag whether regions should be based on Nodes or Regions

node_regs = False
model_regs = list(gen_yr_df.Region.unique())

if node_regs:
    
    load_regs = list(node_yr_df[(node_yr_df.property == 'Load')&(node_yr_df.value != 0)].name.unique())
else:
    load_regs = list(reg_yr_df[(reg_yr_df.property == 'Load')&(reg_yr_df.value != 0)].name.unique())

In [ ]:
model_names = list(np.sort(reg_df.model.drop_duplicates()))


## Annual summary data

In [ ]:
save_dir_sum = os.path.join(save_dir,'summary_out')

In [ ]:
if os.path.exists(save_dir_sum) is False:
    os.mkdir(save_dir_sum)

In [ ]:
ix = pd.IndexSlice

In [ ]:
gen_cap_actuals = pd.read_excel(gen_params_path, sheet_name='AllGenerators', usecols='A:AX', engine='openpyxl', skiprows=2)
#
gen_cap_actuals.loc[:,'Historical (2021)'] = gen_cap_actuals.MaxCap_2021*gen_cap_actuals.Units_2021
gen_cap_actuals.loc[:,'Input_2025'] = gen_cap_actuals.MaxCap_2025*gen_cap_actuals.Units_2025
gen_cap_actuals.loc[:,'Input_2030'] = gen_cap_actuals.MaxCap_2030*gen_cap_actuals.Units_2030
gen_cap_actuals.loc[:,'Input_2037'] = gen_cap_actuals.MaxCap_2037*gen_cap_actuals.Units_2037

gen_cap_actuals.loc[:,'Input2030_NNTp2025'] = gen_cap_actuals.Input_2030
gen_cap_actuals.loc[:,'Input2037_NNTp2025'] = gen_cap_actuals.Input_2037
#
thermal_Tech = ['Gas', 'Coal', 'Oil']
gen_cap_actuals.loc[(gen_cap_actuals.Tech_simple.isin(thermal_Tech))&(gen_cap_actuals.BuiltP2025 == True),'Input2030_NNTp2025'] = 0
gen_cap_actuals.loc[(gen_cap_actuals.Tech_simple.isin(thermal_Tech))&(gen_cap_actuals.BuiltP2025 == True),'Input2037_NNTp2025'] = 0


In [ ]:
gen_cap_actuals_tech = gen_cap_actuals.fillna(0).groupby([ 'ValidationTech']).sum().rename_axis('model', axis=1)

#### Validation actuals

In [ ]:
gen_actuals_tech = pd.read_excel(gen_params_path, sheet_name='Validation2021', usecols='A:B', engine='openpyxl')
gen_actuals_tech = gen_actuals_tech.rename(columns={'Tech':'ValidationTech', 'GWh': 'Generation' })
gen_actuals_tech.loc[:,'model'] = 'Historical (2021)'
gen_actuals_tech = gen_actuals_tech.groupby(['model', 'ValidationTech'])['Generation'].sum()
#

gen_cap_actuals = pd.read_excel(gen_params_path, sheet_name='AllGenerators', usecols='A:AX', engine='openpyxl', skiprows=2)

gen_cap_actuals.loc[:,'Input_2021'] = gen_cap_actuals.MaxCap_2021*gen_cap_actuals.Units_2021
gen_cap_actuals.loc[:,'Input_2025'] = gen_cap_actuals.MaxCap_2025*gen_cap_actuals.Units_2025
gen_cap_actuals.loc[:,'Input_2030'] = gen_cap_actuals.MaxCap_2030*gen_cap_actuals.Units_2030
gen_cap_actuals.loc[:,'Input_2037'] = gen_cap_actuals.MaxCap_2037*gen_cap_actuals.Units_2037

gen_cap_actuals.loc[:,'Input2030_NNTp2025'] = gen_cap_actuals.Input_2030
gen_cap_actuals.loc[:,'Input2037_NNTp2025'] = gen_cap_actuals.Input_2037
#
thermal_Tech = ['Gas', 'Coal', 'Oil']
gen_cap_actuals.loc[(gen_cap_actuals.Tech_simple.isin(thermal_Tech))&(gen_cap_actuals.BuiltP2025 == True),'Input2030_NNTp2025'] = 0
gen_cap_actuals.loc[(gen_cap_actuals.Tech_simple.isin(thermal_Tech))&(gen_cap_actuals.BuiltP2025 == True),'Input2037_NNTp2025'] = 0


gen_cap_actuals = gen_cap_actuals[['PLEXOSname', 'ValidationTech'] + [c for c in gen_cap_actuals if 'Input' in c]].dropna(how='all').fillna(0)

gen_cap_actuals_tech = gen_cap_actuals.groupby([ 'ValidationTech'])[[c for c in gen_cap_actuals.columns if 'Input' in c]].sum().rename_axis('model', axis=1).stack('model').reorder_levels([ 'model', 'ValidationTech'])
gen_cap_actuals_plant = gen_cap_actuals.groupby(['PLEXOSname'])[[c for c in gen_cap_actuals.columns if 'Input' in c]].sum().rename_axis('model', axis=1).stack('model').reorder_levels([ 'model', 'PLEXOSname'])
##########



gen_valid_cap_tech = gen_yr_df[gen_yr_df.property == 'Installed Capacity'].groupby(
    [ 'model', 'ValidationTech']).value.sum()

gen_valid_by_tech = gen_yr_df[gen_yr_df.property == 'Generation'].groupby(
    [ 'model', 'ValidationTech']).value.sum()

gen_valid_cap_plant =gen_yr_df[gen_yr_df.property == 'Installed Capacity'].groupby(['model', 'name',]).value.sum().rename_axis(['model', 'PLEXOSname'])



gen_valid_cap_tech = pd.concat([gen_cap_actuals_tech, gen_valid_cap_tech], axis=0).unstack('ValidationTech')
gen_valid_cap_plant = pd.concat([gen_cap_actuals_plant, gen_valid_cap_plant], axis=0).unstack('model')
gen_valid_by_tech = pd.concat([gen_actuals_tech, gen_valid_by_tech], axis=0).unstack('ValidationTech')

try:
    add_df_column(gen_valid_cap_tech, 'units', 'MW').to_csv(os.path.join(save_dir_sum, '00a_gen_cap_valid_by_tech.csv'), index=False)
    add_df_column(gen_valid_by_tech, 'units', 'GWh').to_csv(os.path.join(save_dir_sum, '00b_gen_valid_by_tech.csv'), index=False)
    add_df_column(gen_valid_cap_plant, 'units', 'MW').to_csv(os.path.join(save_dir_sum, '00c_gen_cap_valid_by_plant.csv'), index=False)

except PermissionError:
    print("Permission error: file not written")

#### Produce outputs

In [ ]:
all_regs = np.sort(np.unique((node_df[geo_cols].values.flatten())))

In [ ]:
node_df.Region.unique()

In [ ]:
### Output 1
### To allow scaling, the group index is maintained (i.e. as_index=True) before resetting the index

load_by_reg = node_yr_df[node_yr_df.property == 'Load'].groupby(['model', 'timestamp']+geo_cols).sum().value
customer_load_by_reg = node_yr_df[(node_yr_df.property == 'Customer Load')|(node_yr_df.property == 'Unserved Energy')].groupby(['model', 'timestamp']+geo_cols).sum().value

add_df_column(load_by_reg, 'units', 'GWh').to_csv(os.path.join(save_dir_sum, '01a_load_by_reg.csv'), index=False)
add_df_column(customer_load_by_reg, 'units', 'GWh').to_csv(os.path.join(save_dir_sum, '01b_customer_load_by_reg.csv'), index=False)



In [ ]:
### Output 2: USE


use_by_reg = node_yr_df[node_yr_df.property == 'Unserved Energy'].groupby(['model'] + geo_cols ).sum().value
use_reg_daily_ts =  node_yr_df[node_yr_df.property == 'Unserved Energy'].groupby(['model'] + geo_cols + [pd.Grouper(key='timestamp',freq='D')] ).sum().value


add_df_column(use_by_reg, 'units', 'GWh').to_csv(os.path.join(save_dir_sum, '02a_use_reg.csv'), index=False)
add_df_column(use_reg_daily_ts, 'units', 'GWh').to_csv(os.path.join(save_dir_sum, '02b_use_reg_daily_ts.csv'), index=False)


In [ ]:
gen_yr_df.WEO_tech.unique()

In [ ]:
### Ouput 3a:

## Standard
gen_by_tech_reg =  gen_yr_df[gen_yr_df.property == 'Generation'].groupby(['model']+geo_cols+ ['Category']).sum().value.unstack(level=geo_cols).fillna(0)
gen_by_tech_reg_orig = gen_by_tech_reg.copy() ## From Indonesia to allow for co-firing fuels to be seperated ... kept only for now to avoid breaking other partss
gen_by_subtech_reg =  gen_yr_df[gen_yr_df.property == 'Generation'].groupby(['model']+geo_cols+ ['CostTech']).sum().value.unstack(level=geo_cols).fillna(0)

gen_techs = gen_yr_df.Category.drop_duplicates().values

##cofiring removed. to be added as standard?
    
    
# gen_by_costTech_reg = gen_by_tech_reg.groupby(['model']+geo_cols+ ['CostCategory']).sum().value.unstack(level=geo_cols).fillna(0)
# gen_by_tech_reg = gen_by_tech_reg
# gen_by_tech_reg_orig = gen_by_tech_reg_orig.groupby(['model']+geo_cols+ ['Category']).sum().value.unstack(level=geo_cols).fillna(0)
# gen_by_weoTech_reg = gen_yr_df[gen_yr_df.property == 'Generation'].groupby(['model'] + geo_cols + ['WEO_Tech_simpl']).sum().value.unstack(level=geo_cols).fillna(0)


try:
    add_df_column(gen_by_tech_reg.stack(geo_cols), 'units', 'GWh').to_csv(os.path.join(save_dir_sum, '03a_gen_by_tech_reg.csv'), index=False)
#     add_df_column(gen_by_tech_reg.stack(geo_cols), 'units', 'GWh').to_csv(os.path.join(save_dir_sum, '03b_gen_by_tech_reg_w_Validation.csv'), index=False)
    add_df_column(gen_by_subtech_reg.stack(geo_cols), 'units', 'GWh').to_csv(os.path.join(save_dir_sum, '03a_gen_by_subtech_reg.csv'), index=False)

#     add_df_column(gen_by_costTech_reg.stack(geo_cols), 'units', 'GWh').to_csv(os.path.join(save_dir_sum, '03c_gen_by_costTech_reg.csv'), index=False)
#     add_df_column(gen_by_weoTech_reg.stack(geo_cols), 'units', 'GWh').to_csv(os.path.join(save_dir_sum, '03d_gen_by_weoTech_reg.csv'), index=False)
#     add_df_column(gen_by_tech_subreg.stack(), 'units', 'GWh').to_csv(os.path.join(save_dir_sum, '04b_gen_by_tech_subreg.csv'), index=False)
#     add_df_column(gen_by_tech_isl.stack(), 'units', 'GWh').to_csv(os.path.join(save_dir_sum, '04c_gen_by_tech_isl.csv'), index=False)
except PermissionError:
    print("Permission error: file not written")

In [ ]:
### Ouput xx: Fuel offtake and emissions

## Doesn't work as emissions.Fuels doesnt exist in model. need to add some check?

## Standard
fuel_by_type =  fuel_yr_df[fuel_yr_df.property == 'Offtake']
fuel_by_type = fuel_by_type.groupby(['model','Type','Category']).sum().value


co2_fuels_by_reg = em_fuel_yr_df[(em_fuel_yr_df.property == 'Production')&(em_fuel_yr_df.parent == 'CO2')]
co2_fuels_by_reg = co2_fuels_by_reg.groupby(['model']+geo_cols + ['Type','Category']).sum().value

    
# gen_by_costTech_reg = gen_by_tech_reg.groupby(['model']+geo_cols+ ['CostCategory']).sum().value.unstack(level=geo_cols).fillna(0)
# gen_by_tech_reg = gen_by_tech_reg.groupby(['model']+geo_cols+ ['Category']).sum().value.unstack(level=geo_cols).fillna(0)


# if validation:
#     ### This needs to be fixed to the region/subregion change
#     idn_actuals_by_tech_reg  = idn_actuals_2019.groupby(['model'] +geo_cols +[ 'Category']).sum().Generation.unstack(level=geo_cols).fillna(0)
#     gen_by_tech_reg = pd.concat([gen_by_tech_reg, idn_actuals_by_tech_reg], axis=0)

try:
    add_df_column(fuel_by_type, 'units', 'TJ').to_csv(os.path.join(save_dir_sum, '00x_fuel_by_reg.csv'), index=False)
    add_df_column(co2_fuels_by_reg, 'units', 'tonnes').to_csv(os.path.join(save_dir_sum, '00x_co2_by_fuel.csv'), index=False)
#     add_df_column(gen_by_tech_subreg.stack(), 'units', 'GWh').to_csv(os.path.join(save_dir_sum, '04b_gen_by_tech_subreg.csv'), index=False)
#     add_df_column(gen_by_tech_isl.stack(), 'units', 'GWh').to_csv(os.path.join(save_dir_sum, '04c_gen_by_tech_isl.csv'), index=False)
except PermissionError:
    print("Permission error: file not written")

In [ ]:
### Output 3b: RE/VRE Shares:

gen_techs = list(soln_idx[soln_idx.Object_type == 'Generator'].Category.drop_duplicates())
vre_techs =['Solar', 'Wind']
re_techs = ['Solar', 'Wind', 'Bioenergy', 'Geothermal', 'Other', 'Marine', 'Hydro']

re_by_reg = gen_by_tech_reg.reset_index()
re_by_reg.loc[:,'RE'] = re_by_reg.Category.apply(lambda x: 'RE' if x in re_techs else 'Non-RE')

vre_by_reg = gen_by_tech_reg.reset_index()
vre_by_reg.loc[:,'VRE'] = vre_by_reg.Category.apply(lambda x: 'VRE' if x in vre_techs else 'Non-VRE')


re_by_reg = re_by_reg.groupby(['model','RE']).sum().groupby(level=geo_cols[0], axis=1).sum()
re_by_reg.loc[:,'UKR'] = re_by_reg.sum(axis=1)
re_by_reg = re_by_reg.loc[ix[:,'RE'],].droplevel('RE')/re_by_reg.groupby('model').sum()

vre_by_reg = vre_by_reg.groupby(['model', 'VRE']).sum().groupby(level=geo_cols[0], axis=1).sum()
vre_by_reg.loc[:,'UKR'] = vre_by_reg.sum(axis=1)
vre_by_reg = vre_by_reg.loc[ix[:,'VRE'],].droplevel('VRE')/vre_by_reg.groupby('model').sum()


try:
    add_df_column(re_by_reg, 'units', '%').to_csv(os.path.join(save_dir_sum, '03b_re_by_reg.csv'), index=False)
    add_df_column(vre_by_reg, 'units', '%').to_csv(os.path.join(save_dir_sum, '03c_vre_by_reg.csv'), index=False)
#     add_df_column(gen_by_tech_subreg.stack(), 'units', 'GWh').to_csv(os.path.join(save_dir_sum, '04b_gen_by_tech_subreg.csv'), index=False)
#     add_df_column(gen_by_tech_isl.stack(), 'units', 'GWh').to_csv(os.path.join(save_dir_sum, '04c_gen_by_tech_isl.csv'), index=False)
except PermissionError:
    print("Permission error: file not written")


In [ ]:
### Ouput 5

unit_starts_by_tech = gen_yr_df[gen_yr_df.property == 'Units Started'].groupby(
    [ 'model', 'Category']).sum().value.unstack(level='Category')

add_df_column(unit_starts_by_tech.stack(), 'units', 'starts').to_csv(
    os.path.join(save_dir_sum, '05_unit_starts_by_tech.csv'), index=False)

In [ ]:
### Ouput 6

## Standard
gen_max_by_tech_reg = gen_df[gen_df.property == 'Generation'].groupby(
    ['model']+geo_cols+ ['Category']).max(numeric_only=True).value.unstack(level=geo_cols).fillna(0)

# gen_max_by_tech_subreg = gen_df[gen_df.property == 'Generation'].groupby(
#     ['model',geo_cols[1], 'Category']).max(numeric_only=True).value.unstack(level=geo_cols[1]).fillna(0)

# gen_max_by_tech_isl = gen_df[gen_df.property == 'Generation'].groupby(
#     ['model',geo_cols[0], 'Category']).max(numeric_only=True).value.unstack(level=geo_cols[0]).fillna(0)

try:
    add_df_column(gen_max_by_tech_reg.stack(geo_cols), 'units', 'MW').to_csv(
            os.path.join(save_dir_sum, '06a_gen_max_by_tech_reg.csv'), index=False)
#     add_df_column(gen_max_by_tech_subreg.stack(), 'units', 'MW').to_csv(
#             os.path.join(save_dir_sum, '06a_gen_max_by_tech_subreg.csv'), index=False)
#     add_df_column(gen_max_by_tech_isl.stack(), 'units', 'MW').to_csv(
#             os.path.join(save_dir_sum, '06a_gen_max_by_tech_isl.csv'), index=False)
except PermissionError:
    print("Permission error: file not written")

In [ ]:
### Ouput 7

tx_losses = line_yr_df[line_yr_df.property == 'Loss'].groupby(['model', 'timestamp','name']).sum().value
try:
    add_df_column(tx_losses, 'units', 'GWh').to_csv(os.path.join(save_dir_sum, '07_tx_losses.csv'), index=False)
except PermissionError:
    print("Permission error: file not written")

In [ ]:
# vre_techs =['Solar', 'Wind']

# vre_cap = gen_yr_df[(gen_yr_df.property == 'Installed Capacity')& (gen_yr_df.Category.isin(vre_techs))].groupby(
#     ['model', 'Category'] +geo_cols).max().value.unstack('Category').fillna(0).stack('Category').unstack(level=geo_cols).fillna(0)

# vre_av_abs = gen_df[(gen_df.property == 'Available Capacity')& (gen_df.Category.isin(vre_techs))].groupby(
#     ['model', 'Category'] +geo_cols +[pd.Grouper(key='timestamp',freq='D')]).sum().value.unstack('Category').fillna(0).stack('Category').unstack(level=geo_cols).fillna(0)*hour_corr

# vre_gen_abs = gen_df[(gen_df.property == 'Generation')& (gen_df.Category.isin(vre_techs))].groupby(
#     ['model', 'Category',] +geo_cols +[pd.Grouper(key='timestamp',freq='D')]).sum().value.unstack('Category').fillna(0).stack('Category').unstack(level=geo_cols).fillna(0)*hour_corr


# # ### Add zero values to regions without VRE
# geo_col_filler = pd.Series(data=np.ones(len(load_by_reg.unstack(geo_cols).columns)), index=load_by_reg.unstack(geo_cols).columns)
# vre_cap = (vre_cap*geo_col_filler).fillna(0)
# vre_av_abs = (vre_av_abs*geo_col_filler).fillna(0)
# vre_gen_abs = (vre_gen_abs*geo_col_filler).fillna(0)   

In [ ]:
### Output 8 & 9

## Fill in data for regions which have no VRE (i.e. zero arrays!) to allow similar arrays for load_ts and vre_add_df_columns
## To add something for subregions

### There is an error in PLEXOS with Available Capacity versus Generation (Gen exceeds Av Capacity)

vre_cap = gen_yr_df[(gen_yr_df.property == 'Installed Capacity')& (gen_yr_df.Category.isin(vre_techs))].groupby(
    ['model', 'Category'] +geo_cols).max().value.unstack('Category').fillna(0).stack('Category').unstack(level=geo_cols).fillna(0)

vre_av_abs = gen_df[(gen_df.property == 'Available Capacity')& (gen_df.Category.isin(vre_techs))].groupby(
    ['model', 'Category'] +geo_cols +[pd.Grouper(key='timestamp',freq='D')]).sum().value.unstack('Category').fillna(0).stack('Category').unstack(level=geo_cols).fillna(0)*hour_corr

vre_gen_abs = gen_df[(gen_df.property == 'Generation')& (gen_df.Category.isin(vre_techs))].groupby(
    ['model', 'Category',] +geo_cols +[pd.Grouper(key='timestamp',freq='D')]).sum().value.unstack('Category').fillna(0).stack('Category').unstack(level=geo_cols).fillna(0)*hour_corr


# ### Add zero values to regions without VRE
geo_col_filler = pd.Series(data=np.ones(len(load_by_reg.unstack(geo_cols).columns)), index=load_by_reg.unstack(geo_cols).columns)
vre_cap = (vre_cap*geo_col_filler).fillna(0)
vre_av_abs = (vre_av_abs*geo_col_filler).fillna(0)
vre_gen_abs = (vre_gen_abs*geo_col_filler).fillna(0)      


### 24 periods per day for the daily data
vre_av_norm =  (vre_av_abs / vre_cap / daily_periods).fillna(0)
vre_curtailed = vre_av_abs - vre_gen_abs

### Add non-VRE spillage/curtailment
constr_techs = ['Hydro', 'Bioenergy', 'Geothermal']

other_re_gen_abs = gen_df[(gen_df.property == 'Generation')& (gen_df.Category.isin(constr_techs))].groupby(
    ['model', 'Category',] +geo_cols +[pd.Grouper(key='timestamp',freq='D')]).sum().value.unstack('Category').fillna(0).stack('Category').unstack(level=geo_cols).fillna(0)*hour_corr 
other_re_energy_vio = gen_df[(gen_df.property == 'Min Energy Violation') & (gen_df.Category.isin(constr_techs))].groupby(
    ['model', 'Category'] +geo_cols +[pd.Grouper(key='timestamp',freq='D')]).sum().value.unstack('Category').fillna(0).stack('Category').unstack(level=geo_cols).fillna(0)*hour_corr

other_re_gen_abs = (other_re_gen_abs*geo_col_filler).fillna(0)
other_re_energy_vio = (other_re_energy_vio*geo_col_filler).fillna(0)
other_re_av = other_re_energy_vio + other_re_gen_abs

all_re_av = pd.concat([vre_av_abs, other_re_av], axis=0).reset_index().groupby(['model', 'timestamp', 'Category']).sum()

all_re_curtailed = pd.concat([vre_curtailed, other_re_energy_vio], axis=0).reset_index().groupby(['model', 'timestamp', 'Category']).sum()


re_curtailment_rate_by_tech = (all_re_curtailed.sum(axis=1).groupby(['model','Category']).sum().unstack('Category')/all_re_av.sum(axis=1).groupby(['model','Category']).sum().unstack('Category')).fillna(0)*100
re_curtailment_rate = (all_re_curtailed.sum(axis=1).groupby('model').sum()/all_re_av.sum(axis=1).groupby('model').sum()).fillna(0)*100

curtailment_rate = (vre_curtailed.sum(axis=1).groupby('model').sum()/vre_av_abs.sum(axis=1).groupby('model').sum()).fillna(0)*100


curtailment_rate_by_tech = (vre_curtailed.sum(axis=1).groupby(['model','Category']).sum().unstack('Category')/vre_av_abs.sum(axis=1).groupby(['model','Category']).sum().unstack('Category')).fillna(0)*100
curtailment_rate = (vre_curtailed.sum(axis=1).groupby('model').sum()/vre_av_abs.sum(axis=1).groupby('model').sum()).fillna(0)*100
curtailment_rate_reg = (vre_curtailed.groupby(geo_cols[0],axis=1).sum().groupby('model').sum()/vre_av_abs.groupby(geo_cols[0],axis=1).sum().groupby('model').sum()).fillna(0)*100
curtailment_rate = pd.concat([curtailment_rate_reg, curtailment_rate.rename('UKR')], axis=1)

try:
    add_df_column(vre_cap.stack(geo_cols), 'units', 'MW').to_csv(os.path.join(save_dir_sum, '08a_vre_cap.csv'), index=False)
    add_df_column(vre_av_abs.stack(geo_cols), 'units', 'GWh').to_csv(os.path.join(save_dir_sum, '08b_vre_daily_abs.csv'),index=False)
    add_df_column(vre_av_norm.stack(geo_cols), 'units', '-').to_csv(os.path.join(save_dir_sum, '08c_vre_daily_norm.csv'),index=False)
    add_df_column(vre_curtailed.stack(geo_cols), 'units', 'GWh').to_csv(os.path.join(save_dir_sum, '09a_vre_daily_curtailed.csv'), index=False)
    add_df_column(curtailment_rate, 'units', '%').to_csv(os.path.join(save_dir_sum, '09b_curtailment_rate.csv'), index=False)
    add_df_column(all_re_curtailed.stack(geo_cols), 'units', 'GWh').to_csv(os.path.join(save_dir_sum, '09c_all_RE_daily_curtailed.csv'), index=False)
    add_df_column(re_curtailment_rate, 'units', '%').to_csv(os.path.join(save_dir_sum, '09d_all_RE_curtailment_rate.csv'), index=False)
except PermissionError:
    print("Permission error: file not written")

In [ ]:
### Output 11 & 12 : a) capacity & CFs per technology/region b) CFs per tech only

gen_cap_tech_reg = gen_yr_df[gen_yr_df.property == 'Installed Capacity'].groupby(
    [ 'model'] + geo_cols + ['Category']).sum().value.unstack(level=geo_cols).fillna(0)

# gen_cap_tech_reg_IPPs = gen_yr_df[gen_yr_df.property == 'Installed Capacity'].groupby(
#     [ 'model'] + geo_cols + ['IPP', 'Category']).sum().value.unstack(level=geo_cols).fillna(0)

# gen_cap_tech_subreg = gen_yr_df[gen_yr_df.property == 'Installed Capacity'].groupby(
#     [ 'model', geo_cols[1], 'Category']).sum().value.unstack(level=geo_cols[1]).fillna(0)

gen_cap_subtech_reg = gen_yr_df[gen_yr_df.property == 'Installed Capacity'].groupby(
    [ 'model'] +geo_cols + ['CapacityCategory']).sum().value.unstack(level=geo_cols).fillna(0)

gen_cap_COSTtech_reg = gen_yr_df[gen_yr_df.property == 'Installed Capacity'].groupby(
    [ 'model'] +geo_cols + ['CostTech']).sum().value.unstack(level=geo_cols).fillna(0)

# #### For Capex calcs
# gen_cap_costTech_reg = gen_yr_df[gen_yr_df.property == 'Installed Capacity'].groupby(
#     [ 'model'] +geo_cols + ['CostCategory']).sum().value.unstack(level=geo_cols).fillna(0)

# gen_cap_by_weoTech_reg = gen_yr_df[gen_yr_df.property == 'Installed Capacity'].groupby(['model'] + geo_cols + ['WEO_Tech_simpl']).sum().value.unstack(level=geo_cols).fillna(0)



### Calculate as EN[GWh]/(Capacity[MW]/1000*hours)
### Standard
### As we make adjustments for co_firing on the energy values, we must re-calculate this

cf_tech_reg = (gen_by_tech_reg_orig/(gen_cap_tech_reg/1000*nr_days*24)).fillna(0)

cf_tech = (gen_by_tech_reg_orig.sum(axis=1)/(gen_cap_tech_reg.sum(axis=1)/1000*nr_days*24)).unstack(
    level='Category').fillna(0)

gen_cap_COSTtech_reg
try:
    add_df_column(gen_cap_tech_reg.stack(geo_cols), 'units', 'MW').to_csv(os.path.join(save_dir_sum, '11a_cap_by_tech_reg.csv'), index=False)
    add_df_column(gen_cap_subtech_reg.stack(geo_cols), 'units', 'MW').to_csv(os.path.join(save_dir_sum, '11b_gen_cap_by_subtech_reg.csv'), index=False)
    add_df_column(gen_cap_COSTtech_reg.stack(geo_cols), 'units', 'MW').to_csv(os.path.join(save_dir_sum, '11b_gen_cap_by_COSTtech_reg.csv'), index=False)

#     add_df_column(gen_cap_costTech_reg.stack(geo_cols), 'units', 'MW').to_csv(os.path.join(save_dir_sum, '11c_gen_cap_by_costTech_reg.csv'), index=False)
#     add_df_column(gen_cap_costTech_reg.stack(geo_cols), 'units', 'MW').to_csv(os.path.join(save_dir_sum, '11d_gen_cap_by_weoTech_reg.csv'), index=False)
#     add_df_column(gen_cap_tech_reg_IPPs.stack(geo_cols), 'units', 'MW').to_csv(os.path.join(save_dir_sum, '11d_gen_cap_w_IPPs_by_tech_reg.csv'), index=False)

#     add_df_column(gen_cap_tech_subreg.stack(), 'units', 'MW').to_csv(os.path.join(save_dir_sum, '11b_gen_cap_by_tech_subreg.csv'), index=False)
    add_df_column(cf_tech_reg.stack(geo_cols), 'units', '%').to_csv(os.path.join(save_dir_sum, '12a_cf_tech_reg.csv'), index=False)
#     add_df_column(cf_tech_subreg.stack(), 'units', '%').to_csv(os.path.join(save_dir_sum, '12b_cf_tech_subreg.csv'), index=False)
    add_df_column(cf_tech, 'units', '%').to_csv(os.path.join(save_dir_sum, '12c_cf_tech.csv'), index=False)

except PermissionError:
    print("Permission error: file not written")

In [ ]:
### Output 14: Emissions

## Standard

em_by_type_tech_reg = em_gen_yr_df[(em_gen_yr_df.property=='Production')].groupby(
    [ 'model', 'parent'] + geo_cols + ['Category']).sum().value.reset_index()

em_by_type_tech_reg.loc[:,'parent'] = em_by_type_tech_reg.parent.apply(lambda x: x if '_' not in x else x.split('_')[0])

em_by_type_tech_reg = em_by_type_tech_reg.groupby([ 'model', 'parent'] + geo_cols + ['Category']).sum()

############

co2_by_tech_reg = em_gen_yr_df[em_gen_yr_df.parent.str.contains('CO2')&(em_gen_yr_df.property=='Production')].groupby(
    [ 'model'] + geo_cols + ['Category']).sum().value

co2_by_reg = em_gen_yr_df[em_gen_yr_df.parent.str.contains('CO2')&(em_gen_yr_df.property=='Production')].groupby(
    [ 'model'] + geo_cols).sum().value




try:
    add_df_column(co2_by_tech_reg, 'units', 'tonnes').to_csv(
        os.path.join(save_dir_sum, '13a_co2_by_tech_reg.csv'), index=False)
    add_df_column(co2_by_reg, 'units', 'tonnes').to_csv(
        os.path.join(save_dir_sum, '13b_co2_by_reg.csv'), index=False)
except PermissionError:
    print("Permission error: file not written")

1. a) Load time series; b) Load time-series (regional); c) Load with USE time-series
2. a) Generation time-series; b) VRE available; c) VRE available (regional); d) Curtailment; e) Generator availability 
3. a) Regional generation time-series b) Capacity factor monthly and c) weekly
4. a) Net-load regional timeseries; b) Net-load agg timeseries;  c) Inertia agg timeseries; d) Inertia tech timeseries
5. a) Regional price time-series b) Shadow price; c) SRMC
6. Days of interest/values: max avg load, peak net/total load, max ramp, min avg net load, min inertia, min net/total load
7. Generation stacks on days of interest (output to pre-made charts)
8. Unit out timeseries
9. Peak availability vs installed capacity
10. a) GDC; b) Unused generation by tech
11. Services by tech



Done for all models....maybe to change if performance suffers

In [ ]:
save_dir_ts = os.path.join(save_dir,'timeseries_out')

In [ ]:
if os.path.exists(save_dir_ts) is False:
    os.mkdir(save_dir_ts)

In [ ]:
reg_ts = True

In [ ]:
ix = pd.IndexSlice

### Timeseries outputs

In [ ]:
### Output 1a-b: Load and USE time-series
total_load_ts = reg_df[reg_df.property == 'Load'].groupby(['model', 'timestamp']).sum().value


add_df_column(total_load_ts, 'units', 'MW').reset_index().to_csv(os.path.join(save_dir_ts, '01a_total_load_ts.csv'))

use_ts = reg_df[reg_df.property == 'Unserved Energy'].groupby(['model', 'timestamp']).sum().value
add_df_column(use_ts, 'units', 'MW').reset_index().to_csv(os.path.join(save_dir_ts, '01b_use_ts.csv'))

### Need to calculate whether its 30mn or 1hr within but for now just assume hourly
use_dly_ts = use_ts.groupby([pd.Grouper(level='model'), pd.Grouper(freq='D',level='timestamp')]).sum()/1000
add_df_column(use_dly_ts, 'units', 'GWh').reset_index().to_csv(os.path.join(save_dir_ts, '01c_use_dly_ts.csv'))

load_w_use_ts = pd.concat([total_load_ts.rename('Load'),use_ts.rename('USE')])
add_df_column(load_w_use_ts, 'units', 'MW').reset_index().to_csv(os.path.join(save_dir_ts, '01d_load_w_use_ts.csv'))


if reg_ts:
    
    load_by_reg_ts = node_df[node_df.property == 'Load'].groupby(['model'] + geo_cols + ['timestamp']).sum().value.unstack(
    level='timestamp').fillna(0).stack('timestamp')
    
    use_reg_ts = node_df[node_df.property == 'Unserved Energy'].groupby(['model'] + geo_cols + ['timestamp']).sum().value.unstack(level=geo_cols)
    
    use_dly_reg_ts = use_reg_ts.groupby([pd.Grouper(level='model'), pd.Grouper(freq='D',level='timestamp')]).sum()/1000
    
    
    for geo in geo_cols:
        
        geo_suffix = geo[:3].lower()
        count = ord('e')
        
    
        add_df_column(load_by_reg_ts.unstack(level=geo_cols).groupby(level=geo, axis=1).sum(), 'units', 'MW').to_csv(os.path.join(
            save_dir_ts, '01{}_total_load_{}_ts.csv'.format(chr(count), geo_suffix)), index=False)

    
        add_df_column(use_reg_ts.groupby(level=geo, axis=1).sum(), 'units', 'MW').to_csv(os.path.join(
            save_dir_ts, '01{}_use_{}_ts.csv'.format(chr(count+1), geo_suffix)), index=False)

        
        add_df_column(use_dly_reg_ts.groupby(level=geo, axis=1).sum(), 'units', 'GWh').to_csv(os.path.join(save_dir_ts, '01{}_use_dly_{}_ts.csv'.format(chr(count+2), geo_suffix)), index=False)
        
        count += 3 




In [ ]:
### Output 2: Generation

gen_by_tech_ts = gen_df[gen_df.property == 'Generation'].groupby(
                    ['model', 'Category', 'timestamp']).sum().value.unstack(level='Category').fillna(0)

gen_by_subtech_ts = gen_df[gen_df.property == 'Generation'].groupby(
                    ['model', 'CapacityCategory', 'timestamp']).sum().value.unstack(level='CapacityCategory').fillna(0)

av_cap_by_tech_ts = gen_df[gen_df.property == 'Available Capacity'].groupby(
                    ['model', 'Category', 'timestamp']).sum().value.unstack(level='Category').fillna(0)

add_df_column(gen_by_tech_ts, 'units', 'MW').to_csv(os.path.join(save_dir_ts, '02a_gen_by_tech_ts.csv'),
                                                    index=False)

add_df_column(gen_by_subtech_ts, 'units', 'MW').to_csv(os.path.join(save_dir_ts, '02b_gen_by_tech_ts.csv'),
                                                    index=False)

add_df_column(av_cap_by_tech_ts, 'units', 'MW').to_csv(os.path.join(save_dir_ts, '02c_av_cap_by_subtech_ts.csv'),
                                                    index=False)

In [ ]:
### Output 3: VRE Time-series

vre_av_abs_ts = gen_df[(gen_df.property == 'Available Capacity') & (gen_df.Category.isin(vre_techs))].groupby(
    ['model', 'Category', 'timestamp']).sum().value.unstack(
    level='Category').fillna(0)

vre_gen_abs_ts = gen_df[(gen_df.property == 'Generation') & (gen_df.Category.isin(vre_techs))].groupby(
    ['model', 'Category', 'timestamp']).sum().value.unstack(
    level='Category').fillna(0)

vre_curtailed_ts = vre_av_abs_ts - vre_gen_abs_ts

constr_techs = ['Hydro', 'Bioenergy', 'Geothermal']

min_energy_vio_tech_ts = gen_df[(gen_df.property == 'Min Energy Violation') & (gen_df.Category.isin(constr_techs))].groupby(
    ['model', 'Category', 'timestamp']).sum().value.unstack(
    level='Category').fillna(0)

re_curtailed_ts = pd.concat([vre_curtailed_ts, min_energy_vio_tech_ts])



curtailment_dc = vre_curtailed_ts.sum(axis=1).unstack('model')
curtailment_dc = pd.DataFrame(np.flipud(np.sort(curtailment_dc.values, axis=0)), index=curtailment_dc.index, columns=curtailment_dc.columns)


add_df_column(vre_av_abs_ts, 'units', 'MW').to_csv(os.path.join(save_dir_ts, '03a_vre_available_ts.csv'),
                                                index=False)
add_df_column(vre_av_abs_ts, 'units', 'MW').to_csv(os.path.join(save_dir_ts, '03b_vre_gen_ts.csv'),
                                                index=False)
add_df_column(vre_curtailed_ts, 'units', 'MW').to_csv(os.path.join(save_dir_ts, '03c_vre_curtailed_ts.csv'),
                                                index=False)
add_df_column(re_curtailed_ts, 'units', 'MW').to_csv(os.path.join(save_dir_ts, '03c_RE_curtailed_ts.csv'),
                                                index=False)


### Due to the size, we do the gen DFs per model
if reg_ts:
    
    ### Define model regs in multi-level format
    model_regs_multi = load_by_reg_ts.unstack(geo_cols).columns


    
    vre_av_reg_abs_ts = gen_df[(gen_df.property == 'Available Capacity') &(gen_df.Category.isin(vre_techs))].groupby(
        (['model'] + geo_cols + ['timestamp'])).sum().value.unstack(
        level=geo_cols).fillna(0)

    vre_gen_reg_abs_ts = gen_df[(gen_df.property == 'Generation') & (gen_df.Category.isin(vre_techs))].groupby(
        (['model'] + geo_cols + ['timestamp'])).sum().value.unstack(
        level=geo_cols).fillna(0)
    
    vre_regs = vre_av_reg_abs_ts.columns

    ## Fill in data for regions which have no VRE (i.e. zero arrays!) to allow similar arrays for load_ts and vre_av_ts
    for reg in model_regs_multi:
        if reg not in vre_regs:
            vre_av_reg_abs_ts[reg] = 0
            vre_gen_reg_abs_ts[reg] = 0
            
    ### Columns in alphabetical order        
    vre_av_reg_abs_ts = vre_av_reg_abs_ts[model_regs_multi]  
    vre_gen_reg_abs_ts = vre_gen_reg_abs_ts[model_regs_multi]
    
    vre_curtailed_reg_ts = vre_av_reg_abs_ts - vre_gen_reg_abs_ts
    

    for m in model_names:
        save_dir_model = os.path.join(save_dir_ts, m.replace("/",""))
        if os.path.exists(save_dir_model) is False:
            os.mkdir(save_dir_model)
    
        vre_av = vre_av_reg_abs_ts.loc[ix[m,:]]

        vre_gen = vre_gen_reg_abs_ts.loc[ix[m,:]]
        
        
#         ## Fill in data for regions which have no VRE (i.e. zero arrays!) to allow similar arrays for load_ts and vre_av_ts
#         vre_regs = vre_av.columns

#         for reg in model_regs_multi:
#             if reg not in vre_regs:
#                 vre_av[reg] = 0
#                 vre_gen[reg] = 0

#         ### Columns in alphabetical order
#         vre_av = vre_av[model_regs_multi]  
#         vre_gen = vre_gen[model_regs_multi]

        vre_curtailed =  vre_av - vre_gen
        
        ## For numbering purposes only
        count = ord('d')
        
        ## Output results for every geographical aggregation
        for geo in geo_cols:
            
            geo_suffix = geo[:3].lower()
            ### We probably want to output as wind/solar in columns but maintain the regions as columns for net load
            add_df_column(vre_av.groupby(level=geo,axis=1).sum(), 'units', 'MW').to_csv(
                os.path.join(save_dir_model, '03{}_vre_available_{}_ts.csv'.format(chr(count), geo_suffix)), index=False)
            add_df_column(vre_gen.groupby(level=geo,axis=1).sum(), 'units', 'MW').to_csv(
                os.path.join(save_dir_model, '03{}_vre_gen_{}_ts.csv'.format(chr(count+1), geo_suffix)), index=False)
            add_df_column(vre_curtailed.groupby(level=geo,axis=1).sum(), 'units', 'MW').to_csv(
                os.path.join(save_dir_model, '03{}_vre_curtailed_{}_ts.csv'.format(chr(count+2), geo_suffix)), index=False)
            
            
            ## Three different outputs
            count = count + 3

In [ ]:
### Output 4a-b: Generation by region and capacity reserve margin by technology

av_cap_by_tech_reg_ts = gen_df[gen_df.property == 'Available Capacity'].groupby(
                    ['model', 'Category'] + geo_cols + ['timestamp']).sum().value.unstack(level='Category').fillna(0)

gen_by_tech_reg_ts = gen_df[gen_df.property == 'Generation'].groupby(
                    ['model', 'Category'] + geo_cols + ['timestamp']).sum().value.unstack(level='Category').fillna(0)

av_cap_by_subtech_reg_ts = gen_df[gen_df.property == 'Available Capacity'].groupby(
                    ['model', 'CapacityCategory'] + geo_cols + ['timestamp']).sum().value.unstack(level='CapacityCategory').fillna(0)

gen_by_subtech_reg_ts = gen_df[gen_df.property == 'Generation'].groupby(
                    ['model', 'CapacityCategory'] + geo_cols + ['timestamp']).sum().value.unstack(level='CapacityCategory').fillna(0)


### Spare capacity by region and technology
undisp_cap_by_tech_reg_ts = av_cap_by_tech_reg_ts - gen_by_tech_reg_ts
undisp_cap_by_tech_reg_ts = undisp_cap_by_tech_reg_ts.mask(undisp_cap_by_tech_reg_ts < 0, 0)

undisp_cap_by_subtech_reg_ts = av_cap_by_subtech_reg_ts - gen_by_subtech_reg_ts
undisp_cap_by_subtech_reg_ts = undisp_cap_by_subtech_reg_ts.mask(undisp_cap_by_subtech_reg_ts < 0, 0)

### Name makes no sense
load_geo_ts = node_df[node_df.property == 'Load'].groupby(
                    ['model'] + geo_cols + ['timestamp']).sum().value.unstack(level=geo_cols)


### Cappacity reserves by region
av_cap_by_reg = av_cap_by_tech_reg_ts.sum(axis=1).unstack(level=geo_cols)
# av_cap_by_reg.columns = [c.replace(' ', '_') for c in av_cap_by_reg.columns]
res_by_reg_ts = av_cap_by_reg - load_geo_ts
res_margin_by_reg_ts = res_by_reg_ts/load_geo_ts

res_w_load_by_reg_ts = res_by_reg_ts.stack(geo_cols).reorder_levels(['model'] +geo_cols + ['timestamp']).rename('CapacityReserves')
res_w_load_by_reg_ts = pd.concat([res_w_load_by_reg_ts, load_geo_ts.stack(geo_cols).reorder_levels(['model'] +geo_cols + ['timestamp']).rename('Load')], axis=1)



add_df_column(res_w_load_by_reg_ts, 'units', 'MW').to_csv(os.path.join(save_dir_ts, '03a_cap_reserve_w_load_by_reg_ts.csv'),
                                                    index=False)

add_df_column(res_margin_by_reg_ts, 'units', '%').to_csv(os.path.join(save_dir_ts, '03b_reserve_margin_by_reg_ts.csv'),
                                                    index=False)


### Output 3c: Regional generation...this is likely redundnat as the calcs are done up by CF calculations

if reg_ts:
    
#     gen_by_tech_reg_ts = gen_df[(gen_df.property == 'Generation')].groupby(
#             ['model'] +geo_cols +  ['Category', 'timestamp']).sum().value
    
#     gen_by_subtech_reg_ts = gen_df[(gen_df.property == 'Generation')].groupby(
#         ['model'] +geo_cols +  ['CapacityCategory', 'timestamp']).sum().value
    
    for m in model_names:
        save_dir_model = os.path.join(save_dir_ts, m.replace("/",""))
        if os.path.exists(save_dir_model) is False:
            os.mkdir(save_dir_model)

        gen_by_tech_reg_model_ts = gen_by_tech_reg_ts.loc[ix[m,],].stack('Category')
        
        gen_by_subtech_reg_model_ts = gen_by_subtech_reg_ts.loc[ix[m,],].stack('CapacityCategory')

        add_df_column(gen_by_tech_reg_model_ts, 'units', 'MW').to_csv(
            os.path.join(save_dir_model, '04_gen_by_tech_reg_ts.csv'), index=False)
        add_df_column(gen_by_subtech_reg_model_ts, 'units', 'MW').to_csv(
            os.path.join(save_dir_model, '04_gen_by_subtech_reg_ts.csv'), index=False)


In [ ]:
#### Output 5: to plot undispatched capacity for USE (and maybe other metrics)

from openpyxl import load_workbook

if reg_ts:
    for m in model_names:
        save_dir_model = os.path.join(save_dir_ts, m.replace("/",""))
        if os.path.exists(save_dir_model) is False:
            os.mkdir(save_dir_model)
            
        
        model_use_ts = use_ts.loc[ix[m,:]].reset_index()
        use_periods = model_use_ts[model_use_ts.value>0]
        model_use_reg_ts = use_reg_ts.loc[ix[m,:]].reset_index()
        
        if len(use_periods) == 0:
            print("No USE for {}".format(m))
        else:
            use_days = np.unique(use_periods.timestamp.dt.date)
            fig_template_path = 'Z:/Ukraine/2022-3_next_step_modelling/09_ModellingSupportFiles/00_figure_templates/00_reg_data_template.xlsx'
            fig_path = os.path.join(save_dir_model,'05a_undisp_cap_tech_during_USE_reg.xlsx')
            
            # with pd.ExcelWriter(fig_path, engine='openpyxl', engine_kwargs= {'strings_to_formulas': False}) as writer:
            with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:
                ### load book template i.e. sheet names, etc.
#                 book = load_workbook(fig_template_path)
#                 writer.book = book
#                 writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
                
                for reg in model_regs:
  
                    undisp_cap = undisp_cap_by_tech_reg_ts.groupby(['model',geo_cols[0],'timestamp']).sum().loc[m,reg,:].droplevel([0,1]).reset_index()
                    undisp_cap = undisp_cap.loc[undisp_cap.timestamp.dt.date.isin(use_days)].set_index('timestamp')
    
                    use_reg = model_use_reg_ts.loc[model_use_reg_ts.timestamp.dt.date.isin(use_days)].set_index('timestamp').groupby(level=geo_cols[0], axis=1).sum()[reg].to_frame().rename(columns={reg:'USE'})
                    undisp_cap = pd.concat([undisp_cap, use_reg], axis=1)
                
                    add_df_column(undisp_cap, 'units', 'MW').to_excel(writer, sheet_name=reg, index=False)
                
                ## Add total column
                undisp_cap = undisp_cap_by_tech_reg_ts.groupby(['model','timestamp']).sum().loc[m,:].reset_index()
                undisp_cap = undisp_cap.loc[undisp_cap.timestamp.dt.date.isin(use_days)].set_index('timestamp')
                use_reg = model_use_reg_ts.loc[model_use_reg_ts.timestamp.dt.date.isin(use_days)].set_index('timestamp').sum(axis=1).to_frame().rename(columns={0:'USE'})
                undisp_cap = pd.concat([undisp_cap, use_reg], axis=1)
                add_df_column(undisp_cap, 'units', 'MW').to_excel(writer, sheet_name='UKR', index=False)
                    
            fig_path = os.path.join(save_dir_model,'05b_undisp_cap_subtech_during_USE_reg.xlsx')
            
            with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:
                ### load book template i.e. sheet names, etc.
#                 book = load_workbook(fig_template_path)
#                 writer.book = book
#                 writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

#                 workbook = writer.book
#                 worksheet = writer.sheets[reg]
                
                for reg in model_regs:
  
                    undisp_cap = undisp_cap_by_subtech_reg_ts.groupby(['model',geo_cols[0],'timestamp']).sum().loc[m,reg,:].droplevel([0,1]).reset_index()
                    undisp_cap = undisp_cap.loc[undisp_cap.timestamp.dt.date.isin(use_days)].set_index('timestamp')
    
                    use_reg = model_use_reg_ts.loc[model_use_reg_ts.timestamp.dt.date.isin(use_days)].set_index('timestamp').groupby(level=geo_cols[0], axis=1).sum()[reg].to_frame().rename(columns={reg:'USE'})
                    undisp_cap = pd.concat([undisp_cap, use_reg], axis=1)
                
                    add_df_column(undisp_cap, 'units', 'MW').to_excel(writer, sheet_name=reg, index=False)
                
                ## Add total column
                undisp_cap = undisp_cap_by_subtech_reg_ts.groupby(['model','timestamp']).sum().loc[m,:].reset_index()
                undisp_cap = undisp_cap.loc[undisp_cap.timestamp.dt.date.isin(use_days)].set_index('timestamp')
                use_reg = model_use_reg_ts.loc[model_use_reg_ts.timestamp.dt.date.isin(use_days)].set_index('timestamp').sum(axis=1).to_frame().rename(columns={0:'USE'})
                undisp_cap = pd.concat([undisp_cap, use_reg], axis=1)
                add_df_column(undisp_cap, 'units', 'MW').to_excel(writer, sheet_name='UKR', index=False)

            
        

In [ ]:
# #### CHECK why net_laod_ts is different to net_load_reg_ts, as this appears to be the case.

# customer_load_ts_check = node_df[(node_df.property == 'Customer Load')|(node_df.property == 'Unserved Energy')].groupby(['model', 'timestamp']).sum().value
# net_load_ts_check = pd.DataFrame(customer_load_ts_check - vre_av_abs_ts.fillna(0).sum(axis=1).groupby(['model', 'timestamp']).sum(), columns=['value'])
# nl_min_dly.loc[ix[m,]].min()

In [ ]:
### Output 6: Net-load timeseries and LDCs

### So that we can see the total load before USE and pump load
customer_load_ts = reg_df[(reg_df.property == 'Customer Load')|(reg_df.property == 'Unserved Energy')].groupby(['model', 'timestamp']).sum().value
storage_load_ts = reg_df[(reg_df.property == 'Battery Load')|(reg_df.property == 'Pump Load')].groupby(['model', 'timestamp']).sum().value
storage_gen_ts = gen_by_tech_ts.Storage.rename('value')

######
## Model filler for comparison of models with different inputs (e.g. DSM or EVs not included)
### Series with indices matching the columns of the DF for filling in missing columns
model_filler = pd.Series(data=[1]*len(model_names), index=model_names).rename_axis('model')


if purch_df.shape[0] > 0:
    ev_profiles_ts = purch_df[purch_df.name.str.contains('_EV')&(purch_df.property == 'Load')].groupby(['model', 'timestamp']).sum().value
    ev_profiles_orig_ts = purch_df[purch_df.name.str.contains('_EV')&(purch_df.property == 'x')].groupby(['model', 'timestamp']).sum().value 
    if ev_profiles_ts.shape[0] == 0:
        ev_profiles_ts = ev_profiles_orig_ts = pd.Series(data=[0]*len(customer_load_ts.index), index=customer_load_ts.index)
    else:
        ev_profiles_ts = (ev_profiles_ts.unstack('model')*model_filler).fillna(0).stack('model').reorder_levels(['model', 'timestamp'])
        ev_profiles_orig_ts = (ev_profiles_orig_ts.unstack('model')*model_filler).fillna(0).stack('model').reorder_levels(['model', 'timestamp'])       
    
    dsm_profiles_ts = purch_df[purch_df.name.str.contains('_Shift')&(purch_df.property == 'Load')].groupby(['model', 'timestamp']).sum().value
    dsm_profiles_orig_ts = purch_df[purch_df.name.str.contains('_Shift')&(purch_df.property == 'x')].groupby(['model', 'timestamp']).sum().value
    if dsm_profiles_ts.shape[0] == 0:
        dsm_profiles_ts = dsm_profiles_orig_ts = pd.Series(data=[0]*len(customer_load_ts.index), index=customer_load_ts.index)
    else:
        dsm_profiles_ts = (dsm_profiles_ts.unstack('model')*model_filler).fillna(0).stack('model').reorder_levels(['model', 'timestamp'])
        dsm_profiles_orig_ts = (dsm_profiles_orig_ts.unstack('model')*model_filler).fillna(0).stack('model').reorder_levels(['model', 'timestamp'])
        
        
    electr_profiles_ts = purch_df[purch_df.name.str.contains('_Elec')&(purch_df.property == 'Load')].groupby(['model', 'timestamp']).sum().value
    electr_profiles_orig_ts = purch_df[purch_df.name.str.contains('_Elec')&(purch_df.property == 'x')].groupby(['model', 'timestamp']).sum().value
    if electr_profiles_ts.shape[0] == 0:
        electr_profiles_orig_ts = electr_profiles_ts = pd.Series(data=[0]*len(customer_load_ts.index), index=customer_load_ts.index)
    else:
        electr_profiles_ts = (electr_profiles_ts.unstack('model')*model_filler).fillna(0).stack('model').reorder_levels(['model', 'timestamp'])
        electr_profiles_orig_ts = (electr_profiles_orig_ts.unstack('model')*model_filler).fillna(0).stack('model').reorder_levels(['model', 'timestamp'])        
    
    native_load_ts  = node_df[node_df.property == 'Native Load'].groupby(['model', 'timestamp']).sum().value    
    
    
    customer_load_orig_ts = (native_load_ts + ev_profiles_orig_ts + dsm_profiles_orig_ts + electr_profiles_orig_ts) #.fillna(customer_load_ts) ### For those profiles where EVs are missing, for e.g. ... other DSM to be added
else:
    customer_load_orig_ts = customer_load_ts

    

##  net_load_ts is calculated as a series (as we obtain load 'value' and some across the x-axis (technologies) of vre_abs)
net_load_ts = pd.DataFrame(customer_load_ts - vre_av_abs_ts.fillna(0).sum(axis=1).groupby(['model', 'timestamp']).sum(), columns=['value'])
net_load_curtail_ts = pd.DataFrame(customer_load_ts - vre_gen_abs_ts.fillna(0).sum(axis=1).groupby(['model', 'timestamp']).sum(), columns=['value'])
net_load_orig_ts = pd.DataFrame(customer_load_orig_ts - vre_av_abs_ts.fillna(0).sum(axis=1).groupby(['model', 'timestamp']).sum(), columns=['value'])
net_load_sto_ts =  pd.DataFrame(customer_load_ts - storage_gen_ts + storage_load_ts - vre_av_abs_ts.fillna(0).sum(axis=1).groupby(['model', 'timestamp']).sum(), columns=['value'])
net_load_sto_curtail_ts = pd.DataFrame(customer_load_ts - storage_gen_ts + storage_load_ts - vre_gen_abs_ts.fillna(0).sum(axis=1).groupby(['model', 'timestamp']).sum(), columns=['value'])

#### Calculation of all the different variations of LDC for later comparison 
ldc = customer_load_ts.unstack('model')
ldc = pd.DataFrame(np.flipud(np.sort(ldc.values, axis=0)), index=ldc.index, columns=ldc.columns)

ldc_orig = customer_load_orig_ts.unstack('model')
ldc_orig = pd.DataFrame(np.flipud(np.sort(ldc_orig.values, axis=0)), index=ldc_orig.index, columns=ldc_orig.columns)

nldc = net_load_ts.value.unstack('model')
nldc = pd.DataFrame(np.flipud(np.sort(nldc.values, axis=0)), index=nldc.index, columns=nldc.columns)

nldc_orig = net_load_orig_ts.value.unstack('model')
nldc_orig = pd.DataFrame(np.flipud(np.sort(nldc_orig.values, axis=0)), index=nldc_orig.index, columns=nldc_orig.columns)

nldc_curtail = net_load_curtail_ts.value.unstack('model')
nldc_curtail = pd.DataFrame(np.flipud(np.sort(nldc_curtail.values, axis=0)), index=nldc_curtail.index, columns=nldc_curtail.columns)

nldc_sto = net_load_sto_ts.value.unstack('model')
nldc_sto = pd.DataFrame(np.flipud(np.sort(nldc_sto.values, axis=0)), index=nldc_sto.index, columns=nldc_sto.columns)

nldc_sto_curtail = net_load_sto_curtail_ts.value.unstack('model')
nldc_sto_curtail = pd.DataFrame(np.flipud(np.sort(nldc_sto_curtail.values, axis=0)), index=nldc_sto_curtail.index, columns=nldc_sto_curtail.columns)

#####

add_df_column(net_load_ts.value.unstack('model'), 'units', 'MW').to_csv(os.path.join(save_dir_ts, '06a_net_load_ts.csv'), index=True)
add_df_column(ldc.reset_index(drop=True), 'units', 'MW').to_csv(os.path.join(save_dir_ts, '06b_ldc.csv'), index=True)
add_df_column(nldc.reset_index(drop=True), 'units', 'MW').to_csv(os.path.join(save_dir_ts, '06c_net_ldc.csv'), index=True)
add_df_column(nldc_curtail.reset_index(drop=True), 'units', 'MW').to_csv(os.path.join(save_dir_ts, '06d_net_ldc_curtail.csv'), index=True)
add_df_column(net_load_orig_ts.value.unstack('model'), 'units', 'MW').to_csv(os.path.join(save_dir_ts, '06e_net_load_orig_ts.csv'), index=True)
add_df_column(ldc_orig.reset_index(drop=True), 'units', 'MW').to_csv(os.path.join(save_dir_ts, '06f_ldc_orig.csv'), index=True)
add_df_column(nldc_orig.reset_index(drop=True), 'units', 'MW').to_csv(os.path.join(save_dir_ts, '06g_net_ldc_orig.csv'), index=True)

add_df_column(nldc_sto.reset_index(drop=True), 'units', 'MW').to_csv(os.path.join(save_dir_ts, '06h_nldc_sto.csv'), index=True)
add_df_column(nldc_sto_curtail.reset_index(drop=True), 'units', 'MW').to_csv(os.path.join(save_dir_ts, '06i_net_ldc_sto_curtail.csv'), index=True)


if reg_ts:
    
#     vre_av_reg_abs_ts = gen_df[(gen_df.property == 'Available Capacity') & \
#                                 (gen_df.FlexCategory == 'VRE')].groupby(
#          ['model',geo_cols[0], 'timestamp']).sum().value.unstack(level=geo_cols[0])
    
    
    customer_load_reg_ts = node_df[(node_df.property == 'Customer Load')|(node_df.property == 'Unserved Energy')].groupby(
        ['model'] +geo_cols +['timestamp']).sum().value.unstack(level=geo_cols)
    
    storage_load_reg_ts = node_df[(reg_df.property == 'Battery Load')|(node_df.property == 'Pump Load')].groupby( ['model'] +geo_cols +['timestamp']).sum().value.unstack(level=geo_cols)
#     storage_gen_reg_ts = gen_by_tech_reg_ts.Storage.unstack(geo_cols)
    
    
    if purch_df.shape[0] > 0:
        ev_profiles_reg_ts = purch_df[purch_df.name.str.contains('_EV')&(purch_df.property == 'Load')].groupby(['model']+geo_cols+['timestamp']).sum().value
        ev_profiles_orig_reg_ts = purch_df[purch_df.name.str.contains('_EV')&(purch_df.property == 'x')].groupby(['model']+geo_cols+['timestamp']).sum().value
        if ev_profiles_reg_ts.shape[0] == 0:
            ev_profiles_reg_ts = ev_profiles_orig_reg_ts = pd.Series(data=[0]*len(customer_load_reg_ts.index), index=customer_load_reg_ts.index)
        else:
            ev_profiles_reg_ts = (ev_profiles_reg_ts.unstack('model')*model_filler).fillna(0).stack('model').reorder_levels(['model'] + geo_cols + ['timestamp'])
            ev_profiles_orig_reg_ts = (ev_profiles_orig_reg_ts.unstack('model')*model_filler).fillna(0).stack('model').reorder_levels(['model'] + geo_cols + ['timestamp'])    
        
        dsm_profiles_reg_ts = purch_df[purch_df.name.str.contains('_Shift')&(purch_df.property == 'Load')].groupby(['model']+geo_cols+['timestamp']).sum().value
        dsm_profiles_orig_reg_ts = purch_df[purch_df.name.str.contains('_Shift')&(purch_df.property == 'x')].groupby(['model']+geo_cols+['timestamp']).sum().value
        if dsm_profiles_reg_ts.shape[0] == 0:
            dsm_profiles_reg_ts = dsm_profiles_orig_reg_ts = pd.Series(data=[0]*len(customer_load_reg_ts.index), index=customer_load_reg_ts.index)
        else:
            dsm_profiles_reg_ts = (dsm_profiles_reg_ts.unstack('model')*model_filler).fillna(0).stack('model').reorder_levels(['model'] + geo_cols + ['timestamp'])
            dsm_profiles_orig_reg_ts = (dsm_profiles_orig_reg_ts.unstack('model')*model_filler).fillna(0).stack('model').reorder_levels(['model'] + geo_cols + ['timestamp'])    
            
        electr_profiles_reg_ts = purch_df[purch_df.name.str.contains('_Elec')&(purch_df.property == 'Load')].groupby(['model']+geo_cols+['timestamp']).sum().value
        electr_profiles_orig_reg_ts = purch_df[purch_df.name.str.contains('_Elec')&(purch_df.property == 'x')].groupby(['model']+geo_cols+['timestamp']).sum().value
        if electr_profiles_reg_ts.shape[0] == 0:
            electr_profiles_reg_ts = electr_profiles_orig_reg_ts = pd.Series(data=[0]*len(customer_load_reg_ts.index), index=customer_load_reg_ts.index)
        else:
            electr_profiles_reg_ts = (electr_profiles_reg_ts.unstack('model')*model_filler).fillna(0).stack('model').reorder_levels(['model'] + geo_cols + ['timestamp'])
            electr_profiles_orig_reg_ts = (electr_profiles_orig_reg_ts.unstack('model')*model_filler).fillna(0).stack('model').reorder_levels(['model'] + geo_cols + ['timestamp'])    
        
        native_load_reg_ts  = node_df[node_df.property == 'Native Load'].groupby(['model']+geo_cols+['timestamp']).sum().value
        
        
        
        customer_load_orig_reg_ts = (native_load_reg_ts + dsm_profiles_orig_reg_ts + ev_profiles_orig_ts + electr_profiles_orig_reg_ts).fillna(native_load_reg_ts).unstack(geo_cols)
    else:
         customer_load_orig_reg_ts = customer_load_reg_ts


    net_load_reg_ts = customer_load_reg_ts - vre_av_reg_abs_ts
    net_load_curtail_reg_ts = customer_load_reg_ts - vre_gen_reg_abs_ts
    net_load_orig_reg_ts = customer_load_orig_reg_ts - vre_av_reg_abs_ts    
#     net_load_sto_reg_ts =  customer_load_reg_ts - (storage_gen_reg_ts*geo_col_filler).fillna(0)  + (storage_load_reg_ts*geo_col_filler).fillna(0) - (vre_av_reg_abs_ts*geo_col_filler).fillna(0)


    
    add_df_column(net_load_reg_ts, 'units', 'MW').to_csv(os.path.join(save_dir_ts, '06h_net_load_reg_ts.csv'), index=False)
    add_df_column(net_load_curtail_reg_ts, 'units', 'MW').to_csv(os.path.join(save_dir_ts, '06i_net_load_curtail_reg_ts.csv'), index=False)
    add_df_column(net_load_orig_reg_ts, 'units', 'MW').to_csv(os.path.join(save_dir_ts, '06j_net_load_orig_reg_ts.csv'), index=False)
    
    for m in model_names:
        ### One by one to have the LDC (which are not coincident necessarily)
        ldc_reg = pd.DataFrame(None)
        nldc_reg = pd.DataFrame(None)
        ldc_orig_reg = pd.DataFrame(None)
        nldc_orig_reg = pd.DataFrame(None)        
        
        for col in geo_cols:
            
            save_dir_model = os.path.join(save_dir_ts, m.replace("/",""))
            if os.path.exists(save_dir_model) is False:
                os.mkdir(save_dir_model)
                
            ldc_m = customer_load_reg_ts.loc[ix[m,]].groupby(col,axis=1).sum()
            ldc_m = pd.DataFrame(np.flipud(np.sort(ldc_m.values, axis=0)), index=ldc_m.index, columns=ldc_m.columns)

            nldc_m = net_load_reg_ts.loc[ix[m,]].groupby(col,axis=1).sum()
            nldc_m = pd.DataFrame(np.flipud(np.sort(nldc_m.values, axis=0)), index=nldc_m.index, columns=nldc_m.columns)
            
            ldc_orig_m = customer_load_orig_reg_ts.loc[ix[m,]].groupby(col,axis=1).sum()
            ldc_orig_m = pd.DataFrame(np.flipud(np.sort(ldc_orig_m.values, axis=0)), index=ldc_orig_m.index, columns=ldc_orig_m.columns)

            nldc_orig_m = net_load_orig_reg_ts.loc[ix[m,]].groupby(col,axis=1).sum()
            nldc_orig_m = pd.DataFrame(np.flipud(np.sort(nldc_orig_m.values, axis=0)), index=nldc_orig_m.index, columns=nldc_orig_m.columns)
            
            ##############
                
            ldc_reg = pd.concat([ldc_reg, ldc_m], axis=1)
            nldc_reg = pd.concat([nldc_reg, nldc_m], axis=1)
            ldc_orig_reg = pd.concat([ldc_orig_reg, ldc_orig_m], axis=1)
            nldc_orig_reg = pd.concat([nldc_orig_reg, nldc_orig_m], axis=1)        
        

        add_df_column(ldc_reg.reset_index(drop=True), 'units', 'MW').to_csv(os.path.join(save_dir_model, '06i_ldc_reg.csv'), index=False)
        add_df_column(nldc_reg.reset_index(drop=True), 'units', 'MW').to_csv(os.path.join(save_dir_model, '06j_net_ldc_reg.csv'.format(chr(count+1), geo_suffix)), index=False)
        add_df_column(nldc_orig_reg.reset_index(drop=True), 'units', 'MW').to_csv(os.path.join(save_dir_model, '06k_net_ldc_orig_reg.csv'.format(chr(count+1), geo_suffix)), index=False)
        add_df_column(nldc_orig_reg.reset_index(drop=True), 'units', 'MW').to_csv(os.path.join(save_dir_model, '06l_net_ldc_orig_reg.csv'.format(chr(count+1), geo_suffix)), index=False)


In [ ]:
### Output 7 : Daily gap between peak and off-peak net load

# This could be future proofed with using the geo_col/suffix approach....

nl_max_dly_reg = net_load_reg_ts.groupby(geo_cols[0], axis=1).sum().groupby(['model', pd.Grouper(level='timestamp', freq='D')]).max()
nl_max_dly_subreg =net_load_reg_ts.groupby(geo_cols[1], axis=1).sum().groupby(['model', pd.Grouper(level='timestamp', freq='D')]).max()
nl_max_dly = net_load_reg_ts.sum(axis=1).groupby(['model', pd.Grouper(level='timestamp', freq='D')]).max()

nl_min_dly_reg = net_load_reg_ts.groupby(geo_cols[0], axis=1).sum().groupby(['model', pd.Grouper(level='timestamp', freq='D')]).min()
nl_min_dly_subreg = net_load_reg_ts.groupby(geo_cols[1], axis=1).sum().groupby(['model', pd.Grouper(level='timestamp', freq='D')]).min()
nl_min_dly = net_load_reg_ts.sum( axis=1).groupby(['model', pd.Grouper(level='timestamp', freq='D')]).min()

nl_dly_gap_reg = nl_max_dly_reg - nl_min_dly_reg
nl_dly_gap_subreg = nl_max_dly_subreg - nl_min_dly_subreg
nl_dly_gap = nl_max_dly - nl_min_dly

nl_dly_gap_reg_pc = (nl_max_dly_reg - nl_min_dly_reg)/nl_max_dly_reg
nl_dly_gap_subreg_pc = (nl_max_dly_subreg - nl_min_dly_subreg)/nl_max_dly_subreg
nl_dly_gap_pc = (nl_max_dly - nl_min_dly)/nl_max_dly

add_df_column(nl_dly_gap_reg, 'units', 'MW').to_csv(os.path.join(save_dir_ts, '07a_dly_gap_reg.csv'), index=False)
add_df_column(nl_dly_gap_subreg, 'units', 'MW').to_csv(os.path.join(save_dir_ts, '07b_dly_gap_subreg.csv'), index=False)
add_df_column(nl_dly_gap, 'units', 'MW').to_csv(os.path.join(save_dir_ts, '07d_dly_gap.csv'), index=False)

add_df_column(nl_dly_gap_reg_pc, 'units', 'MW').to_csv(os.path.join(save_dir_ts, '07e_dly_gap_reg_pc.csv'), index=False)
add_df_column(nl_dly_gap_subreg_pc, 'units', 'MW').to_csv(os.path.join(save_dir_ts, '07f_dly_gap_subreg_pc.csv'), index=False)
add_df_column(nl_dly_gap_pc, 'units', 'MW').to_csv(os.path.join(save_dir_ts, '07h_dly_gap_pc.csv'), index=False)

In [ ]:
# ### Output 8: Inertia (total)

gen_units_gen = gen_df[gen_df.property == 'Units Generating'].groupby(['model', 'name','timestamp']).sum().value
gen_units = gen_df[gen_df.property == 'Units'].groupby(['model', 'name','timestamp']).sum().value

## Take only the sum to maintain the capacity value & inertia constant in the dataframe
gen_cap = gen_df[gen_df.property == 'Installed Capacity'].groupby(['model', 'name','timestamp']).sum()
gen_cap = pd.merge(gen_cap.reset_index(), gen_inertia_idx, on='name', how='left').set_index(['model', 'name', 'timestamp'])

##  As installed capacity is [Units] * [Max Capacity], we must calculate the unit capacity
gen_inertia_lo = (gen_units_gen/gen_units)*(gen_cap.value*gen_cap.InertiaLOW)
gen_inertia_hi = (gen_units_gen/gen_units)*(gen_cap.value*gen_cap.InertiaHI)


gen_inertia = pd.merge(
    pd.DataFrame(gen_inertia_lo, columns=['InertiaLo']),
    pd.DataFrame(gen_inertia_hi, columns=['InertiaHi']),
    left_index=True, right_index=True)

gen_inertia = pd.merge(gen_inertia.reset_index(), soln_idx[['name', geo_cols[0], geo_cols[1], 'Category', 'CapacityCategory']],
                       on='name')

inertia_by_tech = gen_inertia.groupby(['model', 'Category', 'timestamp']).sum()
# inertia_by_isl = gen_inertia.drop.groupby(['model', geo_cols[0], 'timestamp']).sum()
inertia_by_reg = gen_inertia.groupby(['model'] + geo_cols + ['timestamp']).sum()
total_inertia_ts  = gen_inertia.groupby(['model', 'timestamp']).sum()

add_df_column(inertia_by_tech, 'units', 'MWs').to_csv(os.path.join(save_dir_ts, '08a_inertia_by_tech_ts.csv'), index=False)
# add_df_column(inertia_by_isl, 'units', 'MWs').to_csv(os.path.join(save_dir_ts, '08b_inertia_by_isl_ts.csv'), index=False)
add_df_column(inertia_by_reg, 'units', 'MWs').to_csv(os.path.join(save_dir_ts, '08c_inertia_by_reg_ts.csv'), index=False)
add_df_column(total_inertia_ts, 'units', 'MWs').to_csv(os.path.join(save_dir_ts, '08d_total_inertia_ts.csv'), index=False)

In [ ]:
### Output 9: Ramp time-series

ramp_ts = (net_load_ts.unstack(level='model') - net_load_ts.unstack(level='model').shift(1)).fillna(
    0).stack().sort_index(level=1).reset_index().set_index(['model', 'timestamp']).rename(columns={0:'value'})
th_ramp_ts = (net_load_ts.unstack(level='model') - net_load_ts.unstack(level='model').shift(3)).fillna(
    0).stack().sort_index(level=1).reset_index().set_index(['model', 'timestamp']).rename(columns={0:'value'})

ramp_orig_ts = (net_load_orig_ts.unstack(level='model') - net_load_orig_ts.unstack(level='model').shift(1)).fillna(
    0).stack().sort_index(level=1).reset_index().set_index(['model', 'timestamp']).rename(columns={0:'value'})
th_ramp_orig_ts = (net_load_orig_ts.unstack(level='model') - net_load_orig_ts.unstack(level='model').shift(3)).fillna(
    0).stack().sort_index(level=1).reset_index().set_index(['model', 'timestamp']).rename(columns={0:'value'})

#########
daily_pk_ts = customer_load_ts.groupby([pd.Grouper(level='model'), pd.Grouper(level='timestamp', freq='D')]).max().reindex(customer_load_ts.index).ffill()

ramp_pc_ts = ramp_ts.value/daily_pk_ts*100
th_ramp_pc_ts = th_ramp_ts.value/daily_pk_ts*100

########
ramp_by_gen_tech_ts = (gen_by_tech_ts - gen_by_tech_ts.shift(1)).fillna(0)
ramp_by_gen_subtech_ts = (gen_by_subtech_ts - gen_by_subtech_ts.shift(1)).fillna(0)

th_ramp_by_gen_tech_ts = (gen_by_tech_ts - gen_by_tech_ts.shift(3)).fillna(0)
th_ramp_by_gen_subtech_ts = (gen_by_subtech_ts - gen_by_subtech_ts.shift(3)).fillna(0)
#####

add_df_column(ramp_ts, 'units', 'MW.hr-1').to_csv(os.path.join(save_dir_ts, '08a_ramp_ts.csv'), index=False)
add_df_column(th_ramp_ts, 'units', 'MW.hr-1').to_csv(os.path.join(save_dir_ts, '08b_3hr_ramp_ts.csv'), index=False)
add_df_column(ramp_pc_ts, 'units', 'MW.hr-1').to_csv(os.path.join(save_dir_ts, '08c_ramp_pc_ts.csv'), index=False)
add_df_column(th_ramp_pc_ts, 'units', 'MW.hr-1').to_csv(os.path.join(save_dir_ts, '08d_3hr_ramp_pc_ts.csv'), index=False)

add_df_column(ramp_by_gen_tech_ts, 'units', 'MW.hr-1').to_csv(os.path.join(save_dir_ts, '08e_ramp_by_gen_tech_ts.csv'), index=False)
add_df_column(ramp_by_gen_subtech_ts, 'units', 'MW.hr-1').to_csv(os.path.join(save_dir_ts, '08f_ramp_by_gen_subtech_ts.csv'), index=False)


add_df_column(th_ramp_by_gen_tech_ts, 'units', 'MW.hr-1').to_csv(os.path.join(save_dir_ts, '08g_3hr_ramp_by_gen_tech_ts.csv'), index=False)
add_df_column(th_ramp_by_gen_subtech_ts, 'units', 'MW.hr-1').to_csv(os.path.join(save_dir_ts, '08h_3hr_ramp_by_gen_subtech_ts.csv'), index=False)



if reg_ts:
    
    ### Reg will be only the top order reg by default
    ramp_reg_ts = (net_load_reg_ts.unstack(level='model') - net_load_reg_ts.unstack(level='model').shift(1)).fillna(
        0).stack(level='model').sort_index(level='model').stack(geo_cols).reorder_levels(['model'] + geo_cols +['timestamp'])
    
    th_ramp_reg_ts = (net_load_reg_ts.unstack(level='model') - net_load_reg_ts.unstack(level='model').shift(3)).fillna(
        0).stack(level='model').sort_index(level='model').stack(geo_cols).reorder_levels(['model'] + geo_cols +['timestamp'])
    
    
    #########
    daily_pk_reg_ts = customer_load_reg_ts.groupby([pd.Grouper(level='model'), pd.Grouper(level='timestamp', freq='D')]).max().reindex(customer_load_reg_ts.index).ffill()

    ramp_reg_pc_ts = ramp_reg_ts.unstack(geo_cols)/daily_pk_reg_ts*100
    th_ramp_reg_pc_ts = th_ramp_reg_ts.unstack(geo_cols)/daily_pk_reg_ts*100

    ## These are used in later calculations
    ramp_by_gen_reg_tech_ts = (gen_by_tech_reg_ts - gen_by_tech_reg_ts.shift(1)).fillna(0)
    ramp_by_gen_reg_subtech_ts = (gen_by_subtech_reg_ts - gen_by_subtech_reg_ts.shift(1)).fillna(0)

    th_ramp_by_gen_reg_tech_ts = (gen_by_tech_reg_ts - gen_by_tech_reg_ts.shift(3)).fillna(0)
    th_ramp_by_gen_reg_subtech_ts = (gen_by_subtech_reg_ts - gen_by_subtech_reg_ts.shift(3)).fillna(0)    
    

    
    add_df_column(ramp_reg_ts.unstack(geo_cols), 'units', 'MW.hr-1').to_csv(os.path.join(save_dir_ts, '08g_ramp_reg_ts.csv'), index=False)
    add_df_column(ramp_reg_pc_ts, 'units', 'MW.hr-1').to_csv(os.path.join(save_dir_ts, '08g_ramp_reg_pc_ts.csv'), index=False)
    add_df_column(th_ramp_reg_pc_ts, 'units', 'MW.hr-1').to_csv(os.path.join(save_dir_ts, '08g_3hr_ramp_reg_pc_ts.csv'), index=False)

    add_df_column(th_ramp_reg_ts.unstack(geo_cols), 'units', 'MW.hr-1').to_csv(os.path.join(save_dir_ts, '08h_3hr_ramp_reg_ts.csv'), index=False)
    add_df_column(ramp_by_gen_reg_tech_ts, 'units', 'MW.hr-1').to_csv(os.path.join(save_dir_ts, '08i_ramp_by_gen_tech_reg_ts.csv'), index=False)
    add_df_column(th_ramp_by_gen_reg_tech_ts, 'units', 'MW.hr-1').to_csv(os.path.join(save_dir_ts, '08j_3hr_ramp_by_gen_tech_reg_ts.csv'), index=False)
    add_df_column(ramp_by_gen_reg_subtech_ts, 'units', 'MW.hr-1').to_csv(os.path.join(save_dir_ts, '08k_ramp_by_gen_subtech_reg_ts.csv'), index=False)
    add_df_column(th_ramp_by_gen_reg_subtech_ts, 'units', 'MW.hr-1').to_csv(os.path.join(save_dir_ts, '08l_3hr_ramp_by_gen_subtech_reg_ts.csv'), index=False)


In [ ]:
### Ouput 4: Costs and savings.


gen_cost_props_w_penalty = ['Emissions Cost', 'Fuel Cost', 'Max Energy Violation Cost', 'VO&M Cost',
                  'Min Energy Violation Cost', 'Ramp Down Cost', 'Ramp Up Cost', 'Start & Shutdown Cost']

### Excludes VO&M as this is excluded for some generators because of bad scaling of the objective function
gen_op_cost_props = ['Emissions Cost', 'Fuel Cost', 'Start & Shutdown Cost', 'VO&M Cost']

gen_op_cost_props_wo_co2 = ['Fuel Cost', 'Start & Shutdown Cost', 'VO&M Cost']


### Standard costs reported as USD'000
gen_op_costs = gen_yr_df[gen_yr_df.property.isin(gen_op_cost_props)]
gen_op_costs_wo_co2 = gen_yr_df[gen_yr_df.property.isin(gen_op_cost_props_wo_co2)]
gen_op_costs_w_pen = gen_yr_df[gen_yr_df.property.isin(gen_cost_props_w_penalty)]


try:
    ### Scale costs to be also USD'000
    gen_vom = gen_yr_df[gen_yr_df.property == 'Generation'].fillna(0)
    gen_vom.loc[:,'value'] = gen_vom.apply(lambda x: x.value*x.VOM, axis=1).fillna(0)
    gen_vom.loc[:,'property'] = 'VO&M Cost'
except:
    gen_vom = pd.DataFrame(None)

try:
    gen_fom = gen_yr_df.loc[gen_yr_df.property == 'Installed Capacity',:]
    gen_fom.loc[:,'value'] = gen_fom.apply(lambda x: x.value*x.FOM, axis=1).fillna(0)
    gen_fom.loc[:,'property'] = 'FO&M Cost'
except:
    gen_fom = pd.DataFrame(None)

try:
    gen_capex = gen_yr_df.loc[gen_yr_df.property == 'Installed Capacity',:]
    gen_capex.loc[:,'value'] = gen_capex.apply(lambda x: x.value*x.CAPEX_2037, axis=1).fillna(0)
    gen_capex.loc[:,'property'] = 'Investment Cost'
    
    gen_capex_2030 = gen_yr_df.loc[gen_yr_df.property == 'Installed Capacity',:]
    gen_capex_2030.loc[:,'value'] = gen_capex_2030.apply(lambda x: x.value*x.CAPEX_2030, axis=1).fillna(0)
    gen_capex_2030.loc[:,'property'] = 'Investment Cost 2030'
except:
    gen_capex = pd.DataFrame(None)

    
### Total costs for 2030 and 2037 based on different investment costs. This is very poorly implemented right now
gen_total_costs = pd.concat([gen_op_costs, gen_vom, gen_fom, gen_capex], axis=0)
gen_total_costs_2030 = pd.concat([gen_op_costs, gen_vom, gen_fom, gen_capex_2030], axis=0)
gen_total_costs_w_pen = pd.concat([gen_op_costs_w_pen, gen_vom, gen_fom, gen_capex], axis=0)
gen_total_costs_w_pen_2030 = pd.concat([gen_op_costs_w_pen, gen_vom, gen_fom, gen_capex_2030], axis=0)


gen_op_costs = pd.concat([gen_op_costs, gen_vom], axis=0)
gen_op_cost_wo_co2 = pd.concat([gen_op_costs_wo_co2, gen_vom], axis=0)

### Scale to USDm
gen_op_costs_by_reg = gen_op_costs.groupby(
    ['model']+geo_cols+ ['Category', 'property']).sum().value/1e3
gen_op_costs_wo_co2_by_reg = gen_op_cost_wo_co2.groupby(
    ['model']+geo_cols+ ['Category', 'property']).sum().value/1e3


### Total costs for 2030 and 2037 based on different investment costs. This is very poorly implemented right now
gen_total_costs_by_reg = gen_total_costs.groupby(
    ['model']+geo_cols+ ['Category', 'property']).sum().value/1e3
gen_total_costs_by_reg_w_pen = gen_total_costs_w_pen.groupby(
    ['model']+geo_cols+ ['Category', 'property']).sum().value/1e3
###
gen_total_costs_by_reg_2030 = gen_total_costs_2030.groupby(
    ['model']+geo_cols+ ['Category', 'property']).sum().value/1e3
gen_total_costs_by_reg_w_pen_2030 = gen_total_costs_w_pen_2030.groupby(
    ['model']+geo_cols+ ['Category', 'property']).sum().value/1e3



##TOP an other violation costs
    
#### Non-standard cost elements (i.e. ToP, min CF violations, etc.)
if fuelcontract_yr_df.shape[0] > 0:
    top_cost_reg = fuelcontract_yr_df[fuelcontract_yr_df.property == 'Take-or-Pay Violation Cost']
    top_cost_reg.loc[:,'Category'] = 'Gas'
    top_cost_reg.loc[:,'property'] = 'ToP Violation Cost'
    top_cost_reg.loc[:,geo_cols] = 'CAC'  ### As there is no region!!    
    top_cost_reg = top_cost_reg.groupby(['model'] + geo_cols + ['Category','property']).sum().value/1e3
    top_cost_reg = top_cost_reg.reset_index()
    
else:
    top_cost_reg = pd.DataFrame(None)
    
#### Cost of minimum energy violation for coal contracts

gen_min_vio_coal_reg = gen_yr_df[(gen_yr_df.property == 'Min Energy Violation') & (gen_yr_df.Category == 'Coal')].groupby(['model'] + geo_cols + ['Category','timestamp']).sum().value  ## GWh
gen_avg_price_coal_reg = gen_yr_df[(gen_yr_df.property == 'Average Cost')& (gen_yr_df.Category == 'Coal')].groupby(['model'] + geo_cols + ['Category','timestamp']).sum().value ## $/Mwh
gen_min_vio_cost_coal_reg = gen_min_vio_coal_reg * gen_avg_price_coal_reg ### $'000
gen_min_vio_cost_coal_reg = gen_min_vio_cost_coal_reg.reset_index()
gen_min_vio_cost_coal_reg.loc[:,'property'] = 'PPA Violation Cost'
gen_min_vio_cost_coal_reg = gen_min_vio_cost_coal_reg.groupby(['model'] + geo_cols + ['Category', 'property']).sum().value/1e3

### Ramp costs by reg in USDm
gen_by_name_ts = gen_df[gen_df.property=='Generation'].set_index(['model', 'name'] +geo_cols + ['Category','timestamp']).value
ramp_by_gen_name = (gen_by_name_ts-gen_by_name_ts.shift(1)).fillna(0)
ramp_costs_by_gen_name = pd.merge(ramp_by_gen_name.reset_index(), gen_cost_idx[['name', 'RampCost']], on='name', how='left').set_index(['model', 'name'] +geo_cols + ['Category','timestamp'])
ramp_costs_by_gen_name.loc[:,'value'] = (ramp_costs_by_gen_name.value.abs()*ramp_costs_by_gen_name.RampCost.fillna(0))
ramp_costs_by_gen_name.loc[:,'property'] = 'Ramp Cost'
gen_ramp_costs_by_reg = ramp_costs_by_gen_name.reset_index().groupby(['model'] + geo_cols +['Category', 'property']).sum().value/1e6
gen_op_costs_by_reg = pd.concat([gen_op_costs_by_reg, gen_ramp_costs_by_reg], axis=0).reset_index().groupby(['model'] + geo_cols +['Category', 'property']).sum().value
gen_total_costs_by_reg = pd.concat([gen_total_costs_by_reg, gen_ramp_costs_by_reg], axis=0).reset_index().groupby(['model'] + geo_cols +['Category', 'property']).sum().value
gen_total_costs_by_reg_w_pen = pd.concat([gen_total_costs_by_reg_w_pen, gen_ramp_costs_by_reg], axis=0).reset_index().groupby(['model'] + geo_cols +['Category', 'property']).sum().value


# ### Final dataframes of costs with ToP etc
gen_op_and_vio_costs_reg = pd.concat([gen_op_costs_by_reg.reset_index(), top_cost_reg ], axis=0).groupby(['model'] + geo_cols +['Category', 'property']).sum().value
gen_op_and_vio_costs_reg_wo_co2 = pd.concat([gen_op_costs_wo_co2_by_reg.reset_index(), top_cost_reg ], axis=0).groupby(['model'] + geo_cols +['Category', 'property']).sum().value


### this is different from the vio costs as it includes actual penalty costs as per the model (which are much higher than real violation costs due to modelling considerations)


### USDm/GWh = USD'000/MWh
lcoe_by_tech_reg = (gen_total_costs_by_reg.groupby(['model']+geo_cols+ ['Category']).sum().unstack(
    geo_cols).fillna(0)/gen_by_tech_reg).fillna(0).stack(geo_cols).reorder_levels(['model']+geo_cols+ ['Category'])*1000
lcoe_tech = gen_total_costs_by_reg.groupby(['model', 'Category']).sum()/gen_by_tech_reg.stack(geo_cols).groupby(['model', 'Category']).sum()*1000


add_df_column(gen_op_costs_by_reg, 'units', 'USDm').to_csv(os.path.join(save_dir_sum, '04a_gen_op_costs_reg.csv'),index=False)
add_df_column(gen_op_costs_wo_co2_by_reg, 'units', 'USDm').to_csv(os.path.join(save_dir_sum, '04b_gen_op_costs_wo_co2_reg.csv'),index=False)

add_df_column(gen_op_and_vio_costs_reg, 'units', 'USDm').to_csv(os.path.join(save_dir_sum, '04c_gen_op_and_vio_costs_reg.csv'),index=False)
add_df_column(gen_op_and_vio_costs_reg_wo_co2, 'units', 'USDm').to_csv(os.path.join(save_dir_sum, '04d_gen_op_and_vio_costs_wo_co2_reg.csv'),index=False)


add_df_column(gen_total_costs_by_reg, 'units', 'USDm').to_csv(os.path.join(save_dir_sum, '04e_gen_total_costs_reg.csv'),index=False)
add_df_column(gen_total_costs_by_reg_w_pen, 'units', 'USDm').to_csv(os.path.join(save_dir_sum, '04f_gen_total_costs_reg_w_penalty_costs.csv'),index=False)
add_df_column(gen_total_costs_by_reg_2030, 'units', 'USDm').to_csv(os.path.join(save_dir_sum, '04e_gen_total_costs_reg_2030.csv'),index=False)
add_df_column(gen_total_costs_by_reg_w_pen_2030, 'units', 'USDm').to_csv(os.path.join(save_dir_sum, '04f_gen_total_costs_reg_w_penalty_costs_2030.csv'),index=False)


add_df_column(lcoe_by_tech_reg, 'units', 'USD/MWh').to_csv(os.path.join(save_dir_sum, '04g_lcoe_tech_reg.csv'),index=False)
add_df_column(lcoe_tech, 'units', 'USD/MWh').to_csv(os.path.join(save_dir_sum, '04h_lcoe_tech.csv'),index=False)

In [ ]:
### Output 10: Generation out-of-service 
### Its noted that units out was done per generator. Unsure why, but this may be a more useful output

gen_out_tech_ts = gen_df[(gen_df.property == 'Forced Outage') | \
                    (gen_df.property == 'Maintenance')].groupby(
                    ['model', 'Category', 'timestamp']).sum().value.unstack(level='Category')

gen_out_by_type_ts = gen_df[(gen_df.property == 'Forced Outage') | \
                    (gen_df.property == 'Maintenance')].groupby(
                    ['model', 'property', 'timestamp']).sum().value.unstack(level='property')

add_df_column(gen_out_tech_ts, 'units', 'MW').to_csv(os.path.join(save_dir_ts, '10a_outages_by_tech_ts.csv'), index=False)
add_df_column(gen_out_by_type_ts, 'units', 'MW').to_csv(os.path.join(save_dir_ts, '10b_outages_by_outtype_ts.csv'), index=False)

In [ ]:
### Output 11a: Calculate days of interest & values
### This being max avg net load, min avg net load, max net/total load,, min net/total load, max ramp, min inertia 

net_load_avg = net_load_ts.unstack(level='model').resample('D').mean().stack().reorder_levels(['model', 'timestamp'])
### Time dataframes done nationally////    

try:
    wet_season = gen_by_tech_ts.loc[ix[m,:]]['Hydro'].groupby([pd.Grouper(level='timestamp',freq='M')]).sum().idxmax()
    dry_season = gen_by_tech_ts.loc[ix[m,:]]['Hydro'].groupby([pd.Grouper(level='timestamp',freq='M')]).sum().idxmin()
except KeyError:
    wet_season = gen_by_tech_ts.loc[ix[m,:]]['RoR'].groupby([pd.Grouper(level='timestamp',freq='M')]).sum().idxmax()
    dry_season = gen_by_tech_ts.loc[ix[m,:]]['RoR'].groupby([pd.Grouper(level='timestamp',freq='M')]).sum().idxmin()
    
hi_vre_season  = gen_by_tech_ts.loc[ix[m,:]][['Wind', 'Solar']].sum(axis=1).groupby( pd.Grouper(level='timestamp',freq='M')).idxmax()
low_vre_seaspm =  gen_by_tech_ts.loc[ix[m,:]][['Wind', 'Solar']].sum(axis=1).groupby( [ pd.Grouper(level='timestamp',freq='M')]).sum().idxmin()


##########
net_load_avg_max = net_load_avg.groupby(level='model').max().rename(columns={'value':'net_load_avg_max'})
net_load_avg_min = net_load_avg.groupby(level='model').min().rename(columns={'value':'net_load_avg_min'})
net_load_max = net_load_ts.groupby(level='model').max().rename(columns={'value':'net_load_max'})
net_load_min = net_load_ts.groupby(level='model').min().rename(columns={'value':'net_load_min'})
# net_load_sto_min = net_load_sto_ts.groupby(level='model').min().rename(columns={'value':'net_load_sto_min'})
curtail_max = vre_curtailed_reg_ts.sum(axis=1).groupby(level='model').max().rename('curtail_max').to_frame()

net_load_max_wet = net_load_ts[net_load_ts.index.get_level_values('timestamp').month == wet_season.month].groupby(level='model').max().rename(columns={'value':'net_load_max_wet'})
net_load_min_wet = net_load_ts[net_load_ts.index.get_level_values('timestamp').month == wet_season.month].groupby(level='model').min().rename(columns={'value':'net_load_min_wet'})
net_load_max_dry = net_load_ts[net_load_ts.index.get_level_values('timestamp').month == dry_season.month].groupby(level='model').max().rename(columns={'value':'net_load_max_dry'})
net_load_min_dry = net_load_ts[net_load_ts.index.get_level_values('timestamp').month == wet_season.month].groupby(level='model').min().rename(columns={'value':'net_load_min_dry'})
total_load_max = total_load_ts.groupby(level='model').max().rename('total_load_max').to_frame()
total_load_min = total_load_ts.groupby(level='model').min().rename('total_load_min').to_frame()
ramp_max = ramp_ts.groupby(level='model').max().rename(columns={'value':'ramp_max'})
inertia_min = total_inertia_ts.groupby(level='model').min().drop(columns='InertiaHi').rename(columns={'InertiaLo':'inertia_min'})
use_max = use_ts.groupby(level='model').max().rename('use_max').to_frame()
use_dly_max = use_dly_ts.groupby(level='model').max().rename('use_dly_max').to_frame()

# ###########
net_load_max['time_nlmax'] = net_load_ts.unstack(level='model').idxmax().values
net_load_min['time_nlmin'] = net_load_ts.unstack(level='model').idxmin().values
# net_load_sto_min['time_nlstomin'] = net_load_sto_ts.unstack(level='model').idxmin().values
curtail_max['time_curtailmax'] = vre_curtailed_reg_ts.sum(axis=1).unstack(level='model').idxmax().values

net_load_max_wet['time_nlmax_wet'] = net_load_ts[net_load_ts.index.get_level_values('timestamp').month == wet_season.month].unstack(level='model').idxmax().values
net_load_min_wet['time_nlmin_wet'] = net_load_ts[net_load_ts.index.get_level_values('timestamp').month == wet_season.month].unstack(level='model').idxmin().values
net_load_max_dry['time_nlmax_dry'] = net_load_ts[net_load_ts.index.get_level_values('timestamp').month == dry_season.month].unstack(level='model').idxmax().values
net_load_min_dry['time_nlmin_dry'] = net_load_ts[net_load_ts.index.get_level_values('timestamp').month == dry_season.month].unstack(level='model').idxmin().values
net_load_avg_max['time_nlamax'] = net_load_avg.unstack(level='model').idxmax().values
net_load_avg_min['time_nlamin'] = net_load_avg.unstack(level='model').idxmin().values
total_load_max['time_tlmax'] = total_load_ts.unstack(level='model').idxmax().values
total_load_min['time_tlmin'] = total_load_ts.unstack(level='model').idxmin().values
ramp_max['time_ramp'] = ramp_ts.unstack(level='model').idxmax().values
inertia_min['time_H'] = total_inertia_ts.InertiaLo.unstack(level='model').idxmin().values
use_max['time_usemax'] = use_ts.unstack(level='model').idxmax().values
use_dly_max['time_usedmax'] = use_dly_ts.unstack(level='model').idxmax().values
# ############


doi_summary = pd.concat([net_load_max, net_load_min, curtail_max, net_load_avg_max, net_load_avg_min, net_load_max_wet, net_load_min_wet, net_load_max_dry, net_load_min_dry,
                        total_load_max, total_load_min, ramp_max, inertia_min, use_max, use_dly_max], # net_load_sto_min,
                        axis=1).stack().unstack(level='model').rename_axis('property', axis=0)


doi_summary.to_csv(os.path.join(save_dir_ts, '11a_days_of_interest_summary.csv'), index=True)

In [ ]:
# ### Output 12_NEW: Import/export by region

# node_col = geo_cols[1]
# node_idx = soln_idx[soln_idx.Object_type == 'Node'][geo_cols].set_index(node_col)


# ### Series with indices matching the columns of the DF for filling in missing columns
# reg_filler = pd.Series(data=[1]*len(node_idx.index), index=node_idx.index)
 

# line_flows = line_df[line_df.property == 'Flow']
# line_imports = line_flows.copy()
# line_exports = line_flows.copy()



# line_imports.loc[:,'value'] = [-v if v <=0 else 0 for v in line_imports.value.values  ]
# line_exports.loc[:,'value'] = [v if v > 0 else 0 for v in line_exports.value.values  ]
# imports_by_reg = line_imports.groupby(['model', 'regTo', 'timestamp']).sum().value.reset_index().rename(columns={'regTo':node_col}).set_index(['model',node_col, 'timestamp']).unstack(node_col).droplevel(0,axis=1)
# exports_by_reg = line_exports.groupby(['model', 'regFrom', 'timestamp']).sum().value.reset_index().rename(columns={'regFrom':node_col}).set_index(['model',node_col, 'timestamp']).unstack(node_col).droplevel(0,axis=1)

# imports_by_reg = (imports_by_reg*reg_filler).fillna(0).stack().reset_index()
# exports_by_reg = (exports_by_reg*reg_filler).fillna(0).stack().reset_index()

# ## There will be periods where nodes are both importing and exporting simultaneously
# imports_by_reg = pd.merge(imports_by_reg, node_idx, left_on=node_col, right_index=True).set_index(['model'] + geo_cols + ['timestamp']).rename(columns={0:'Imports'})
# exports_by_reg = pd.merge(exports_by_reg, node_idx, left_on=node_col, right_index=True).set_index(['model'] + geo_cols + ['timestamp']).rename(columns={0:'Exports'})

# imp_exp_by_reg = pd.concat([imports_by_reg.rename(columns={'value':'Imports'}), 
#                           exports_by_reg.rename(columns={'value':'Exports'})], axis=1)


# if reg_ts:
#     for m in model_names:
#         save_dir_model = os.path.join(save_dir_ts, m)
#         if os.path.exists(save_dir_model) is False:
#             os.mkdir(save_dir_model)
            
#         add_df_column(imp_exp_by_reg.loc[ix[m,]], 'units', '%').to_csv(
#             os.path.join(save_dir_model, '12c_imports_exports_by_reg.csv'), index=False)


In [ ]:
# ### Output 12: Time-series (daily) on loading and flows (absolute) on Tx corrdiors


# line_flows = line_df[line_df.property == 'Flow'].groupby(
#     ['model', 'regTo', 'regFrom', 'timestamp']).sum().loc[:,['value']]

# ### Seperate imports from exports
# ### Currently done per line and not at region as the Exports/Imports dont register without Resistance/Reactance included (?)


# line_imports = line_flows.copy()
# line_exports = line_flows.copy()

# line_imports.loc[:,'value'] = [-v if v <=0 else 0 for v in line_flows.value.values  ]
# line_exports.loc[:,'value'] = [v if v > 0 else 0 for v in line_flows.value.values  ]

# line_flows = pd.merge(line_imports.rename(columns={'value':'Imports'}), 
#                         line_exports.rename(columns={'value':'Exports'}),
#                         left_index=True, right_index=True)


# line_loading_exp = line_exports.value/line_cap['Export Limit']*100
# line_loading_imp = line_imports.value/np.abs(line_cap['Import Limit'])*100
# line_loading =  line_loading_imp + line_loading_exp

# line_flows_daily = line_flows.reset_index().groupby(
#     ['model', 'regTo', 'regFrom', pd.Grouper(key='timestamp', freq='D')]).mean()

# line_loading_daily = line_loading.reset_index().groupby(
#     ['model', 'regTo', 'regFrom', pd.Grouper(key='timestamp', freq='D')]).mean()

# add_df_column(line_flows_daily, 'units', 'MW').to_csv(
#     os.path.join(save_dir_ts, '12a_line_flow_daily_ts.csv'), index=False)
# add_df_column(line_loading_daily, 'units', '%').to_csv(
#     os.path.join(save_dir_ts, '12b_line_loading_daily_ts.csv'), index=False)


# for m in model_names:
    
#     save_dir_model = os.path.join(save_dir_ts, m)
#     if os.path.exists(save_dir_model) is False:
#         os.mkdir(save_dir_model)
        
#     model_line_loading = line_loading.reset_index()[line_loading.reset_index().model == m]
#     model_line_flows = line_flows.reset_index()[line_flows.reset_index().model == m]

#     add_df_column(line_flows, 'units', 'MW').reset_index().to_csv(os.path.join(save_dir_model, '17a_line_flow_ts.csv'))
#     add_df_column(line_flows, 'units', '%').reset_index().to_csv(os.path.join(save_dir_model, '17a_line_loading_ts.csv'))



In [ ]:
# ### Output 14: Operational reserves
# ### This could be soemwhere in the SolnIdex....
# res_df.loc[:,'Type'] = res_df.name.str.split('_').str[0]

# res_ts = res_df[res_df.property == 'Risk']
# res_ts.groupby(['model','Type','timestamp']).sum().value.unstack(level='Type')
# res_shorage_ts = res_df[res_df.property == 'Shortage'].groupby(['model','Type','timestamp']).sum().value.unstack(level='Type')

# ## As solution index has the info for generators, not reserves
# res_gen_df['ResType'] = res_gen_df.parent.str.split('_').str[0]

# # reserve_by_reg_ts = res_df.groupby(['model','name','property','timestamp']).sum().value.unstack(level='property')


# spinres_av_ts = res_gen_df[((res_gen_df.property == 'Available Response')| (res_gen_df.property == 'Provision'))& (res_gen_df.ResType == 'Spinning')].groupby(
#     ['model','CapacityCategory', 'Category','timestamp']).sum().value

# regres_av_ts = res_gen_df[((res_gen_df.property == 'Available Response')| (res_gen_df.property == 'Provision'))&(res_gen_df.ResType == 'Regulating')].groupby(
#    ['model','CapacityCategory', 'Category','timestamp']).sum().value

# spinres_prov_ts = res_gen_df[(res_gen_df.property == 'Provision')& (res_gen_df.ResType == 'Spinning')].groupby(
#     ['model','CapacityCategory', 'Category','timestamp']).sum().value

# regres_prov_ts = res_gen_df[(res_gen_df.property == 'Provision')&(res_gen_df.ResType == 'Regulating')].groupby(
#    ['model','CapacityCategory', 'Category','timestamp']).sum().value



# add_df_column(res_ts, 'units', 'MW').to_csv(os.path.join(save_dir_ts, '14a_reserve_ts.csv'), index=False)
# add_df_column(res_shorage_ts, 'units', 'MW').to_csv(os.path.join(save_dir_ts, '14b_reserve_shortage_ts.csv'), index=False)
# add_df_column(spinres_av_ts, 'units', 'MW').to_csv(os.path.join(save_dir_ts, '14c_spinres_av_tech_ts.csv'), index=False)
# add_df_column(regres_av_ts, 'units', 'MW').to_csv(os.path.join(save_dir_ts, '14d_regres_av_tech_ts.csv'), index=False)
# add_df_column(spinres_prov_ts, 'units', 'MW').to_csv(os.path.join(save_dir_ts, '14e_spinres_prov_tech_ts.csv'), index=False)
# add_df_column(regres_prov_ts, 'units', 'MW').to_csv(os.path.join(save_dir_ts, '14f_regres_prov_tech_ts.csv'), index=False)

In [ ]:
# ### Output 15: Services by technology
# '''
# property

# Generation
# InertiaContribution
# PeakContribution
# PeakContribution_actual
# Regulating reserve
# Spinning reserve
# UpRampContribution
# '''

# service_tech_idx = pd.read_excel('Z:/Indonesia/2021_IPSE/09_ModellingSupportFiles/figure_templates/fig00_services_template.xlsx', sheet_name='tech_idx')


# for m in model_names:
    
#     ### All subtechs for consistent solution size/columns
#     subtechs = soln_idx[soln_idx.Object_type == 'Generator'].CapacityCategory.unique()
#     services_out = pd.DataFrame(index=subtechs).rename_axis('Technology')

#     save_dir_model = os.path.join(save_dir_ts, m)
    

# ## A. Energy contribution
#     energy_contr = gen_yr_df[(gen_yr_df.property == 'Generation')&(gen_yr_df.model == m)].groupby('CapacityCategory').sum().value

# ## B. Inertia
#     ### Calc. lowest 100 inertia periods

#     total_inertia = inertia_by_reg.loc[ix[m,]].groupby(level='timestamp').sum().InertiaLo
#     inertia_100 = total_inertia.nsmallest(n=100)
#     stability_100 = total_inertia.nsmallest(n=100)

#     inertia_by_tech100 = gen_inertia.groupby(['model','CapacityCategory','timestamp']).sum().loc[ix[m,]].InertiaLo.unstack(
#         'CapacityCategory').loc[inertia_100.index]
#     inertia_contr = inertia_by_tech100.sum(axis=0)
    
# ### C. Peak contribution
#     ## Calc 100 top periods
#     vre_techs = ['Solar', 'Wind']
# #     gen_by_subtech_ts = gen_df[gen_df.property == 'Generation'].groupby(['model','CapacityCategory','timestamp']).sum().loc[ix[m,]]
#     non_vre_techs = [c for c in gen_by_subtech_ts.columns if c not in vre_techs ]
#     load_100 = customer_load_orig_ts.loc[ix[m,:]].nlargest(100)
#     netload_100 = net_load_orig_ts.loc[ix[m,:]].value.nlargest(100)
    

#     ### NA should be filled up top instead
#     pk_contr = gen_by_subtech_ts.fillna(0).loc[ix[m,netload_100.index],:].droplevel(0).reset_index(drop=True)
#     pk_contr = pk_contr.mean()
#     pk_contr.loc[vre_techs] = 0
#     vre_pk_contr = pd.Series(data=np.mean(load_100.values-netload_100.values), index = ['VRE'])
#     dsm_pk_contr = pd.Series(data=np.mean(net_load_orig_ts.loc[ix[m,:]].value.nlargest(100) - net_load_ts.loc[ix[m,:]].value.nlargest(100)), index=['DSM'])
#     pk_contr = pd.concat([pk_contr, vre_pk_contr, dsm_pk_contr])
   
# #     ### DSM would go here too!
# #     pk_contr = pd.merge(non_vre_pk_contr, vre_pk_contr, left_index=True, right_index=True)
# #     pk_contr = pk_contr.sum()/pk_contr.sum().sum()
    
# ### D. Reserves
#     spin_res_contr = spinres_prov_ts.groupby(['model','CapacityCategory']).sum().loc[ix[m,:],].droplevel(0)
#     spin_res_av_ann_contr = spinres_av_ts.groupby(['model','CapacityCategory']).sum().loc[ix[m,:],].droplevel(0)
#     spin_res_contr = spin_res_contr.mask(spin_res_contr<0).fillna(0)
#     spin_res_av_ann_contr = spin_res_av_ann_contr.mask(spin_res_av_ann_contr<0).fillna(0)
        
#     ## 100 most difficult periods for stability/reserves = highest net load ==> systen is the most strained 
#     spinres_av100 = spinres_av_ts.loc[ix[:,:,:,netload_100.index]]
#     spinres_av100 = spinres_av100.mask(spinres_av100<0).fillna(0)
#     spin_res_av_contr = spinres_av100.groupby(['model','CapacityCategory']).sum().loc[ix[m,:],].droplevel(0)



#     ### Validation model doesnt have reg reserves so, this avoids this from breaking
#     try:
#         reg_res_contr = regres_prov_ts.groupby(['model','CapacityCategory']).sum().loc[ix[m,:],].droplevel(0)
#         reg_res_av_ann_contr = regres_av_ts.groupby(['model','CapacityCategory']).sum().loc[ix[m,:],].droplevel(0)
#         ### Some bug?
#         reg_res_contr = reg_res_contr.mask(reg_res_contr<0).fillna(0)
#         reg_res_av_ann_contr = reg_res_av_ann_contr.mask(reg_res_av_contr<0).fillna(0)
        
#         ## 100 most difficult periods for stability/reserves = lowest inertia periods    
#         regres_av100 = regres_av_ts.loc[ix[:,:,:,netload_100.index]]
#         regres_av100 = regres_av100.mask(regres_av100<0).fillna(0)
#         reg_res_av_contr = reg_res_av_contr.groupby(['model','CapacityCategory']).sum().loc[ix[m,:],].droplevel(0)
#     except KeyError:
#         reg_res_contr = pd.Series(data=np.zeros(len(spin_res_contr)), index = spin_res_contr.index)
#         reg_res_av_ann_contr = pd.Series(data=np.zeros(len(spin_res_av_contr)), index = spin_res_av_contr.index)
#         reg_res_av_contr = pd.Series(data=np.zeros(len(spin_res_av_contr)), index = spin_res_av_contr.index)
        
    
# ### E. Upward ramp contribution
#     ramp_100 = ramp_ts.loc[ix[m,:],:].droplevel(0).value.nlargest(int(0.1*8760*hour_corr))
#     ramp_contr = ramp_by_gen_subtech_ts.loc[ix[m,ramp_100.index],:].droplevel(0)
#     ramp_contr = ramp_contr.mask(ramp_contr < 0).fillna(0).mean()
#     dsm_ramp_contr = pd.Series(data=np.mean(ramp_orig_ts.loc[ix[m,:],:].droplevel(0).value.nlargest(int(0.1*8760*hour_corr)) - ramp_100), index=['DSM'])
#     ramp_contr = pd.concat([ramp_contr, dsm_ramp_contr])
    
# #     services_out = pd.concat([energy_contr, inert])

# ### Out

#     services_out = pd.concat([services_out, energy_contr.rename('Energy'), pk_contr.rename('Peak'), ramp_contr.rename('UpwardRamps'),
#                           spin_res_contr.rename('SpinRes'), reg_res_contr.rename('RegRes'), inertia_contr.rename('Inertia'),
#                               spin_res_av_contr.rename('SpinResAv'), reg_res_av_contr.rename('RegResAv'), ], axis=1).fillna(0).T
    
#     services_out.reset_index().rename(columns={'index':'property'}).to_csv(os.path.join(save_dir_model, '15_services_fig.csv'), index=False)


## Plots

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import calmap
ix = pd.IndexSlice

In [ ]:
import calplot
import numpy as np; np.random.seed(sum(map(ord, 'calplot')))
import pandas as pd

In [ ]:
save_dir_plots = os.path.join(save_dir, 'plots/')
if os.path.exists(save_dir_plots) is False:
            os.mkdir(save_dir_plots)

In [ ]:
model_names

***

### Plot functions and palettes

In [ ]:
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib import colors


iea_palette = {'grey5': '#f2f2f2','grey10':'#e6e6e6','pl':'#b187ef', 'bl':'#49d3ff', 'tl':'#00e0e0', 'gl':'#68f394', 'yl':'#fff45a',
               'ol':'#ffb743', 'rl':'#ff684d', 'gl':'#68f394', 'yl':'#fff45a','grey40':'#949494','grey50':'#6f6f6f',
               'p':'#af6ab1', 'b':'#3e7ad3', 't':'#00ada1', 'g':'#1dbe62', 'y':'#fed324',
               'o':'#f1a800', 'r':'#e34946', 'grey20':'#afafaf', 'black':'#000000', 'white':'#ffffff', 'iea_b':'#0044ff', 'iea_b50':'#80a2ff' }

extended_palette = dict( {'{}'.format(i): plt.matplotlib.colors.rgb2hex(plt.cm.get_cmap('tab20b').colors[i]) for i in np.arange(0,20)},
                        **{'{}'.format(i+20): plt.matplotlib.colors.rgb2hex(plt.cm.get_cmap('tab20c').colors[i]) for i in np.arange(0,20)})

### For overflow, i.e. things that go beyond the 16 or so colors of IEA palette. 
iea_palette_plus = dict(iea_palette, **extended_palette)

In [ ]:
tech_palette = { 'Coal':'grey20', 'Abated coal':'grey10', 'Cofiring':'grey10','Gas':'p', 'Abated gas':'p', 'Oil':'grey50','Hydro':'bl','Geothermal':'r','Bioenergy':'gl','Solar':'y','Wind':'g', 'Fuel Cell':'t', 'Other':'t', 'Battery':'b','Storage':'b'}

# model_palette = {'2019':'rl', '2025 Base':'o', '2025 SolarPlus':'bl', '2025 SolarPlus Lite':'pl',  '2025 SolarPlus Extra':'gl' }


### We should add load vs customer load (to see charging effect),
### Similarly could add load + exports to get effective load
stack_palette = {'Geothermal':'o','Bioenergy':'gl', 'Coal':'grey20', 'Cofiring':'grey5', 'Abated coal':'grey5', 'Gas':'p', 'Abated gas':'pl', 'Hydro':'bl', 'Oil':'grey50', 'Imports':'t', 'Other':'t', 'Fuel Cell':'tl', 'Storage':'b',
                 'Solar':'y','Wind':'g', 'Total Load':'black', 'Load2':'white', 'Exports':'p','Net Load': 'r', 'Curtailment':'yl','Unserved Energy':'iea_b', 'Underlying Load':'p' , 'Storage Load':'grey50', 'Nuclear': 'r'}

reg_palette = {'CAC':'b', 'MAC':'rl', 'NAC':'pl', 'NEC':'t','SAC':'ol', 'UKR': 'grey20'}
iea_cmap_regs = colors.ListedColormap([ iea_palette[reg_palette[i]] for i in reg_palette.keys() if i != 'UKR'])
# subreg_palette = {'CAC':'bl', 'MAC':'b', 'NAC':'gl', 'NEC':'t', 'SAC':'ol', 'UKR': 'grey20'}

#### General palette for WEO tech stuff which has > 20 items
# weo_Tech_palette = { tech : '{}'.format(i) for i, tech in enumerate(gen_yr_df.WEO_Tech_simpl.unique())}


iea_palette_l8 = ['rl', 'ol', 'gl', 'bl', 'pl', 'grey10', 'yl', 'tl'] ### got rid of light yellow as its a poor choice for plots.
iea_palette_d8 = ['r', 'o', 'y', 'g', 't', 'b', 'p', 'grey50']
iea_palette_16 = iea_palette_l8 + iea_palette_d8


iea_palette_14 = ['rl', 'ol', 'bl', 'gl', 'pl', 'grey10', 'y', 'tl',  'g', 't', 'b', 'grey50', 'yl', 'r', 'p']
iea_cmap_l8 = colors.ListedColormap([ iea_palette[c] for c in iea_palette_l8])
iea_cmap_d8 = colors.ListedColormap([ iea_palette[c] for c in iea_palette_d8])
iea_cmap_16 = colors.ListedColormap([ iea_palette[c] for c in iea_palette_16])
iea_cmap_14 = colors.ListedColormap([ iea_palette[c] for c in iea_palette_14])

tab20bc = colors.ListedColormap([ extended_palette[i] for i in extended_palette.keys()])

model_palette = dict(zip([m for m in pretty_model_names.values() if m in model_names], iea_palette_14[:len(model_names)]))

combined_palette = dict(tech_palette, **reg_palette, **model_palette) #, **weo_Tech_palette)

In [ ]:
def write_xlsx_scatter(df, writer, excel_file=None, sheet_name='Sheet1', colour=None, palette=combined_palette, units='', alpha=80, to_combine=False, markersize=4, common_yr=2041):

    cm_to_pixel = 37.7953
    
    
    if excel_file:
        writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')
        
    df.to_excel(writer, sheet_name=sheet_name)

    # Access the XlsxWriter workbook and worksheet objects from the dataframe.
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]
    
    if units == '':
        num_fmt = '0%'
        units = ''
    else:
        num_fmt = '# ###'
        

    # Create a chart object.
    chart = workbook.add_chart({'type': 'scatter'})
    chart.set_size({'width': 15*cm_to_pixel, 'height': 8.5*cm_to_pixel})


    if to_combine:
        
        chart.set_plotarea({
            'layout': {
                'x':      0.1,
                'y':      0,
                'width':  0.9,
                'height': 0.8,
            },
            'fill':{'none':True}
        })
    else:
        chart.set_plotarea({
    #         'layout': {
    #             'x':      0.1,
    #             'y':      0,
    #             'width':  0.9,
    #             'height': 0.8,
    #         },
            'fill':{'none':True}
        })
        
        
    chart.set_chartarea({
        'fill':{'none':True},
        'border': {'none': True},

    })
    


    
    
    
    # Configure the series of the chart from the dataframe data.
    ## Col_num iterates from first data column, which varies if it is multiindex columns or not
    for col_num in np.arange(1, df.shape[1] + 1):
        
        if colour != None:
            fill_colour = colour
        else:
            try:
                fill_colour = iea_palette_plus[palette[df.columns[col_num-df.index.nlevels]]]
            except KeyError:
                fill_colour = iea_cmap_16.colors[col_num-df.index.nlevels]
            
        chart.add_series({
            'name':       [sheet_name, 0, col_num],
            'categories': [sheet_name, 1, 0, df.shape[0], df.index.nlevels-1],
            'values':     [sheet_name, 1, col_num, df.shape[0], col_num],
            'marker': { 'type': 'circle',
                        'size': markersize,
                        'border': {'color': fill_colour},
                        'fill':   {'color':fill_colour, 'transparency':alpha},
                    },

        })

    # Configure the chart axes.
    ### Set label_position to low if there are negative values
    if( df < 0).sum().sum() > 0:
        label_position = 'low'
    else:
        label_position = 'next_to'
    


    chart.set_x_axis({ 'num_font':  {'name': 'Arial', 'size': 10},
                       'num_format': 'mmm',
                       'date_axis': True,
                       'min': pd.to_datetime('01-01-{}'.format(common_yr)),
                       'max': pd.to_datetime('31-12-{}'.format(common_yr)),
                      'major_unit': 31,
                      'interval_unit':31,
                      'interval_tick':31,
                      'label_position':label_position
                     })
    
    
    chart.set_y_axis({'major_gridlines': {'visible': False},
                      'num_font':  {'name': 'Arial', 'size': 10},
                      'num_format':num_fmt,
                      'name': units,
                      'name_font': {'name': 'Arial', 'size': 10, 'bold':False, 'text_rotation':-90},
                      'name_layout': { 'x': 0.02, 'y': 0.02 },
                      'line': {'none': True},
                      'major_gridlines': {
                            'visible': True,
                            'line': {'width': 1, 'color' : '#d9d9d9'}}
                      })
    
    chart.set_title({'none': True})
    
    chart.set_legend({'font':  {'name': 'Arial', 'size': 10},
                      'position': 'bottom',
#                       'layout': {'x':      0,
#                                 'y':      0.7,
#                                 'width':  1,
#                                 'height': 0.25
#                                },
#                      'delete_series': [leg_del_idx]
                     })

                      
#     chart.set_legend({'num_font':  {'name': 'Arial', 'size': 10}})

    # Insert the chart into the worksheet....this probably should depend on the size of the dataframe
    if to_combine:
        worksheet.insert_chart('K22', chart)
    else:
        worksheet.insert_chart('K2', chart)

In [ ]:
def write_xlsx_line(df, writer, excel_file=None, sheet_name='Sheet1', subtype='timeseries', palette=combined_palette, units='', ldc_idx=None, label_position='next_to', to_combine=False,line_width=1.5):

    cm_to_pixel = 37.7953
    
    ## Sort columns by order in palettes described above
    sort_cols = [c for c in palette.keys() if c in df.columns] + [c for c in df.columns if c not in palette.keys()]
    sort_index = [i for i in palette.keys() if i in df.index] + [i for i in df.index if i not in palette.keys()]

    if type(df.index) == pd.Index:
        df = df.loc[sort_index, sort_cols]
    else:
        df = df.loc[:,sort_cols]
    
   
    if subtype == 'ldc':
        if ldc_idx is None:
            df.index = (np.arange(0,df.shape[0]) + 1)/df.shape[0]
        else:
            df.index = ldc_idx            
    
    if excel_file:
        writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')
        
    df.to_excel(writer, sheet_name=sheet_name)

    # Access the XlsxWriter workbook and worksheet objects from the dataframe.
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]
    
    if units == '':
        num_fmt = '0%'
        units = ''
    else:
        num_fmt = '# ###'
        
    if subtype == 'timeseries':
        chart = workbook.add_chart({'type': 'line'})
    else:
        chart = workbook.add_chart({'type': 'scatter', 'subtype': 'line'})

    # Create a chart object.
    if to_combine:
        chart.set_size({'width': 7.5*cm_to_pixel, 'height': 7*cm_to_pixel})
    else:
        chart.set_size({'width': 15*cm_to_pixel, 'height': 7*cm_to_pixel})


    if to_combine:
        
        chart.set_plotarea({
            'layout': {
                'x':      0.25,
                'y':      0,
                'width':  0.75,
                'height': 0.65,
            },
            'fill':{'none':True}
        })
    else:
        chart.set_plotarea({
            'layout': {
                'x':      0.25,
                'y':      0,
                'width':  0.75,
                'height': 0.65,
            },
            'fill':{'none':True}
        })
        
        
    chart.set_chartarea({
        'fill':{'none':True},
        'border': {'none': True},

    })
    
    
    # Configure the series of the chart from the dataframe data.
    ## Col_num iterates from first data column, which varies if it is multiindex columns or not
    for col_num in np.arange(df.index.nlevels, df.shape[1] + df.index.nlevels):
        
        try:
            line_colour = iea_palette_plus[palette[df.columns[col_num-df.index.nlevels]]]
        except KeyError:
            line_colour = iea_cmap_16.colors[col_num-df.index.nlevels]
            
        chart.add_series({
            'name':       [sheet_name, 0, col_num],
            'categories': [sheet_name, 1, 0, df.shape[0], df.index.nlevels-1],
            'values':     [sheet_name, 1, col_num, df.shape[0], col_num],
            'line':   {'color': line_colour,
                       'width':line_width},
            'marker': {'type': 'none'}
        })
        
        
    ### Set label_position to low if there are negative values
    if( df < 0).sum().sum() > 0:
        label_position = 'low'
    else:
        label_position = 'next_to'
    

    chart.set_x_axis({'num_font':  {'name': 'Arial', 'size': 10},
                      'num_format': 'dd mmm hh:mm',
                      'major_unit': 24,
                      'interval_unit':24,
                      'interval_tick':12,
                      'line': {'color': 'black'},
                      'text_axis': True,
                      'label_position':label_position})
    

    if subtype == 'ldc':
        if np.round(np.max(df.index)/5,-1) < 1: 
            chart.set_x_axis({'num_font':  {'name': 'Arial', 'size': 10},
                         'line': {'color': 'black'},
                             'major_unit':np.round(np.max(df.index)/5,-1),
                             'min':0,
                             'max':np.max(df.index),
                             'num_format': '0.0%',
                             'label_position':label_position
                             })
        else:
            chart.set_x_axis({'num_font':  {'name': 'Arial', 'size': 10},
             'line': {'color': 'black'},
                 'major_unit':np.round(np.max(df.index)/5,-1),
                 'min':0,
                 'max':np.max(df.index),
                 'num_format': '0%',
                 'label_position':label_position
                 })
    else:
        
        chart.set_x_axis({'num_font':  {'name': 'Arial', 'size': 10},
                     'line': {'color': 'black'},
                     'num_format': 'mmm',
                      'major_unit': 30,
                      'label_position':label_position})
    
    
    chart.set_y_axis({'major_gridlines': {'visible': False},
                      'num_font':  {'name': 'Arial', 'size': 10},
                      'num_format':num_fmt,
                      'name': units,
                      'name_font': {'name': 'Arial', 'size': 10, 'bold':False, 'text_rotation':-90},
                      'name_layout': { 'x': 0.02, 'y': 0.02 },
                      'line': {'none': True},
                      'major_gridlines': {
                            'visible': True,
                            'line': {'width': 1, 'color' : '#d9d9d9'}}
                      })
    
    chart.set_title({'none': True})
    
    if df.shape[1] > 1:
    
        chart.set_legend({'font':  {'name': 'Arial', 'size': 10},
                      'position':'bottom',
                      'layout': {'x': 0,
                                'y':      0.85,
                                'height': 0.15,
                                'width':1
                               }}
                        )
        
        
#         max_chars = np.max([len(c) for c in df.columns])
#         min_width = 0.075
#         width = min_width + max_chars*0.01
        
#         chart.set_legend({'font':  {'name': 'Arial', 'size': 10},
#                           'layout': {'x': 1- width,
#                                     'y':      0,
#                                     'height': 1,
#                                     'width':width
#                                    }})
    else:
        chart.set_legend({'visble':False})
        chart.set_legend({'position': 'none'})

    # Insert the chart into the worksheet....this probably should depend on the size of the dataframe
    
    if to_combine:
        worksheet.insert_chart('K22', chart)
    else:
        worksheet.insert_chart('K2', chart)

In [ ]:
def write_xlsx_column(df, writer, excel_file=None, sheet_name='Sheet1', palette=combined_palette, subtype='stacked', units='', total_scatter_col=None, total_fill=iea_palette['white'], to_combine=False, right_ax=None):

    cm_to_pixel = 37.7953
    
    ## Sort columns by order in palettes described above
    sort_cols = [c for c in palette.keys() if c in df.columns] + [c for c in df.columns if c not in palette.keys()]
    sort_index = [i for i in palette.keys() if i in df.index] + [i for i in df.index if i not in palette.keys()]

    if type(df.index == pd.Index):
        df = df.loc[sort_index, sort_cols]
    else:
        df = df.loc[:,sort_cols]
    

    if excel_file:
        writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')
    
    ### Whether we caluclate the scatter col or not. Should probably rename the variable from total_col, as its not always a total
    if (total_scatter_col != None)&(total_scatter_col not in df.columns):
        df.loc[:,total_scatter_col] = df.sum(axis=1)
        
    df.to_excel(writer, sheet_name=sheet_name)
        

    # Access the XlsxWriter workbook and worksheet objects from the dataframe.
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]
    
    if units == '%':
        num_fmt = '0%'
        units = ''
    else:
        num_fmt = '# ###'
        
    
    # Create a chart object.
    chart = workbook.add_chart({'type': 'column', 'subtype': subtype})
    chart.set_size({'width': 15*cm_to_pixel, 'height': 7*cm_to_pixel})

    if to_combine:
        
        chart.set_plotarea({
            'layout': {
                'x':      0.1,
                'y':      0,
                'width':  0.9,
                'height': 0.8,
            },
            'fill':{'none':True}
        })
    else:
        chart.set_plotarea({
    #         'layout': {
    #             'x':      0.1,
    #             'y':      0,
    #             'width':  0.9,
    #             'height': 0.8,
    #         },
            'fill':{'none':True}
        })
        

    chart.set_chartarea({
        'fill':{'none':True},
        'border': {'none': True},

    })
    
    if (total_scatter_col != None) & (df.shape[1] > 1):
        chart2 = workbook.add_chart({'type': 'scatter'})
    
    
    
    for col_num in np.arange(df.index.nlevels, df.shape[1] + df.index.nlevels):
        if df.columns[col_num-df.index.nlevels] != total_scatter_col:
                # Configure the series of the chart from the dataframe data.
            ## Col_num iterates from first data column, which varies if it is multiindex columns or not

            try:
                fill_colour = iea_palette_plus[palette[df.columns[col_num-df.index.nlevels]]]
            except KeyError:
                fill_colour = iea_cmap_16.colors[col_num-df.index.nlevels]

            # fill_colour = matplotlib.colors.rgb2hex(plt.cm.get_cmap('tab20c').colors[20-col_num-df.index.nlevels])


    # Or using a list of values instead of category/value formulas:
#     [sheetname, first_row, first_col, last_row, last_col]

# Or using a list of values instead of category/value formulas:
#     [sheetname, first_row, first_col, last_row, last_col]
            chart.add_series({
                'name':       [sheet_name, 0, col_num],
                'categories': [sheet_name, 1, 0, df.shape[0], df.index.nlevels-1],
                'values':     [sheet_name, 1, col_num, df.shape[0], col_num],
                'gap':        75,
                'fill':       {'color': fill_colour, 'border':'#000000'},
                'border': {'color': '#000000'}
            })
        else:
            if right_ax != None:
                chart2.add_series({
                    'name':       [sheet_name, 0, col_num],
                    'categories': [sheet_name, 1, 0, df.shape[0], df.index.nlevels-1],
                    'values':     [sheet_name, 1, col_num, df.shape[0], col_num],
                    'marker': { 'type': 'circle',
                                'size': 8,
                                    'border': {'color': '#000000'},
                                    'fill':   {'color': '#ffffff', 'transparency':30},
                                },
                    'y2_axis':    True

                })
            else:
                chart2.add_series({
                    'name':       [sheet_name, 0, col_num],
                    'categories': [sheet_name, 1, 0, df.shape[0], df.index.nlevels-1],
                    'values':     [sheet_name, 1, col_num, df.shape[0], col_num],
                    'marker': { 'type': 'circle',
                                'size': 7,
                                    'border': {'color': '#000000'},
                                    'fill':   {'color': total_fill, 'transparency':0},
                                }
                })

                
            
        ### This is the total column and will always be last, so we can just combine here and save an if/else loop
            chart.combine(chart2)
     
    ### Set label_position to low if there are negative values
    if( df < 0).sum().sum() > 0:
        label_position = 'low'
    else:
        label_position = 'next_to'
        

            

    chart.set_x_axis({'num_font':  {'name': 'Arial', 'size': 10},
                     'line': {'color': 'black'},
                     'label_position':label_position})
    
    
    chart.set_y_axis({'major_gridlines': {'visible': False},
                      'num_font':  {'name': 'Arial', 'size': 10},
                      'num_format':num_fmt,
                      'name': units,
                      'name_font': {'name': 'Arial', 'size': 10, 'bold':False, 'text_rotation':-90},
                      'name_layout': { 'x': 0.02, 'y': 0.02 },
                      'line': {'none': True},
                      'major_gridlines': {
                            'visible': True,
                            'line': {'width': 1, 'color' : '#d9d9d9'}},
                      })
    
    if right_ax != None:
        if df.shape[1] > 1:
            max_chars = np.max([len(c) for c in df.columns] )
            min_width = 0.075
            width = min_width + max_chars*0.01
        else:
            width= 0
        
        
        chart2.set_y2_axis({'major_gridlines': {'visible': False},
                      'num_font':  {'name': 'Arial', 'size': 10},
                      'num_format':num_fmt,
                      'name': right_ax,
                      'name_font': {'name': 'Arial', 'size': 10, 'bold':False, 'text_rotation':-90},
                      'name_layout': { 'x': 0.98 - width, 'y': 0.02 },
                      'line': {'none': True},
                      'major_gridlines': {
                            'visible': True,
                            'line': {'width': 1, 'color' : '#d9d9d9'}},
                      })
        
    
    chart.set_title({'none': True})
    
    if df.shape[1] > 1:
        max_chars = np.max([len(c) for c in df.columns] )
        
        ### Legend should not exceed 16chars and should always be more than 8chars
        
        if max_chars <8:
            max_chars=8
        elif max_chars >16:
            max_chars =16
        
        min_width = 0.075
        width = min_width + max_chars*0.01
        
        chart.set_legend({'font':  {'name': 'Arial', 'size': 10},
                          'layout': {'x': 1- width,
                                    'y':      0,
                                    'height': 1,
                                    'width':width
                                   }}
                        )
    else:
        chart.set_legend({'visble':False } )
        chart.set_legend({'position': 'none'})

    # Insert the chart into the worksheet....this probably should depend on the size of the dataframe
    if to_combine:
        worksheet.insert_chart('K22', chart)
    else:
        worksheet.insert_chart('K2', chart)

In [ ]:
def write_xlsx_stack(df, writer, excel_file=None, sheet_name='Sheet1', palette=stack_palette, units='MW', to_combine=False):

    cm_to_pixel = 37.7953
    
    ## Sort columns by order in palettes described above
    sort_cols = [c for c in palette.keys() if c in df.columns] + [c for c in df.columns if c not in palette.keys()]
    df = df.loc[:, sort_cols]
    
    if excel_file:
        writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')
          
    df.to_excel(writer, sheet_name=sheet_name)

    # Access the XlsxWriter workbook and worksheet objects from the dataframe.
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]
          

    # Create a chart object.
    chart = workbook.add_chart({'type': 'area', 'subtype': 'stacked'})
    chart2 = workbook.add_chart({'type': 'line'})

    
    sec_axis_vars = ['Load2', 'Curtailment']
    write_xlsx_stack= ['Net Load', 'Load']
    
    # Configure the series of the chart from the dataframe data.
    ## Col_num iterates from first data column, which varies if it is multiindex columns or not
    for col_num in np.arange(df.index.nlevels, df.shape[1] + df.index.nlevels):
        
        try:
            fill_colour = iea_palette_plus[palette[df.columns[col_num-1]]]
        except KeyError:
            print('Non-specified colour for: {}'.format(df.columns[col_num-1]))
            fill_colour = iea_cmap_16.colors[col_num-1]
            
        if df.columns[col_num-1] == 'Load2':
            
            chart.add_series({
                'name':       [sheet_name, 0, col_num],
                'categories': [sheet_name, 1, 0, df.shape[0], 0],
                'values':     [sheet_name, 1, col_num, df.shape[0], col_num],
                'fill':       {'none': True},
                'border': {'none': True},
                'y2_axis': True
            })
            
            leg_del_idx = [ int(col_num -1) ]
            
        elif df.columns[col_num-1] == 'Curtailment':
            chart.add_series({
                'name':       [sheet_name, 0, col_num],
                'categories': [sheet_name, 1, 0, df.shape[0], 0],
                'values':     [sheet_name, 1, col_num, df.shape[0], col_num],
                'pattern': { 'pattern':  'light_upward_diagonal',
                             'fg_color': iea_palette['y'],
                             'bg_color': iea_palette['r']
                            },
                'border': {'none': True},
                'y2_axis': True
            })
        elif df.columns[col_num-1] == 'Total Load':
            chart2.add_series({
                'name':       [sheet_name, 0, col_num],
                'categories': [sheet_name, 1, 0, df.shape[0], 0],
                'values':     [sheet_name, 1, col_num, df.shape[0], col_num],
                'line': {'width': 0.25, 'color':'black', 'dash_type': 'solid'},
            })   
        elif df.columns[col_num-1] == 'Underlying Load':
            continue
        elif df.columns[col_num-1] == 'Storage Load':
            continue
#             chart2.add_series({
#                 'name':       [sheet_name, 0, col_num],
#                 'categories': [sheet_name, 1, 0, df.shape[0], 0],
#                 'values':     [sheet_name, 1, col_num, df.shape[0], col_num],
#                 'line': {'width': 1.00, 'color':iea_palette['p'], 'dash_type': 'dash'},
#             })   
        elif df.columns[col_num-1] == 'Net Load':
            chart2.add_series({
                'name':       [sheet_name, 0, col_num],
                'categories': [sheet_name, 1, 0, df.shape[0], 0],
                'values':     [sheet_name, 1, col_num, df.shape[0], col_num],
                'line': {'width': 1.00, 'color':iea_palette['r'], 'dash_type': 'dash'},
            })
        else:
            chart.add_series({
                'name':       [sheet_name, 0, col_num],
                'categories': [sheet_name, 1, 0, df.shape[0], 0],
                'values':     [sheet_name, 1, col_num, df.shape[0], col_num],
                'fill':       {'color': fill_colour},
                'border': {'none': True}
            })
            
        
    # Configure the chart axes.
    num_fmt = '# ###'
    
    chart.combine(chart2)
    
        # Create a chart object.
    if to_combine:
        chart.set_size({'width': 7.5*cm_to_pixel, 'height': 5.5*cm_to_pixel})
    else:
        chart.set_size({'width': 15*cm_to_pixel, 'height': 7*cm_to_pixel})



        
    chart.set_plotarea({
        'layout': {
            'x':      0.1,
            'y':      0,
            'width':  0.9,
            'height': 0.8,
        },
        'fill':{'none':True}
    })

        
    chart.set_chartarea({
        'fill':{'none':True},
        'border': {'none': True},

    })
    
    
    ### Set label_position to low if there are negative values
    if( df < 0).sum().sum() > 0:
        label_position = 'low'
    else:
        label_position = 'next_to'
    

    chart.set_x_axis({'num_font':  {'name': 'Arial', 'size': 10},
                      'num_format': 'dd mmm',
                      'major_unit': 24,
                      'interval_unit':24,
                      'interval_tick':12,
                      'line': {'color': 'black'},
                      'text_axis': True,
                      'label_position':label_position})
    
    
    chart.set_y_axis({'major_gridlines': {'visible': False},
                      'num_font':  {'name': 'Arial', 'size': 10},
                      'num_format':num_fmt,
                      'name': units,
                      'name_font': {'name': 'Arial', 'size': 10, 'bold':False, 'text_rotation':-90},
                      'name_layout': { 'x': 0.02, 'y': 0.02 },
                      'line': {'none': True},
                      'major_gridlines': {
                            'visible': True,
                            'line': {'width': 1, 'color' : '#d9d9d9'}}
                      })
                      
    chart.set_y2_axis({'visible':False})
                      
#     leg_del_idx = df.shape[1] 

    if 'Load2' in df.columns:
    
        chart.set_legend({'font':  {'name': 'Arial', 'size': 10},
                          'position': 'bottom',
                          'layout': {'x':      0,
                                    'y':      0.7,
                                    'width':  1,
                                    'height': 0.25
                                   },
                         'delete_series': leg_del_idx
                         })
    else:
        chart.set_legend({'font':  {'name': 'Arial', 'size': 10},
                  'position': 'bottom',
                  'layout': {'x':      0,
                            'y':      0.7,
                            'width':  1,
                            'height': 0.25
                           }
                 })
        
    # Insert the chart into the worksheet.
    if to_combine == True:
        worksheet.insert_chart('S32', chart)
    else:
        worksheet.insert_chart('S2', chart)
    


### Thaialand-specific, floating hydro outputs

In [ ]:
hydro_by_model_ts = gen_by_tech_ts.Hydro.unstack('model')

In [ ]:
northern_hydro_ts = gen_by_tech_reg_ts[(gen_by_tech_reg_ts.index.get_level_values('Region') == 'NEC')|(gen_by_tech_reg_ts.index.get_level_values('Region') == 'NAC')].Hydro.unstack('model').droplevel(geo_cols)

In [ ]:
# hy_plots = {'hy_monthly': hydro_by_model_ts.groupby(pd.Grouper(level='timestamp', freq='M')).sum()/1e6,
#             'hy_weekly':hydro_by_model_ts.groupby(pd.Grouper(level='timestamp', freq='W')).sum()/1e6,
#             'hy_daily':hydro_by_model_ts.groupby(pd.Grouper(level='timestamp', freq='D')).sum()/1e3,
#             'hy_avg': hydro_by_model_ts.groupby([hydro_by_model_ts.index.hour]).mean(),
            
#             'nor_hy_monthly': northern_hydro_ts.groupby(pd.Grouper(level='timestamp', freq='M')).sum()/1e6,
#             'nor_hy_weekly':northern_hydro_ts.groupby(pd.Grouper(level='timestamp', freq='W')).sum()/1e6,
#             'nor_hy_daily':northern_hydro_ts.groupby(pd.Grouper(level='timestamp', freq='D')).sum()/1e3,
#             'nor_hy_avg': northern_hydro_ts.groupby([northern_hydro_ts.index.hour]).mean(),
            
#             'sto_monthly': sto_by_model_ts.groupby(pd.Grouper(level='timestamp', freq='M')).sum()/1e6,
#             'sto_avg': sto_by_model_ts.groupby([sto_by_model_ts.index.hour]).mean(),
            
#             'hy_sto_plus_monthly': hydro_sto_by_model_ts.groupby(pd.Grouper(level='timestamp', freq='M')).sum()/1e6,
#             'hy_sto_plus_avg': hydro_sto_by_model_ts.groupby([hydro_sto_by_model_ts.index.hour]).mean(),
#            }

# hy_plot_units = {'hy_monthly': 'TWh',
#             'hy_weekly':'TWh',
#             'hy_daily':'GWh',
#             'hy_avg': 'MW',
            
#             'nor_hy_monthly': 'TWh',
#             'nor_hy_weekly': 'TWh',
#             'nor_hy_daily': 'GWh',
#             'nor_hy_avg': 'MW',
                 
#             'sto_monthly': 'TWh',
#             'sto_avg':'MW',
#             'hy_sto_plus_monthly':'TWh',
#             'hy_sto_plus_avg': 'MW',
                 
                
#            }

# hy_plot_type = {'hy_monthly': 'line',
#             'hy_weekly':'line',
#             'hy_daily':'line',
#             'hy_avg': 'line',
    
                 
#             'nor_hy_monthly': 'line',
#             'nor_hy_weekly': 'line',
#             'nor_hy_daily': 'line',
#             'nor_hy_avg': 'line',
                
                
#             'sto_monthly': 'line',
#             'sto_avg':'line',
#             'hy_sto_plus_monthly':'line',
#             'hy_sto_plus_avg': 'line',
#            }


In [ ]:
# ### Plot 2:  Hydro summary plots by columnn

# fig_path = os.path.join(save_dir_plots,'plot2_hy_output.xlsx')

# with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:
    
#     for i in hy_plots.keys():
#         if hy_plots[i].shape[0] == 0:
#             print('Empty dataframe for: {}'.format(i))
#         elif hy_plot_type[i] == 'col':
#             write_xlsx_column(df=hy_plots[i], writer=writer, sheet_name=i, units=hy_plot_units[i], palette=combined_palette)
#         else:
#             write_xlsx_line(df=hy_plots[i], writer=writer, sheet_name=i, units=hy_plot_units[i], line_width=0.1) 

            
    
    

### Excel plots and exports

In [ ]:
reg_ids = list(np.unique(np.append(load_by_reg.unstack(geo_cols).droplevel(level=[c for c in geo_cols if c != geo_cols[0]], axis=1).replace(0, np.nan).dropna(how='all', axis=1).columns,
gen_by_tech_reg.droplevel(level=[c for c in geo_cols if c != geo_cols[0]], axis=1).replace(0, np.nan).dropna(how='all', axis=1).columns)))

model_regs = reg_ids + ['UKR']


In [ ]:
### Output 16: Generation stacks
    
### Combine variables together to make a stack DF
### Return only those regions which have anything modelled


pumpload_reg_ts = node_df[(node_df.property == 'Pump Load')|(node_df.property == 'Battery Load')].groupby(['model'] + geo_cols + ['timestamp']).sum().value.rename('Storage Load')
underlying_load_reg = (load_by_reg_ts - pumpload_reg_ts).rename('Underlying Load')
net_load_reg_sto_ts = (net_load_reg_ts.stack(geo_cols).reorder_levels(['model'] + geo_cols + ['timestamp']) + pumpload_reg_ts).rename('Net Load')

gen_stack_by_reg = pd.concat([gen_by_tech_reg_ts.fillna(0), 
#                               line_flows, 
                              net_load_reg_sto_ts,
                              underlying_load_reg,
                              pumpload_reg_ts,
                              load_by_reg_ts.rename('Total Load'),
                              load_by_reg_ts.rename('Load2'),
                              vre_curtailed_reg_ts.stack(geo_cols).reorder_levels(['model'] + geo_cols + ['timestamp']).rename('Curtailment'),
                              use_reg_ts.stack(geo_cols).reorder_levels(['model'] + geo_cols + ['timestamp']).rename('Unserved Energy'), 
                              ], axis=1)

### Drop specific columns...order is currently set in the function, though this can be contreolled by a flag

### Get island and total summaries..... 
# gen_stack_by_isl  = gen_stack_by_reg.groupby(['model', geo_cols[0], 'timestamp']).sum().reset_index()
gen_stack_total = gen_stack_by_reg.groupby(['model', 'timestamp']).sum().reset_index()

# gen_stack_by_isl.loc[:,geo_cols[0]] = gen_stack_by_isl.loc[:,geo_cols[1]] = gen_stack_by_isl.Island
gen_stack_total.loc[:,geo_cols[0]] = gen_stack_total.loc[:,geo_cols[0]] = gen_stack_total.loc[:,geo_cols[1]] = 'UKR'

# gen_stack_by_isl = gen_stack_by_isl.set_index(['model'] + geo_cols + ['timestamp'])
gen_stack_total = gen_stack_total.set_index(['model'] + geo_cols + ['timestamp'])

### Avoid doubles for regions that encompass entire islands
# incl_isl = [i for i in gen_stack_by_isl.index.get_level_values(geo_cols[0]).unique() if i not in gen_stack_by_reg.index.get_level_values(geo_cols[0]).unique()]


gen_stack_by_reg = pd.concat([gen_stack_by_reg, gen_stack_total], axis=0).groupby(['model', geo_cols[0], 'timestamp']).sum() # gen_stack_by_isl.loc[ix[:,incl_isl,:]],

In [ ]:
### Plot 1: Generation stacks for national days of interest


####
doi_periods = [doi for doi in doi_summary.index if 'time' in doi]
doi_names = [doi for doi in doi_summary.index if 'time' not in doi]


for i, p in enumerate(doi_periods):
    
    doi = doi_summary.loc[p]
    doi_name = doi_names[i]
    
    for m in model_names:
        
        save_dir_model = os.path.join(save_dir_plots, m.replace("/",""))
        if os.path.exists(save_dir_model) is False:
            os.mkdir(save_dir_model)
            
        gen_stack = gen_stack_by_reg.loc[ix[m,:,:],:]
        toi = doi.loc[m]
        
        gen_stack_doi = gen_stack.reset_index()
        gen_stack_doi = gen_stack_doi.loc[(gen_stack_doi.timestamp.dt.date >= toi.date() - pd.Timedelta('3D')) &\
                                          (gen_stack_doi.timestamp.dt.date <= toi.date() + pd.Timedelta('3D'))]
        gen_stack_doi = gen_stack_doi.set_index(['model', geo_cols[0], 'timestamp'])
        
        #gen_stack_doi = gen_stack_doi_reg.groupby(['model', 'timestamp'], as_index=False).sum()
        fig_path = os.path.join(save_dir_model,'plot1a_stack_ntl_doi_{}.xlsx'.format(doi_name))
#         shutil.copyfile(fig_template_path, fig_path)


        with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:

            ## ExcelWriter for some reason uses writer.sheets to access the sheet.
            ## If you leave it empty it will not know that sheet Main is already there
            ## and will create a new sheet.


            for reg in model_regs:
                gen_stack_doi_reg = gen_stack_doi.loc[ix[:,reg,:],:].droplevel([0,1])
                write_xlsx_stack(df=gen_stack_doi_reg, writer=writer, sheet_name=reg, palette=stack_palette)



In [ ]:
### Plot 1xx: National generation stacks by model in a single file for easy reference and making report figures


####
doi_periods = [doi for doi in doi_summary.index if 'time' in doi]
doi_names = [doi for doi in doi_summary.index if 'time' not in doi]


for i, p in enumerate(doi_periods):
    
    doi = doi_summary.loc[p]
    doi_name = doi_names[i]
    

    gen_stack = gen_stack_by_reg.loc[ix[:,'UKR',:],:]
    toi = doi.loc[m]

    gen_stack_doi = gen_stack.reset_index()
    gen_stack_doi = gen_stack_doi.loc[(gen_stack_doi.timestamp.dt.date >= toi.date() - pd.Timedelta('3D')) &\
                                      (gen_stack_doi.timestamp.dt.date <= toi.date() + pd.Timedelta('3D'))]
    gen_stack_doi = gen_stack_doi.set_index(['model', geo_cols[0], 'timestamp'])


    #gen_stack_doi = gen_stack_doi_reg.groupby(['model', 'timestamp'], as_index=False).sum()
    fig_path = os.path.join(save_dir_plots,'plot1c_ntl_stack_doi_{}.xlsx'.format(doi_name))
#         shutil.copyfile(fig_template_path, fig_path)


    with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:

        ## ExcelWriter for some reason uses writer.sheets to access the sheet.
        ## If you leave it empty it will not know that sheet Main is already there
        ## and will create a new sheet.


        for m in model_names:
            gen_stack_doi_m = gen_stack_doi.loc[ix[m,],:].droplevel(0)
            write_xlsx_stack(df=gen_stack_doi_m, writer=writer, sheet_name=m.replace(' ','').replace('Flex','')[:31], palette=stack_palette)
            write_xlsx_stack(df=gen_stack_doi_m, writer=writer, sheet_name=m.replace(' ','').replace('Flex','')[:31], palette=stack_palette, to_combine=True)




In [ ]:
### Plot 1b: Generation stacks for national days of interest a specified reference model



### Ref model is based on highest USE....this can be changed/even selected via drop-down menu, etc.
ref_model = use_reg_ts.groupby('model').sum().idxmax().iloc[0]
####|
doi_periods = [doi for doi in doi_summary.index if 'time' in doi]
doi_names = [doi for doi in doi_summary.index if 'time' not in doi]


for i, p in enumerate(doi_periods):
    
    doi = doi_summary.loc[p]
    doi_name = doi_names[i]
    
    for m in model_names:
        
        save_dir_model = os.path.join(save_dir_plots, m.replace("/",""))
        if os.path.exists(save_dir_model) is False:
            os.mkdir(save_dir_model)
            
        gen_stack = gen_stack_by_reg.loc[ix[m,:,:],:]
        toi_ref = doi.loc[ref_model]
        
        gen_stack_doi = gen_stack.reset_index()
        gen_stack_doi = gen_stack_doi.loc[(gen_stack_doi.timestamp.dt.date >= toi_ref.date() - pd.Timedelta('3D')) &\
                                          (gen_stack_doi.timestamp.dt.date <= toi_ref.date() + pd.Timedelta('3D'))]
        gen_stack_doi = gen_stack_doi.set_index(['model', geo_cols[0], 'timestamp'])
        
        #gen_stack_doi = gen_stack_doi_reg.groupby(['model', 'timestamp'], as_index=False).sum()
        fig_path = os.path.join(save_dir_model,'plot1b_stack_ntl_ref_doi_{}.xlsx'.format(doi_name))
#         shutil.copyfile(fig_template_path, fig_path)


        with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:

            ## ExcelWriter for some reason uses writer.sheets to access the sheet.
            ## If you leave it empty it will not know that sheet Main is already there
            ## and will create a new sheet.

            for reg in model_regs:
                gen_stack_doi_reg = gen_stack_doi.loc[ix[:,reg,:],:].droplevel([0,1])
                write_xlsx_stack(df=gen_stack_doi_reg, writer=writer, sheet_name=reg, palette=stack_palette)


In [ ]:
# ### Plot 1d: Generation stacks for regional days of interest of a specified reference

# model_regs = reg_ids + ['JVB', 'SUM', 'UKR']


# ### Ref model is based on highest USE....this can be changed/even selected via drop-down menu, etc.
# ref_model = use_reg_ts.groupby('model').sum().idxmax().iloc[0]


# if reg_ts:
#     for m in model_names:
#         save_dir_model = os.path.join(save_dir_plots, m)
#         if os.path.exists(save_dir_model) is False:
#             os.mkdir(save_dir_model)
            
#         ### DOI/TOI are based on reference model!
#         model_doi = doi_summary_complete[ref_model]
#         doi_periods = [doi for doi in model_doi.index if 'time' in doi]
#         doi_names = [doi for doi in model_doi.index if 'time' not in doi]
        
#         gen_stack = gen_stack_by_reg.loc[ix[m,:,:],:]

#         for i, p in enumerate(doi_periods):

#             doi = model_doi.loc[p].dropna()
#             doi_name = doi_names[i]
#             fig_path = os.path.join(save_dir_model,'plot1d_stack_reg_ref_doi_{}.xlsx'.format(doi_name))
            
#             with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:
      
#                 for reg in model_regs:
#                     toi = doi.loc[reg]


#                     gen_stack_doi_reg = gen_stack.loc[ix[:,reg,:],:].reset_index()  
#                     gen_stack_doi_reg = gen_stack_doi_reg.loc[(gen_stack_doi_reg.timestamp.dt.date >= toi.date() - pd.Timedelta('3D')) &\
#                                               (gen_stack_doi_reg.timestamp.dt.date <= toi.date() + pd.Timedelta('3D'))]
#                     gen_stack_doi_reg = gen_stack_doi_reg.set_index(['model', geo_cols[0], 'timestamp']).droplevel(['model',geo_cols[0]])

#                     write_xlsx_stack(df=gen_stack_doi_reg, writer=writer, sheet_name=reg, palette=stack_palette)



### New outputs for plots only.
To merge with rest of script at some point during clean up

In [ ]:
# ### Plot0: Battery plots

# # Battery calculations #
# # -------------------- #

# sto_cap_reg = gen_yr_df[(gen_yr_df.Category == 'Storage')&(gen_yr_df.property == 'Installed Capacity')]
# # sto_cap_reg.loc[:,'Subcategory'] = sto_cap_reg.name.str[:-4]
# sto_cap_reg = sto_cap_reg.groupby(['model']+geo_cols + ['StoDuration','timestamp']).sum().value/1000
# sto_cap_reg = sto_cap_reg.reset_index()



# sto_cap_avg_duration = sto_cap_reg.copy()
# sto_cap_avg_duration.loc[:,'Average Duration (hrs) - right-axis'] = sto_cap_avg_duration.StoDuration*sto_cap_avg_duration.value
# sto_cap_avg_duration = sto_cap_avg_duration.rename(columns={'value': 'Installed Capacity (GW)'}).drop(columns='StoDuration')
# sto_cap_avg_duration = sto_cap_avg_duration.groupby(['model']+geo_cols).sum()
# sto_cap_avg_duration.loc[:,'Average Duration (hrs) - right-axis'] = sto_cap_avg_duration.loc[:,'Average Duration (hrs) - right-axis']/sto_cap_avg_duration.loc[:,'Installed Capacity (GW)']

# ########
# sto_cap_reg = sto_cap_reg.groupby(['model']+geo_cols + ['StoDuration']).sum().value.unstack('StoDuration').fillna(0)
# sto_cap_reg.columns = ['Installed Capacity (GW) - {:0.1f} hr'.format(c) if c ==2.5 else 'Installed Capacity (GW) - {:0.0f} hr'.format(c)  for c in sto_cap_reg.columns]
# sto_cap_reg = pd.concat([sto_cap_reg, sto_cap_avg_duration['Average Duration (hrs) - right-axis']], axis=1)


# for m in sto_cap_reg.index.get_level_values(0):
    
#     save_dir_model = os.path.join(save_dir_plots, m)
#     if os.path.exists(save_dir_model) is False:
#         os.mkdir(save_dir_model)

#     sto_cap_reg_m = sto_cap_reg.loc[ix[m,]].groupby(geo_cols[0]).sum()
#     fig_path = os.path.join(save_dir_model,'plot0_battery_plots_{}.xlsx'.format(m))
#     with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:

#         write_xlsx_column(df=sto_cap_reg_m, writer=writer, sheet_name='battey_cap_plot', subtype='stacked', units='GW', total_scatter_col='Average Duration (hrs) - right-axis', right_ax='hours')   



#### Annual summary plots in Excel

In [ ]:
model_years = {}

for m in model_names:
    for y in co2_targets.index.astype(str):
        if y in m:
            model_years[m] = np.float64(y)
        
        

In [ ]:
### For plotting of emission tagerets vs. actual emissions
co2_target_by_model = pd.Series({m:np.round(co2_targets[model_years[m]],2) for m in model_years.keys()} , name='Emission target')

In [ ]:
plot_cols = {'load_by_reg': customer_load_by_reg.groupby(['model', geo_cols[0]]).sum().unstack(level=geo_cols[0])/1000,
             
             'pk_load_by_reg': pd.concat([ customer_load_reg_ts.stack(geo_cols).groupby(['model', geo_cols[0]]).max().unstack(level='model')/1000,
                                          customer_load_reg_ts.sum(axis=1).groupby(['model']).max().rename('UKR').to_frame().T/1000], axis=0),
             
             'pk_netload_by_reg': pd.concat([ net_load_reg_ts.stack(geo_cols).groupby(['model', geo_cols[0]]).max().unstack(level='model')/1000, 
                                   net_load_reg_ts.sum(axis=1).groupby(['model']).max().rename('UKR').to_frame().T/1000], axis=0),

             
#              'line_cap_isl': line_cap_isl['Export Limit'].rename('value').unstack(level = 'line')/1000,
#              'line_net_exports_isl': (line_imp_exp_isl['Flow'] - line_imp_exp_isl['Flow Back']).unstack('line')/1000,
#              'line_exports_isl': (line_imp_exp_isl['Flow']).unstack('line')/1000,
#              'line_imports_isl': (line_imp_exp_isl['Flow Back']).unstack('line')/1000,
             
             'use_by_reg': use_by_reg.groupby(['model',geo_cols[0]]).sum().unstack(level=geo_cols[0])/1000,
          
             'gen_by_tech': gen_by_tech_reg.stack(geo_cols).groupby(['model', 'Category']).sum().unstack(level='Category')/1000, 
             'gen_by_reg': gen_by_tech_reg.stack(geo_cols).groupby(['model', geo_cols[0]]).sum().unstack(level=geo_cols[0]), 

             'net_gen_by_reg': gen_by_tech_reg.stack(geo_cols).groupby(['model', geo_cols[0]]).sum().unstack(level=geo_cols[0]).fillna(0)/1000-load_by_reg.groupby(['model', geo_cols[0]]).sum().unstack(level=geo_cols[0])/1000,
             'gen_cap_by_reg': gen_cap_tech_reg.stack(geo_cols).groupby(['model', geo_cols[0]]).sum().unstack(level=geo_cols[0])/1000,
             'gen_cap_by_tech': gen_cap_tech_reg.stack(geo_cols).groupby(['model', 'Category']).sum().unstack(level='Category')/1000,  

             'cf_tech': cf_tech,
             'cf_tech_transposed': cf_tech.T,
             'vre_by_reg_byGen': vre_by_reg,
             'vre_by_reg_byAv': pd.concat([vre_av_reg_abs_ts.groupby('model').sum().groupby(geo_cols[0], axis=1).sum()/1000/gen_by_tech_reg.groupby('model').sum().groupby(geo_cols[0], axis=1).sum(), 
                                           (vre_av_reg_abs_ts.groupby('model').sum().sum(axis=1)/1000/gen_by_tech_reg.groupby('model').sum().sum(axis=1)).rename('UKR')],axis=1),
             're_by_reg': re_by_reg,
             
             'curtailment_rate': curtailment_rate/100,
             're_curtailed_by_tech': re_curtailment_rate_by_tech,
             'fuels_by_type': fuel_by_type.groupby(['model', 'Type']).sum().unstack('Type').replace(0,np.nan).dropna(axis=1,how="all").fillna(0),
             'fuels_by_subtype': fuel_by_type.groupby(['model', 'Category']).sum().unstack('Category').replace(0,np.nan).dropna(axis=1,how="all").fillna(0),
             
             'co2_by_tech': pd.concat([co2_by_tech_reg.groupby(['model', 'Category']).sum().unstack(level='Category')/1e6, co2_target_by_model],axis=1).fillna(0),
#              'co2_by_subfuels': co2_fuels_by_reg.groupby(['model', 'Category']).sum().unstack('Category')/1e6,
             'co2_by_reg': pd.concat([co2_by_tech_reg.groupby(['model', geo_cols[0]]).sum().unstack(level=geo_cols[0])/1e6, co2_target_by_model],axis=1).fillna(0),
#              'co2_by_isl': co2_by_tech_reg.groupby(['model', geo_cols[0]]).sum().unstack(level=geo_cols[0])/1e6,
             'co2_intensity_reg': co2_by_reg.unstack(geo_cols).groupby(geo_cols[0], axis=1).sum()/gen_by_tech_reg.groupby('model').sum().groupby(geo_cols[0],axis=1).sum(),
#              'co2_intensity_isl': co2_by_reg.unstack(geo_cols).groupby(geo_cols[0], axis=1).sum()/gen_by_tech_reg.groupby('model').sum().groupby(geo_cols[0],axis=1).sum(),             
             

             'op_costs_by_tech' : gen_op_costs_by_reg.groupby(['model', 'Category']).sum().unstack(level='Category'),
             'op_costs_by_prop' : gen_op_costs_by_reg.groupby(['model', 'property']).sum().unstack(level='property'),
             'op_and_vio_costs_by_prop': gen_op_and_vio_costs_reg.groupby(['model', 'property']).sum().unstack(level='property'),
             'tsc_by_tech' : gen_total_costs_by_reg.groupby(['model', 'Category']).sum().unstack(level='Category'),
             'tsc_by_prop' : gen_total_costs_by_reg.groupby(['model', 'property']).sum().unstack(level='property'),
             'tsc_by_tech_2030' : gen_total_costs_by_reg_2030.groupby(['model', 'Category']).sum().unstack(level='Category'),
             'tsc_by_prop_2030' : gen_total_costs_by_reg_2030.groupby(['model', 'property']).sum().unstack(level='property'),
             
             'lcoe_by_tech' : lcoe_tech.unstack(level='Category'),
             'lcoe_by_tech_T' : lcoe_tech.unstack(level='model'),
             'ramp_pc_by_reg' :pd.concat([(ramp_reg_ts.groupby(['model', geo_cols[0], 'timestamp']).sum()/daily_pk_reg_ts.stack(geo_cols).groupby(['model', geo_cols[0], 'timestamp']).sum()).groupby(['model', geo_cols[0]]).max().unstack(level=geo_cols[0])*100, 
                                          ramp_pc_ts.groupby(['model']).max().rename('UKR')], axis=1),
             'th_ramp_pc_by_reg' :pd.concat([(th_ramp_reg_ts.groupby(['model', geo_cols[0], 'timestamp']).sum()/daily_pk_reg_ts.stack(geo_cols).groupby(['model', geo_cols[0], 'timestamp']).sum()).groupby(['model', geo_cols[0]]).max().unstack(level=geo_cols[0])*100, 
                                          th_ramp_pc_ts.groupby(['model']).max().rename('UKR')], axis=1),
             'ramp_by_reg' : pd.concat([ramp_reg_ts.unstack(geo_cols).groupby(level=geo_cols[0],axis=1).sum().groupby(['model']).max(), ramp_ts.groupby(['model']).max().value.rename('UKR')], axis=1),
             
             'dsm_pk_contr': (nldc_orig.iloc[:100,:] - nldc.iloc[:100,:]).mean().rename('value').to_frame()
            }
 




plot_type = {'load_by_reg': 'stacked',

                          
             'pk_load_by_reg':  'clustered',

             'pk_netload_by_reg': 'clustered',

             
#              'line_cap_isl': 'clustered',
#              'line_net_exports_isl': 'clustered',
#              'line_exports_isl': 'clustered',
#              'line_imports_isl': 'clustered',
             
             'use_by_reg': 'stacked',

             'gen_by_tech': 'stacked', 
             'gen_by_WEOtech': 'stacked', 
             'gen_by_reg': 'stacked', 
             'net_gen_by_reg': 'clustered', 
             'vre_by_reg_byGen': 'clustered',
             'vre_by_reg_byAv': 'clustered',
             're_by_reg': 'clustered',
             
             'fuels_by_type': 'stacked',
             'fuels_by_subtype': 'stacked',
             'co2_by_tech': 'stacked',
             'co2_by_fuels': 'stacked',
             'co2_by_subfuels': 'stacked',
             
             'co2_by_tech':'stacked',
             'co2_by_reg': 'stacked',

             'co2_intensity_reg': 'clustered',
    
             'curtailment_rate': 'clustered',
             're_curtailed_by_tech':'clustered',
             'gen_cap_by_reg': 'stacked',

             'gen_cap_by_tech': 'stacked',
             'gen_cap_by_WEOtech': 'stacked',
             'cf_tech': 'clustered',
             'cf_tech_transposed': 'clustered',
             'op_costs_by_tech' : 'stacked',
             'op_costs_by_prop' :'stacked',
             'op_and_vio_costs_by_prop' :'stacked',
             'tsc_by_tech' : 'stacked',
             'tsc_by_prop' : 'stacked',
             'tsc_by_tech_2030' : 'stacked',
             'tsc_by_prop_2030' : 'stacked',
             
             'lcoe_by_tech' : 'clustered',
             'lcoe_by_tech_T' : 'clustered',
             'ramp_pc_by_reg' :'clustered',
             'th_ramp_pc_by_reg' :'clustered',
             'ramp_by_reg' :'clustered',
             'th_ramp_by_reg' :'clustered',
             'dsm_pk_contr':'clustered'
            }
               


plot_units = {'load_by_reg': 'TWh',

             'use_by_reg': 'TWh',

             'gen_by_tech': 'TWh', 
             'gen_by_WEOtech': 'TWh', 
             'gen_by_reg': 'TWh', 

             'net_gen_by_reg':'TWh',
             'vre_by_reg_byGen': '%',
             'vre_by_reg_byAv': '%',
             're_by_reg': '%',       
              
             'pk_load_by_reg':  'GW',

             'pk_netload_by_reg': 'GW',

              
             'fuels_by_type': 'TJ',
             'fuels_by_subtype': 'TJ',
             'co2_by_tech': 'million tonnes',
             'co2_by_fuels': 'million tonnes',
             'co2_by_subfuels': 'million tonnes',
              
             'co2_by_tech':'million tonnes',
             'co2_by_reg': 'million tonnes',

             'co2_intensity_reg': 'kg/MWh',

             'curtailment_rate': '%',
              're_curtailed_by_tech':'%',
             'gen_cap_by_reg': 'GW',

             'gen_cap_by_tech': 'GW',
             'gen_cap_by_WEOtech': 'GW',
             'cf_tech': '%',
             'cf_tech_transposed': '%',
             'op_costs_by_tech' : 'USDm',
             'op_costs_by_prop' :'USDm',
            'op_and_vio_costs_by_prop' :'USDm',
             'tsc_by_tech' : 'USDm',
             'tsc_by_prop' : 'USDm',
             'tsc_by_tech_2030' : 'USDm',
             'tsc_by_prop_2030' : 'USDm',
             'lcoe_by_tech' : 'USD/MWh',
             'lcoe_by_tech_T' : 'USD/MWh',
             'ramp_pc_by_reg' :'%/hr',
             'th_ramp_pc_by_reg' :'%/hr',
             'ramp_by_reg' :'MW/hr',
             'th_ramp_by_reg' :'MW/hr',
             'dsm_pk_contr': 'GW'}



In [ ]:
# ### Gen by tech/reg plots per model
# for ref_m in model_names:
#     save_dir_model = os.path.join(save_dir_plots,ref_m)
#     if os.path.exists(save_dir_model) is False:
#         os.mkdir(save_dir_model)
        
#     fig_path = os.path.join(save_dir_model,'plot3_gen_by_tech_reg_{}.xlsx'.format(ref_m))
#     with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:
    
#         gen_tech_reg_m = gen_by_tech_reg.loc[ix[ref_m,:]].groupby(geo_cols[0], axis=1).sum().T/1000
#         gen_cap_tech_reg_m = gen_cap_tech_reg.loc[ix[ref_m,:]].groupby(geo_cols[0], axis=1).sum().T/1000
        
        

#         write_xlsx_column(df=gen_tech_reg_m, writer=writer, sheet_name='gen_tech_reg', subtype='stacked', units='TWh', palette=combined_palette)   
#         write_xlsx_column(df=gen_cap_tech_reg_m, writer=writer, sheet_name='gen_cap_tech_reg', subtype='stacked', units='GW', palette=combined_palette)   

In [ ]:
### Plot 2: Annual summary plots by columnn

fig_path = os.path.join(save_dir_plots,'plot2_annual_summary_plots.xlsx')

with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:
    
    for i in plot_cols.keys():
        total_col = None 
        
        if plot_cols[i].shape[0] == 0:
            print('Empty dataframe for: {}'.format(i))
        else:

            if 'UKR' in plot_cols[i].columns:
                total_col = 'UKR'
            elif 'co2' in i:
                total_col = plot_cols[i].columns[-1]
                
            write_xlsx_column(df=plot_cols[i], writer=writer, sheet_name=i, subtype=plot_type[i], units=plot_units[i], palette=combined_palette, total_scatter_col=total_col)
    
    

In [ ]:
### Plot 6: Cost savings plots by reference model

### Get rid of cofiring if any for the purpose of comparison
gen_op_costs_by_tech = gen_op_costs_by_reg.unstack('Category').rename(columns={'Cofiring':'Coal'}).stack().groupby(['model','Category']).sum().unstack('model').fillna(0)
gen_total_costs_by_tech = gen_total_costs_by_reg.unstack('Category').rename(columns={'Cofiring':'Coal'}).stack().groupby(['model','Category']).sum().unstack('model').fillna(0)

gen_op_costs_by_prop = gen_op_costs_by_reg.groupby(['model','property']).sum().unstack('model').fillna(0)
gen_op_costs_wo_co2_by_prop = gen_op_costs_wo_co2_by_reg.groupby(['model','property']).sum().unstack('model').fillna(0)


gen_total_costs_by_prop = gen_total_costs_by_reg.groupby(['model','property']).sum().unstack('model').fillna(0)
gen_op_vio_costs_by_prop = gen_op_and_vio_costs_reg.groupby(['model','property']).sum().unstack('model').fillna(0)
gen_op_vio_costs_wo_co2_by_prop = gen_op_and_vio_costs_reg_wo_co2.groupby(['model','property']).sum().unstack('model').fillna(0)


for ref_m in model_names:
    save_dir_model = os.path.join(save_dir_plots,ref_m.replace("/",""))
    if os.path.exists(save_dir_model) is False:
        os.mkdir(save_dir_model)
        
    fig_path = os.path.join(save_dir_model,'plot6_cost_savings_ref_{}.xlsx'.format(ref_m.replace("/","")))
    with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:
    
        ref_op_prop = gen_op_costs_by_prop[ref_m]
        ref_op_prop_wo_co2 = gen_op_costs_wo_co2_by_prop[ref_m]
        ref_op_vio_prop = gen_op_vio_costs_by_prop[ref_m]
        ref_op_vio_prop_wo_co2 = gen_op_vio_costs_wo_co2_by_prop[ref_m]

        ref_tsc_prop = gen_total_costs_by_prop[ref_m]
        ref_op_tech = gen_op_costs_by_tech[ref_m]
        ref_tsc_tech = gen_total_costs_by_tech[ref_m]

        savings_op_by_prop =  (-gen_op_costs_by_prop).drop(columns=ref_m).subtract(-ref_op_prop, axis=0).T
        savings_op_vio_by_prop =  (-gen_op_vio_costs_by_prop).drop(columns=ref_m).subtract(-ref_op_vio_prop, axis=0).T
        savings_op_wo_co2_by_prop =  (-gen_op_costs_wo_co2_by_prop).drop(columns=ref_m).subtract(-ref_op_prop_wo_co2, axis=0).T
        savings_op_vio_wo_co2_by_prop =  (-gen_op_vio_costs_wo_co2_by_prop).drop(columns=ref_m).subtract(-ref_op_vio_prop_wo_co2, axis=0).T
        savings_tsc_by_prop = (-gen_total_costs_by_prop).drop(columns=ref_m).subtract(-ref_tsc_prop, axis=0).T
        savings_op_by_tech = (-gen_op_costs_by_tech).drop(columns=ref_m).subtract(-ref_op_tech, axis=0).T
        savings_tsc_by_tech = (-gen_total_costs_by_tech).drop(columns=ref_m).subtract(-ref_tsc_tech, axis=0).T
        
        savings_op_by_prop_pc  =  savings_op_by_prop/ref_op_prop.sum()
        savings_op_vio_by_prop_pc  =  savings_op_vio_by_prop/ref_op_vio_prop.sum()
        savings_op_wo_co2_by_prop_pc  =  savings_op_wo_co2_by_prop/ref_op_prop_wo_co2.sum()
        savings_op_vio_wo_co2_by_prop_pc  =  savings_op_vio_wo_co2_by_prop/ref_op_vio_prop_wo_co2.sum()
        savings_tsc_by_prop_pc = savings_tsc_by_prop/ref_tsc_prop.sum()
        savings_op_by_tech_pc = savings_op_by_tech/ref_op_tech.sum()
        savings_tsc_by_tech_pc = savings_tsc_by_tech/ref_tsc_tech.sum()
        
        write_xlsx_column(df=savings_op_by_prop, writer=writer, sheet_name='savings_op_by_prop', subtype='stacked', units='USDm', total_scatter_col='Total savings')   
        write_xlsx_column(df=savings_op_vio_by_prop, writer=writer, sheet_name='savings_op_vio_by_prop', subtype='stacked', units='USDm', total_scatter_col='Total savings')   
        write_xlsx_column(df=savings_op_wo_co2_by_prop, writer=writer, sheet_name='savings_op_wo_co2_by_prop', subtype='stacked', units='USDm', total_scatter_col='Total savings')   
        write_xlsx_column(df=savings_op_vio_wo_co2_by_prop, writer=writer, sheet_name='savings_op_vio_wo_co2_by_prop', subtype='stacked', units='USDm', total_scatter_col='Total savings')   
        write_xlsx_column(df=savings_tsc_by_prop, writer=writer, sheet_name='savings_tsc_by_prop', subtype='stacked', units='USDm', total_scatter_col='Total savings')     
        write_xlsx_column(df=savings_op_by_tech, writer=writer, sheet_name='savings_op_by_tech', subtype='stacked', units='USDm', total_scatter_col='Total savings')   
        write_xlsx_column(df=savings_tsc_by_tech, writer=writer, sheet_name='savings_tsc_by_tech', subtype='stacked', units='USDm', total_scatter_col='Total savings') 
        
        write_xlsx_column(df=savings_op_by_prop_pc, writer=writer, sheet_name='savings_op_by_prop_pc', subtype='stacked', units='', total_scatter_col='Relative savings')   
        write_xlsx_column(df=savings_op_vio_by_prop_pc, writer=writer, sheet_name='savings_op_vio_by_prop_pc', subtype='stacked', units='', total_scatter_col='Relative savings') 
        write_xlsx_column(df=savings_op_wo_co2_by_prop_pc, writer=writer, sheet_name='savings_op_wo_co2_by_prop_pc', subtype='stacked', units='', total_scatter_col='Relative savings')   
        write_xlsx_column(df=savings_op_vio_wo_co2_by_prop_pc, writer=writer, sheet_name='savings_opvio_noco2_by_prop_pc', subtype='stacked', units='', total_scatter_col='Relative savings')   
        write_xlsx_column(df=savings_tsc_by_prop_pc, writer=writer, sheet_name='savings_tsc_by_prop_pc', subtype='stacked', units='', total_scatter_col='Relative savings')     
        write_xlsx_column(df=savings_op_by_tech_pc, writer=writer, sheet_name='savings_op_by_tech_pc', subtype='stacked', units='', total_scatter_col='Relative savings')   
        write_xlsx_column(df=savings_tsc_by_tech_pc, writer=writer, sheet_name='savings_tsc_by_tech_pc', subtype='stacked', units='', total_scatter_col='Relative savings')
        

In [ ]:
### Plot7: CO2 savings plots by reference model

### Get rid of cofiring if any for the purpose of comparison
co2_by_tech = co2_by_tech_reg.unstack('Category').rename(columns={'Cofiring':'Coal'}).stack().groupby(['model','Category']).sum().unstack('model').fillna(0)/1e6
co2_by_reg_plt = co2_by_tech_reg.groupby(['model',geo_cols[0]]).sum().unstack('model').fillna(0)/1e6

em_by_type = em_by_type_tech_reg.groupby(['model','parent']).sum().value.unstack('model')/1e6


for ref_m in model_names:
    save_dir_model = os.path.join(save_dir_plots,ref_m.replace("/",""))
    if os.path.exists(save_dir_model) is False:
        os.mkdir(save_dir_model)
        
    fig_path = os.path.join(save_dir_model,'plot7_co2_savings_ref_{}.xlsx'.format(ref_m.replace("/","")))
    with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:
    
        ref_co2_tech = co2_by_tech[ref_m]
        ref_co2_reg = co2_by_reg_plt[ref_m]
        ref_em_type = em_by_type[ref_m]

        co2_savings_by_tech =  (-co2_by_tech).drop(columns=ref_m).subtract(-ref_co2_tech, axis=0).T
        co2_savings_by_reg = (-co2_by_reg_plt).drop(columns=ref_m).subtract(-ref_co2_reg, axis=0).T

        em_savings_by_reg = (-em_by_type).drop(columns=ref_m).subtract(-ref_em_type, axis=0).T
        
        co2_savings_by_tech_pc =  (-co2_by_tech).drop(columns=ref_m).subtract(-ref_co2_tech, axis=0).T/ref_co2_tech.sum()
        co2_savings_by_reg_pc = (-co2_by_reg_plt).drop(columns=ref_m).subtract(-ref_co2_reg, axis=0).T/ref_co2_reg.sum()
        em_savings_by_reg_pc = (-em_by_type).drop(columns=ref_m).subtract(-ref_em_type, axis=0).T/ref_em_type
        
        write_xlsx_column(df=co2_savings_by_tech, writer=writer, sheet_name='co2_savings_by_tech_abs', subtype='stacked', units='million tonnes', total_scatter_col='Total reduction')   
        write_xlsx_column(df=co2_savings_by_reg, writer=writer, sheet_name='co2_savings_by_reg_abs', subtype='stacked', units='million tonnes', total_scatter_col='Total reduction')    

        write_xlsx_column(df=em_savings_by_reg, writer=writer, sheet_name='em_savings_by_type_abs', subtype='stacked', units='million tonnes', total_scatter_col='Total reduction') 
        
        write_xlsx_column(df=co2_savings_by_tech_pc, writer=writer, sheet_name='co2_savings_by_tech_pc', subtype='stacked', units='', total_scatter_col='Relative reduction')   
        write_xlsx_column(df=co2_savings_by_reg_pc, writer=writer, sheet_name='co2_savings_by_reg_pc', subtype='stacked', units='', total_scatter_col='Relative reduction')    

        write_xlsx_column(df=em_savings_by_reg_pc, writer=writer, sheet_name='em_savings_by_type_pc', subtype='clustered', units='')  
        

#### Time-series summary plots

In [ ]:
### Plot 3: Daily gap plot

nl_dly_gap_complete = pd.concat([nl_dly_gap_reg, nl_dly_gap.rename('UKR')], axis=1)
nl_dly_gap_pc_complete = pd.concat([nl_dly_gap_reg_pc, nl_dly_gap_pc.rename('UKR')], axis=1)


fig_path = os.path.join(save_dir_plots,'plot3_daily_gap.xlsx')

with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:
    
    for reg in nl_dly_gap_complete.columns:
        nl_dly_gap_plot = nl_dly_gap_complete[reg].unstack('model')
        
        write_xlsx_scatter(df=nl_dly_gap_plot, writer=writer, sheet_name=reg, palette=combined_palette, units='MW', markersize=4, alpha=90) 

fig_path = os.path.join(save_dir_plots,'plot3_daily_gap_pc.xlsx')

with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:
    
    for reg in nl_dly_gap_complete.columns:
        nl_dly_gap_pc_plot = nl_dly_gap_pc_complete[reg].unstack('model')
        
       
        write_xlsx_scatter(df=nl_dly_gap_pc_plot, writer=writer, sheet_name=reg, palette=combined_palette, units='', markersize=4, alpha=90) 
    

In [ ]:
# ### Plot 3: Annual summary line plots

plot_lines = {'ldc':ldc/1000,
             'nldc':nldc/1000,
             'nldc_curtail':nldc_curtail/1000,
             'nldc_sto':nldc_sto/1000,
             'nldc_sto_curtail':nldc_sto_curtail/1000,
             'curtailment_dc':curtailment_dc/1000
            }      

ln_plot_type = {'ldc':'ldc',
             'nldc':'ldc',
             'nldc_curtail':'ldc',
             'nldc_sto':'ldc',
             'nldc_sto_curtail':'ldc',
             'curtailment_dc':'ldc',

            }      
 

ln_plot_units = {'ldc':'GW',
             'nldc':'GW',
             'nldc_curtail':'GW',
             'nldc_sto':'GW',
             'nldc_sto_curtail':'GW',
             'curtailment_dc':'GW',
             
            }      


In [ ]:
### Plot 3: Annual summary plots by columnn

fig_path = os.path.join(save_dir_plots,'plot3_ldc_plots.xlsx')

with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:
    
    for i in plot_lines.keys():
        write_xlsx_line(df=plot_lines[i], writer=writer, sheet_name=i,subtype=ln_plot_type[i], units=ln_plot_units[i], line_width=1.5)   

In [ ]:
### Plot 3: Annual summary plots by columnn

fig_path = os.path.join(save_dir_plots,'plot3_ldc_plots_by_model.xlsx')

with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:
    
    for i in plot_lines.keys():
        write_xlsx_line(df=plot_lines[i], writer=writer, sheet_name=i,subtype=ln_plot_type[i], units=ln_plot_units[i], line_width=1.5)   

In [ ]:
### Output 5: Inertia plots

inertia_geo_col = [geo_cols[0]]

for col in inertia_geo_col:
    inertia_col = inertia_by_reg.groupby([col,'model', 'timestamp']).sum()
    for reg in inertia_col.index.get_level_values(col).unique():
        
        inertia_reg = inertia_col.loc[ix[reg,]]
        
        fig_path = os.path.join(save_dir_plots,'plot5_inertia_plot_{}.xlsx'.format(reg))
        with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:

            for m in model_names:
                inertia_plot = inertia_reg.loc[ix[m,]].rename(columns={'InertiaLo':'Inertia (lower range)', 'InertiaHi':'Inertia (higher range)'})
                write_xlsx_line(df=inertia_plot, writer=writer, sheet_name=m[:31].replace("/",""), subtype='timeseries', units='MWs') 
                write_xlsx_line(df=inertia_plot, writer=writer, sheet_name=m[:31].replace("/",""), subtype='timeseries', units='MWs', to_combine=True) 
                
                
fig_path = os.path.join(save_dir_plots,'plot5_inertia_plot_total.xlsx')
with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:

    for m in model_names:
        inertia_plot = total_inertia_ts.loc[ix[m,]].rename(columns={'InertiaLo':'Inertia (lower range)', 'InertiaHi':'Inertia (higher range)'})
        write_xlsx_line(df=inertia_plot, writer=writer, sheet_name=m[:31].replace("/",""), subtype='timeseries', units='MWs', line_width=0.1)
        write_xlsx_line(df=inertia_plot, writer=writer, sheet_name=m[:31].replace("/",""), subtype='timeseries', units='MWs', to_combine=True, line_width=0.1) 

        



In [ ]:

### Output 5: Inertia plots

inertia_geo_col = [geo_cols[0]]

for col in inertia_geo_col:
    
    inertia_col = inertia_by_reg.groupby([pd.Grouper(level=col), pd.Grouper(level='model'),pd.Grouper(level='timestamp', freq='D')]).min()
    
    for reg in inertia_col.index.get_level_values(col).unique():
        
        inertia_reg = inertia_col.loc[ix[reg,]]
        
        fig_path = os.path.join(save_dir_plots,'plot5_inertia_plot_daily_min_{}.xlsx'.format(reg))
        with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:

            for m in model_names:
                inertia_plot = inertia_reg.loc[ix[m,]].rename(columns={'InertiaLo':'Inertia (lower range)', 'InertiaHi':'Inertia (higher range)'})
                write_xlsx_line(df=inertia_plot, writer=writer, sheet_name=m[:31].replace("/",""), subtype='timeseries', units='MWs') 
                write_xlsx_line(df=inertia_plot, writer=writer, sheet_name=m[:31].replace("/",""), subtype='timeseries', units='MWs', to_combine=True) 
                
                
fig_path = os.path.join(save_dir_plots,'plot5_inertia_plot_total_daily_min.xlsx')
with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:

    for m in model_names:
        inertia_plot = total_inertia_ts.loc[ix[m,]].rename(columns={'InertiaLo':'Inertia (lower range)', 'InertiaHi':'Inertia (higher range)'})
        inertia_plot = inertia_plot.groupby(pd.Grouper(level='timestamp', freq='D')).min()
        write_xlsx_line(df=inertia_plot, writer=writer, sheet_name=m[:31].replace("/",""), subtype='timeseries', units='MWs', line_width=0.1)
        write_xlsx_line(df=inertia_plot, writer=writer, sheet_name=m[:31].replace("/",""), subtype='timeseries', units='MWs', to_combine=True, line_width=0.1) 

        



In [ ]:
### Output 4a: GDCs

fig_path = os.path.join(save_dir_plots,'plot4a_gdc_plots_national.xlsx')

with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:

    for tech in gen_by_tech_ts.columns:


        gdc = gen_by_tech_ts[tech].unstack('model')/1000
        gdc = pd.DataFrame(np.flipud(np.sort(gdc.values, axis=0)), index=gdc.index, columns=gdc.columns)
        write_xlsx_line(df=gdc, writer=writer, sheet_name=tech, subtype='ldc', units='GW') 

### Output 4b: GDC regs

gdc_reg_dict = {}
gdc_geo_col = [geo_cols[0]]

for col in gdc_geo_col:
    gen_by_tech_col = gen_by_tech_reg_ts.groupby(['model',col,'timestamp']).sum()
    
    for reg in gen_by_tech_col.index.get_level_values(col).unique():
        
        fig_path = os.path.join(save_dir_plots,'plot4a_gdc_plots_{}.xlsx'.format(reg))
        with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:
        
            for tech in gen_by_tech_col.columns:

                gdc_reg = gen_by_tech_col[tech].unstack('model').loc[reg]
                gdc_reg = pd.DataFrame(np.flipud(np.sort(gdc_reg.values, axis=0)), index=gdc_reg.index, columns=gdc_reg.columns)
                write_xlsx_line(df=gdc_reg, writer=writer, sheet_name=tech, subtype='ldc', units='GW')  



In [ ]:
### Output 4c: Normalised GDCs

fig_path = os.path.join(save_dir_plots,'plot4c_gdc_normalised_plots_national.xlsx')

with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:

    for tech in gen_by_tech_ts.columns:
        ### In case of validation being present, we need to include only the models as per the model_names
        cap = gen_cap_tech_reg.sum(axis=1).unstack('model').loc[tech,][model_names]
        gdc_norm = np.divide(gen_by_tech_ts[tech].unstack('model'), cap)
        gdc_norm = pd.DataFrame(np.flipud(np.sort(gdc_norm.values, axis=0)), index=gdc_norm.index, columns=gdc_norm.columns)
        write_xlsx_line(df=gdc_norm, writer=writer, sheet_name=tech, subtype='ldc', units='-')  
        
### Output 4d: GDC normalised regs

gdc_reg_dict = {}
gdc_geo_col = [geo_cols[0]]

for col in gdc_geo_col:
    gen_by_tech_col = gen_by_tech_reg_ts.groupby(['model',col,'timestamp']).sum()
    gen_cap_tech_col = gen_cap_tech_reg.groupby(col, axis=1).sum()

    
    for reg in gen_by_tech_col.index.get_level_values(col).unique():
        
        fig_path = os.path.join(save_dir_plots,'plot4d_gdc_normalised_plots_{}.xlsx'.format(reg))
        with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:
        
            for tech in gen_by_tech_col.columns:
                        ### In case of validation being present, we need to include only the models as per the model_names
                cap = gen_cap_tech_col[reg].unstack('model').loc[tech][model_names]
                gdc_reg_norm = np.divide(gen_by_tech_col[tech].unstack('model').loc[reg],cap)
                gdc_reg_norm = pd.DataFrame(np.flipud(np.sort(gdc_reg_norm.values, axis=0)), index=gdc_reg_norm.index, columns=gdc_reg_norm.columns)
                write_xlsx_line(df=gdc_reg_norm, writer=writer, sheet_name=tech, subtype='ldc', units='GW')  



In [ ]:
### Output 5a: GDC zoomed

fig_path = os.path.join(save_dir_plots,'plot5a_gdc_1pc_national.xlsx')

with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:

    for tech in gen_by_tech_ts.columns:


        gdc = gen_by_tech_ts[tech].unstack('model')/1000
        gdc = pd.DataFrame(np.flipud(np.sort(gdc.values, axis=0)), index=gdc.index, columns=gdc.columns)
        gdc = gdc.iloc[:int(np.ceil(gdc.shape[0]*0.01))]
        gdc_idx = (np.arange(0,gdc.shape[0]) + 1)/gdc.shape[0]*0.01
        write_xlsx_line(df=gdc, writer=writer, sheet_name=tech, subtype='ldc', units='GW', ldc_idx=gdc_idx)

### Output 5b: GDC regs zoomed

gdc_reg_dict = {}
gdc_geo_col = [geo_cols[0]]

for col in gdc_geo_col:
    gen_by_tech_col = gen_by_tech_reg_ts.groupby(['model',col,'timestamp']).sum()
    
    for reg in gen_by_tech_col.index.get_level_values(col).unique():
        
        fig_path = os.path.join(save_dir_plots,'plot5a_gdc_1pc_{}.xlsx'.format(reg))
        with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:
        
            for tech in gen_by_tech_col.columns:
                gdc_reg = gen_by_tech_col[tech].unstack('model').loc[reg]
                gdc_reg = pd.DataFrame(np.flipud(np.sort(gdc_reg.values, axis=0)), index=gdc_reg.index, columns=gdc_reg.columns)
                gdc_reg = gdc_reg.iloc[:int(np.ceil(gdc_reg.shape[0]*0.01))]
                gdc_idx = (np.arange(0,gdc_reg.shape[0]) + 1)/gdc_reg.shape[0]*0.01
                write_xlsx_line(df=gdc_reg, writer=writer, sheet_name=tech, subtype='ldc', units='GW', ldc_idx=gdc_idx)  



In [ ]:
### Output 4c: Normalised GDCs zoomed to top 1%

fig_path = os.path.join(save_dir_plots,'plot5c_gdc_normalised_1pc_national.xlsx')

with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:

    for tech in gen_by_tech_ts.columns:
        ### In case of validation being present, we need to include only the models as per the model_names
        cap = gen_cap_tech_reg.sum(axis=1).unstack('model').loc[tech,][model_names]
        gdc_norm = np.divide(gen_by_tech_ts[tech].unstack('model'), cap)
        gdc_norm = pd.DataFrame(np.flipud(np.sort(gdc_norm.values, axis=0)), index=gdc_norm.index, columns=gdc_norm.columns)
        gdc_norm = gdc_norm.iloc[:int(np.ceil(gdc_norm.shape[0]*0.01))]
        gdc_idx = (np.arange(0,gdc_norm.shape[0]) + 1)/gdc_norm.shape[0]*0.01
        write_xlsx_line(df=gdc_norm, writer=writer, sheet_name=tech, subtype='ldc', units='-', ldc_idx=gdc_idx)  
        
### Output 4d: GDC normalised regs zoomed to top 1%

gdc_reg_dict = {}
gdc_geo_col = [geo_cols[0]]

for col in gdc_geo_col:
    gen_by_tech_col = gen_by_tech_reg_ts.groupby(['model',col,'timestamp']).sum()
    gen_cap_tech_col = gen_cap_tech_reg.groupby(col, axis=1).sum()

    
    for reg in gen_by_tech_col.index.get_level_values(col).unique():
        
        fig_path = os.path.join(save_dir_plots,'plot4d_gdc_normalised_1pc_{}.xlsx'.format(reg))
        with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:
        
            for tech in gen_by_tech_col.columns:
                ### In case of validation being present, we need to include only the models as per the model_names
                cap = gen_cap_tech_col[reg].unstack('model').loc[tech][model_names]
                gdc_reg_norm = np.divide(gen_by_tech_col[tech].unstack('model').loc[reg],cap)
                gdc_reg_norm = pd.DataFrame(np.flipud(np.sort(gdc_reg_norm.values, axis=0)), index=gdc_reg_norm.index, columns=gdc_reg_norm.columns)
                gdc_reg_norm = gdc_reg_norm.iloc[:int(np.ceil(gdc_reg_norm.shape[0]*0.01))]
                gdc_idx = (np.arange(0,gdc_reg_norm.shape[0]) + 1)/gdc_reg_norm.shape[0]*0.01
                write_xlsx_line(df=gdc_reg_norm, writer=writer, sheet_name=tech, subtype='ldc', units='GW', ldc_idx=gdc_idx)  



In [ ]:
### Output 4c: CF tech/reg

### Future-proofing
cf_geo_col = [geo_cols[0]]
cf_geo_col = [c for c in cf_geo_col if c in geo_cols]
if len(cf_geo_col) == 0:
    cf_geo_col = geo_cols[:1]

for col in cf_geo_col:
    cf_col = cf_tech_reg.stack(geo_cols).groupby([col,'model', 'Category']).sum()
    fig_path = os.path.join(save_dir_plots,'plot6_cf_plots_reg.xlsx')
    with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:
    
        for reg in cf_col.index.get_level_values(col).unique():

            cf_reg = cf_col.loc[ix[reg,]].unstack('Category')
            write_xlsx_column(df=cf_reg, writer=writer, sheet_name=reg, subtype='clustered', units='%')
            write_xlsx_column(df=cf_reg.T, writer=writer, sheet_name='{}_T'.format(reg), subtype='clustered', units='%')  

        ### Write total
        write_xlsx_column(df=cf_tech, writer=writer, sheet_name='UKR', subtype='clustered', units='%')  
        write_xlsx_column(df=cf_tech.T, writer=writer, sheet_name='UKR_T', subtype='clustered', units='%')  

In [ ]:
### Output 6: Seasonality by tech

fig_path = os.path.join(save_dir_plots,'plot6a_seasonality_plots_by_tech_national.xlsx')

with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:

    for tech in gen_stack_total.drop(columns=['Load2']).columns:
        col_monthly = (gen_stack_total[tech].reset_index().groupby(['model'] + [pd.Grouper(key='timestamp', freq='M')]).sum()/1e6).unstack('model').droplevel(0, axis=1)
        write_xlsx_line(df=col_monthly, writer=writer, sheet_name=tech, units='TWh', palette=combined_palette)

fig_path = os.path.join(save_dir_plots,'plot6b_seasonality_plots_by_model_national.xlsx')

with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:

    for m in model_names:
        col_monthly = gen_stack_total[gen_yr_df.Category.unique()].loc[ix[m,],].reset_index().groupby([pd.Grouper(key='timestamp', freq='M')]).sum()/1e6
        write_xlsx_stack(df=col_monthly, writer=writer, sheet_name=m[:31].replace("/",""), units='TWh', palette=stack_palette)
        
# def write_xlsx_stack(df, writer, excel_file=None, sheet_name='Sheet1', palette=stack_palette, units='MW', to_combine=False):


## TEST Stuff

In [ ]:
all_periods = pd.date_range('1/1/2019', periods=8760, freq='60min')
calplot.calplot(net_load_ts.loc[ix[model_names[0],:]].reset_index().groupby(pd.Grouper(key='timestamp', freq='D')).max(numeric_only=True).value)

In [ ]:
calplot.calplot(ramp_ts[ramp_ts.index.get_level_values('timestamp').hour != 0].reset_index().groupby(pd.Grouper(key='timestamp', freq='D')).max().value)

#### Summary stuff

In [ ]:
input_demand = pd.read_excel("R:/RISE/DOCS/04 PROJECTS/COUNTRIES/INDONESIA/Power system enhancement 2020_21/Modelling/01 InputData/03 Demand/from_WEO/\
WEO_demand_input_byRegion_NZE_2040_20220415.xlsx", sheet_name="Total", header =2)


### drop leap days
input_demand = input_demand[~((input_demand.Datetime.dt.month == 2)&(input_demand.Datetime.dt.day == 29))]


In [ ]:
loss_factor = 1.066

if 'NZE' in soln_choice.value:
    input_load_reg = input_demand.set_index('Datetime').sum().rename('2040 NZE - Input').to_frame().T/1000*loss_factor    
    input_pk_load_reg = input_demand.set_index('Datetime').max().rename('2040 NZE - Input').to_frame().T*loss_factor
else:
    input_load_reg = pd.DataFrame(None)      
    input_pk_load_reg = pd.DataFrame(None)             

In [ ]:
summary_cols = {
             'load_by_reg': pd.concat([customer_load_by_reg.groupby(['model', geo_cols[0]]).sum().unstack(level=geo_cols[0])/1000,
                               input_load_reg],axis=0),
             'pk_load_by_reg':pd.concat([pd.concat([customer_load_reg_ts.stack(geo_cols).groupby(['model', geo_cols[0]]).max().unstack(level=geo_cols[0]), customer_load_reg_ts.sum(axis=1).groupby(['model']).max().rename('UKR')],axis=1)/1000,
                              input_pk_load_reg],axis=0),
             'pk_netload_by_reg': pd.concat([net_load_reg_ts.stack(geo_cols).groupby(['model', geo_cols[0]]).max().unstack(level=geo_cols[0]), net_load_reg_ts.stack(geo_cols).groupby(['model']).max().rename('UKR')],axis=1)/1000,
    

             'use_by_reg': use_by_reg.groupby(['model',geo_cols[0]]).sum().unstack(level=geo_cols[0])/1000,
             'gen_by_reg': gen_by_tech_reg.stack(geo_cols).groupby(['model', geo_cols[0]]).sum().unstack(level=geo_cols[0]).fillna(0)/1000, 
             'net_gen_by_reg': gen_by_tech_reg.stack(geo_cols).groupby(['model', geo_cols[0]]).sum().unstack(level=geo_cols[0]).fillna(0)/1000-load_by_reg.groupby(['model', geo_cols[0]]).sum().unstack(level=geo_cols[0])/1000,
             'gen_by_tech': gen_by_tech_reg.stack(geo_cols).groupby(['model', 'Category']).sum().unstack(level='Category').fillna(0)/1000, 
#              'gen_by_WEOtech': gen_by_weoTech_reg.groupby(['model', 'WEO_Tech_simpl']).sum().sum(axis=1).unstack(level='WEO_Tech_simpl').fillna(0)/1000, 
             'gen_cap_by_reg':  gen_cap_tech_reg.stack(geo_cols).groupby(['model', geo_cols[0]]).sum().unstack(level=geo_cols[0])/1000,
             'gen_cap_by_tech': gen_cap_tech_reg.stack(geo_cols).groupby(['model', 'Category']).sum().unstack(level='Category').fillna(0)/1000,  
#              'gen_cap_by_WEOtech': gen_cap_by_weoTech_reg.groupby(['model', 'WEO_Tech_simpl']).sum().sum(axis=1).unstack(level='WEO_Tech_simpl').fillna(0)/1000,  
             'vre_by_reg_byGen': vre_by_reg,
             'vre_by_reg_byAv': pd.concat([vre_av_reg_abs_ts.groupby('model').sum().groupby(geo_cols[0], axis=1).sum()/1000/gen_by_tech_reg.groupby('model').sum().groupby(geo_cols[0], axis=1).sum(), 
                                           (vre_av_reg_abs_ts.groupby('model').sum().sum(axis=1)/1000/gen_by_tech_reg.groupby('model').sum().sum(axis=1)).rename('UKR')],axis=1),
             're_by_reg': re_by_reg,
             'curtailment_rate': curtailment_rate/100,
            'cf_tech_transposed': cf_tech.T,
            'co2_by_tech': pd.concat([co2_by_tech_reg.groupby(['model', 'Category']).sum().unstack(level='Category')/1e6, co2_target_by_model],axis=1),
            'op_and_vio_costs_by_prop': gen_op_and_vio_costs_reg.groupby(['model', 'property']).sum().unstack(level='property'),
            }      


summary_type = {
             'load_by_reg': 'stacked',
             'pk_load_by_reg':  'clustered',
             'pk_netload_by_reg': 'clustered',
    
             'line_cap_reg': 'clustered',
             'line_net_exports_reg': 'clustered',
             'net_gen_by_reg':'clustered',
    
             'use_by_reg': 'stacked',
             'gen_by_tech': 'stacked', 
             'gen_by_WEOtech': 'stacked', 
             'gen_by_reg': 'stacked', 
             'net_gen_by_reg':'clustered',
             'gen_cap_by_reg': 'stacked',
             'gen_cap_by_tech': 'stacked',
             'gen_cap_by_WEOtech': 'stacked',
             'vre_by_reg_byGen': 'clustered',
             'vre_by_reg_byAv': 'clustered',
             're_by_reg': 'clustered',
             'curtailment_rate': 'clustered',
            'cf_tech_transposed': 'clustered',
            'co2_by_tech': 'stacked',
            'op_and_vio_costs_by_prop' : 'stacked'
}
        
summary_units = {
             'load_by_reg': 'TWh',
             'use_by_reg': 'TWh',
             'gen_by_tech': 'TWh', 
             'gen_by_WEOtech': 'TWh', 
             'gen_by_reg': 'TWh', 
             'net_gen_by_reg':'TWh',
             'gen_cap_by_reg': 'GW',
             'gen_cap_by_tech': 'GW',
             'gen_cap_by_WEOtech': 'GW',
             'vre_by_reg_byGen': '%',
             'vre_by_reg_byAv': '%',
             're_by_reg': '%',    
             'pk_load_by_reg':  'GW',
             'pk_netload_by_reg': 'GW',
    
             'line_cap_reg': 'GW',
             'line_net_exports_reg': 'TWh',
             'curtailment_rate': '%',
            'cf_tech_transposed': '%',
            'co2_by_tech': 'million tonnes',
            'op_and_vio_costs_by_prop' : 'USDm'
}


In [ ]:
### Plot 2: High-level summary
fig_path = os.path.join(save_dir_plots,'plot2b_high_level_summary_plots.xlsx')

with pd.ExcelWriter(fig_path, engine='xlsxwriter') as writer:
    
    for i in summary_cols.keys():   
        if summary_cols[i].shape[0] == 0:
            print('Émpty dataframe for: {}'.format(i))
        else:
            
            if 'UKR' in summary_cols[i].columns:
                total_col = 'UKR'
            elif 'co2' in i:
                total_col= summary_cols[i].columns[-1]
            else:
                total_col = None            
            
            write_xlsx_column(df=summary_cols[i], writer=writer, sheet_name=i, subtype=summary_type[i], units=summary_units[i], palette=combined_palette, total_scatter_col=total_col, total_fill=iea_palette['yl'])   
    
    

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def get_plot_table(df):

    data = df.values

    columns = df.columns
    rows = summary_cols[i].index

    # Get some pastel shades for the colors
    colors = plt.cm.BuPu(np.linspace(0, 0.5, len(rows)))
    n_rows = len(data)

    index = np.arange(len(columns)) + 0.3
    bar_width = 0.4

    # Initialize the vertical-offset for the stacked bar chart.
    y_offset = np.array([0.0] * len(columns))

    # Plot bars and create text labels for the table
    cell_text = []
    for row in range(n_rows):
        cell_text.append(['{:.2f}'.format(x) for x in data_row])
    # Reverse colors and text labels to display the last value at the top.
    colors = colors[::-1]
    cell_text.reverse()

#     # Add a table at the bottom of the axes
#     the_table = plt.table(cellText=cell_text,
#                           rowLabels=rows,
#                           rowColours=colors,
#                           colLabels=columns,
#                           loc='bottom')

    return cell_text

In [ ]:
import matplotlib
matplotlib.rcParams.update(matplotlib.rcParamsDefault)

In [ ]:
import datetime
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt

# Create the PdfPages object to which we will save the pages:
# The with statement makes sure that the PdfPages object is closed properly at
# the end of the block, even if an Exception occurs.
with PdfPages(os.path.join(save_dir,'summary_info.pdf')) as pdf:
    
    ### Plot 2: Annual summary plots by columnn

    
    for i in summary_cols.keys():
        
        
        if summary_type[i] == 'stacked':
            stack=True
        else:
            stack=False
           
        
        if summary_cols[i].shape[0] == 0:
            print('Empty dataframe for: {}'.format(i))
            
            
        elif summary_cols[i].shape[1] > 16:
            print("Skipping {}".format(i))
            fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 5))
            summary_cols[i].plot(ax=ax, kind='bar', stacked=stack,cmap=tab20bc, legend=True)
            plt.title('{} - {}'.format(i, summary_units[i]))
            ax.legend(loc=(1.1, 0.1))
            plt.rc('text', usetex=False)
            pdf.savefig(bbox_inches="tight")  # saves the current figure into a pdf page
            plt.close()
            ##### Table portion, new pagew
            
            
        else:
            fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(9, 5),constrained_layout=True)
            summary_cols[i].plot(ax=ax1, kind='bar', stacked=stack,cmap=iea_cmap_16)

            plt.title('{} - {}'.format(i, summary_units[i]))
            ax1.legend(loc=(1.1, 0.1))

            cell_text = []
            data = summary_cols[i].values
            for row in range(0, summary_cols[i].shape[0]):
                cell_text.append(['{:.2f}'.format(data[row,col]) for col in np.arange(0,summary_cols[i].shape[1]) ])

            ax2.table(cellText=cell_text, colLabels=summary_cols[i].columns, rowLabels=summary_cols[i].index, bbox=[0,0,1,1])
            ax2.axis("off")
            plt.rc('text', usetex=False)
            pdf.savefig(bbox_inches="tight")  # saves the current figure into a pdf page


            plt.close()
        

    # We can also set the file's metadata via the PdfPages object:
    d = pdf.infodict()
    d['Title'] = 'UKR PSE/NZE 2021 Results'
    d['Author'] = 'Craig Hart'
    d['Subject'] = 'Quick view of key results'
    d['Keywords'] = 'Indonesia power system flexibility'
    d['CreationDate'] = datetime.datetime.today()
    d['ModDate'] = datetime.datetime.today()

#### Geo_stuff

## Read in admin layer (state-level) shapes

In [149]:
##  Issues with the CRS disappearing and the concatenation of multiple read shapefiles without a CRS implicit
from pyproj import CRS

default_crs = CRS('EPSG:4326')
utm_crs = CRS('EPSG:3857')

In [150]:
gis_root_folder = "Y:/GIS/ASEAN/ASEAN-GIS/"

adm1_shp_files = get_files(gis_root_folder, "shp", "_adm1")
adm0_shp_files = get_files(gis_root_folder, "shp", "_adm0")

In [151]:
ukr_adm = gpd.GeoDataFrame(pd.concat([gpd.read_file(shp)for shp in adm1_shp_files if 'UKR' in shp]), crs= default_crs)
asean_adm = gpd.GeoDataFrame(pd.concat([gpd.read_file(shp)for shp in adm0_shp_files if 'UKR' not in shp]), crs= default_crs)

In [152]:
ukr_adm_info = pd.read_csv('Y:/GIS/Ukraine/thailand_modelling_regs.csv')
ukr_regs = ukr_adm_info.PLEXOS_GRP2.unique()

In [153]:
ukr_adm = gpd.GeoDataFrame(pd.merge(ukr_adm, ukr_adm_info, on='NAME_1'))

In [154]:
# import warnings
# warnings.simplefilter(action='ignore', category=UserWarning)
# %matplotlib inline

# fig, ax = plt.subplots(1, figsize=(15, 7.5))


# ukr_adm.plot(ax=ax, column='PLEXOS_GRP1', cmap='RdYlGn', alpha = 0.8);

# ## Map formatting
# # apply_iea_style(ax, tick_spacing=5)


### VIZ

#### Read in packages and create palettes

In [155]:
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib import colors

In [156]:
from matplotlib.patches import Patch
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

### Function to apply IEA map style (based on FoC CDD map)

def apply_iea_style(ax, tick_spacing=20, font_type='Arial', fontsize=10, grid_lines=True):
    ### Remove frame
    sns.despine(ax=ax, left=True, top=True, bottom= True, right=True)

    plt.rcParams['axes.labelsize'] = fontsize
    plt.rcParams['axes.titlesize'] = fontsize
    plt.rcParams['ytick.labelsize'] = fontsize

    plt.rcParams['font.size'] = fontsize
    plt.rcParams['font.family'] = font_type
    
#     plt.rcParams['legend.fontsize'] = fontsize
#     plt.rcParams['legend.title_fontsize'] = fontsize

    
    plot_ax = ax.get_figure().get_axes()[0]
    plot_ax.grid(False)
    
    ## Add gridlines
    if grid_lines==True:
        plot_ax.grid(b=True, axis='y', color='gray', linestyle='--')
        plot_ax.grid(clip_on=False)
        
        ## Force y-axis tick spacing to each 20deg
        ax.yaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))

        plot_ax.set_yticklabels(['{}\xb0 N'.format(int(y)) if y > 0 else '{}\xb0 S'.format(int(abs(y))) if y < 0 
                                     else '{}\xb0'.format(int(y)) for y in plot_ax.get_yticks()])
    else:
        ax.get_yaxis().set_visible(False)

    ## Hide x-axis
    ax.get_xaxis().set_visible(False)

    ## Plot gridlines etc below the map
    ax.set_axisbelow(True)

In [157]:
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib import colors

In [158]:
import matplotlib
import numpy as np
def alpha_blending(hex_color, alpha) :
    """ alpha blending as if on the white background.
    """
    foreground_tuple  = matplotlib.colors.hex2color(hex_color)
    foreground_arr = np.array(foreground_tuple)
    final = tuple( (1. -  alpha) + foreground_arr*alpha )
    return(final)

#### Plots

` Alternative categorisation with 2/3 IPPs with bundled/unbundled combined`

In [159]:
global_oceans = gpd.read_file('Y:/GIS/global/oceans/ne_10m_ocean.shp').to_crs(default_crs)

In [160]:
x0 = np.floor(ukr_adm.total_bounds[0] - 2)
x1 = np.ceil(ukr_adm.total_bounds[2] + 2)
y0 = np.floor(ukr_adm.total_bounds[1])
y1 = np.ceil(ukr_adm.total_bounds[3])

In [161]:
fig, ax = plt.subplots(1, figsize=(20,7.5))

ax.set_facecolor(color=alpha_blending(iea_palette['bl'], alpha=0.5));

# global_oceans.plot(ax=ax, color=iea_palette['bl'], alpha=0.5)

ukr_adm.plot(ax=ax, column='PLEXOS_GRP2', cmap=iea_cmap_regs, edgecolor=iea_palette['grey20'], linewidth=0.2,
                          missing_kwds={"color": iea_palette['grey10'], "label": "Missing values"})
asean_adm.plot(ax=ax, color=iea_palette['grey10'], edgecolor=iea_palette['grey10'], linewidth=0.2)


legend_elems = [ Patch(facecolor=iea_cmap_regs(i), label= reg) for i, reg in enumerate(np.sort(ukr_adm.PLEXOS_GRP2.unique()))]
               

leg = ax.legend(handles=legend_elems, loc=(0.8,0.02), title='Region:', framealpha=1);
ax.add_artist(leg)


### IEA style formatting of map
ax.set_xlim(x0,x1)
ax.set_ylim(y0,y1)
apply_iea_style(ax=ax, tick_spacing=20, font_type='Arial', fontsize=10, grid_lines=False)

fig.savefig(os.path.join(save_dir_plots,'UKR_modelled_regions.png'), dpi=300, bbox_inches="tight")